In [1]:
from __future__ import annotations

import re
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Union, Callable

from rdflib import Graph, Literal, Namespace, URIRef

AG = Namespace("http://www.semanticweb.org/AgentProgramParams/")


# -----------------------------
# Tracer
# -----------------------------

@dataclass
class Tracer:
    enabled: bool = True
    print_live: bool = True
    lines: List[str] = None

    def __post_init__(self):
        if self.lines is None:
            self.lines = []

    def log(self, msg: str) -> None:
        if not self.enabled:
            return
        self.lines.append(msg)
        if self.print_live:
            print(msg)


# -----------------------------
# 1) FBD Python Export Analyse
# -----------------------------

@dataclass
class FbdNode:
    var: str
    func: Optional[str] = None
    args: Optional[List["FbdNode"]] = None

    def to_dict(self) -> Dict[str, Any]:
        if not self.func:
            return {"var": self.var}
        return {"var": self.var, "func": self.func, "args": [a.to_dict() for a in (self.args or [])]}


def _parse_fbd_call_assignments(fbd_py_code: str, trace: Optional[Tracer] = None) -> Dict[str, Tuple[str, List[str]]]:
    pattern = r"^\s*(V_\d+)\s*=\s*([A-Za-z_][A-Za-z0-9_]*)\(([^)]*)\)\s*$"
    m: Dict[str, Tuple[str, List[str]]] = {}
    for mm in re.finditer(pattern, fbd_py_code, flags=re.M):
        var, func, argstr = mm.group(1), mm.group(2), mm.group(3)
        args = [a.strip() for a in argstr.split(",") if a.strip()]
        m[var] = (func, args)
    if trace:
        trace.log(f"[FBD] Parsed call assignments: {len(m)} entries (V_* = FUNC(args...))")
    return m


def _find_final_state_assignment_var(fbd_py_code: str, state_name: str = "D2", trace: Optional[Tracer] = None) -> Optional[str]:
    m = re.search(rf"^\s*{re.escape(state_name)}\s*=\s*(V_\d+)\s*$", fbd_py_code, flags=re.M)
    if trace:
        if m:
            trace.log(f"[FBD] Found final state assignment: {state_name} = {m.group(1)}")
        else:
            trace.log(f"[FBD] Did not find final state assignment for {state_name}")
    return m.group(1) if m else None


def _build_fbd_tree(
    var: str,
    assigns: Dict[str, Tuple[str, List[str]]],
    depth: int = 0,
    max_depth: int = 25,
) -> FbdNode:
    if depth >= max_depth:
        return FbdNode(var=var, func="MAX_DEPTH", args=[])

    if var not in assigns:
        return FbdNode(var=var)

    func, args = assigns[var]
    return FbdNode(var=var, func=func, args=[_build_fbd_tree(a, assigns, depth + 1, max_depth) for a in args])


def trace_d2_set_reset_from_fbd(fbd_py_code: str, state_name: str = "D2", trace: Optional[Tracer] = None) -> Dict[str, Any]:
    assigns = _parse_fbd_call_assignments(fbd_py_code, trace=trace)
    d2_from = _find_final_state_assignment_var(fbd_py_code, state_name=state_name, trace=trace)
    if not d2_from:
        return {"error": f"Konnte '{state_name} = V_...' im Code nicht finden."}

    if d2_from not in assigns:
        return {"error": f"'{state_name}' kommt von '{d2_from}', aber '{d2_from}' hat keine Call-Zuweisung im Code."}

    func, args = assigns[d2_from]
    if trace:
        trace.log(f"[FBD] {d2_from} is computed by {func} with args {args}")

    if not (func.startswith("RS_") and len(args) >= 2):
        full_tree = _build_fbd_tree(d2_from, assigns).to_dict()
        if trace:
            trace.log("[FBD] Assignment is not RS_* (set reset). Returning generic dependency tree.")
        return {"state": state_name, "assigned_from": d2_from, "tree": full_tree}

    set_var, reset_var = args[0], args[1]
    if trace:
        trace.log(f"[FBD] Interpreting {func}(SetCond, ResetCond) -> set={set_var}, reset={reset_var}")

    set_tree = _build_fbd_tree(set_var, assigns).to_dict()
    reset_tree = _build_fbd_tree(reset_var, assigns).to_dict()

    return {
        "state": state_name,
        "assigned_from": d2_from,
        "rs_block": func,
        "set_var": set_var,
        "reset_var": reset_var,
        "set_tree": set_tree,
        "reset_tree": reset_tree,
    }


# -----------------------------
# 2) KG Traversal ohne SPARQL
# -----------------------------

def load_graph(ttl_path: str, trace: Optional[Tracer] = None) -> Graph:
    if trace:
        trace.log(f"[KG] Loading TTL: {ttl_path}")
    g = Graph()
    g.parse(ttl_path, format="turtle")
    if trace:
        trace.log(f"[KG] Loaded graph with {len(g)} triples")
    return g


def find_pou_by_name(graph: Graph, pou_name: str, trace: Optional[Tracer] = None) -> Optional[URIRef]:
    dp_hasPOUName = AG["dp_hasPOUName"]
    matches = list(graph.subjects(dp_hasPOUName, Literal(pou_name)))
    if trace:
        trace.log(f"[KG] find_pou_by_name('{pou_name}') -> {len(matches)} matches")
    return matches[0] if matches else None

from rdflib.namespace import RDF, XSD

def _literal_is_true(lit: Optional[Literal]) -> bool:
    if lit is None:
        return False
    # rdflib macht aus xsd:boolean oft Python bool
    try:
        if isinstance(lit.toPython(), bool):
            return bool(lit.toPython())
    except Exception:
        pass
    # fallback string
    s = str(lit).strip().lower()
    return s in ("true", "1", "yes")


def find_gemma_pous(graph: Graph, trace: Optional[Tracer] = None) -> List[Tuple[URIRef, str]]:
    """
    Entspricht SPARQL:
      ?pou a ag:class_CustomFBType ;
           ag:dp_isGEMMAStateMachine true .
      optional: ?pou ag:dp_hasPOUName ?name .
    Gibt Liste (pou_uri, pou_name) zurück, deterministisch sortiert nach Name.
    """
    dp_is_gemma = AG["dp_isGEMMAStateMachine"]
    dp_has_name = AG["dp_hasPOUName"]
    cls_custom = AG["class_CustomFBType"]

    hits: List[Tuple[URIRef, str]] = []

    for pou in graph.subjects(RDF.type, cls_custom):
        lit = next(graph.objects(pou, dp_is_gemma), None)
        if _literal_is_true(lit):
            name_lit = next(graph.objects(pou, dp_has_name), None)
            name = str(name_lit) if name_lit else str(pou)
            hits.append((pou, name))

    hits.sort(key=lambda x: x[1])  # deterministisch
    if trace:
        trace.log(f"[KG] GEMMA candidates (CustomFBType + isGEMMAStateMachine=true): {len(hits)}")
        for pou, name in hits[:10]:
            trace.log(f"[KG]   GEMMA POU: {name} -> {pou}")
    return hits

def get_pou_code(graph: Graph, pou_uri: URIRef, trace: Optional[Tracer] = None) -> Optional[str]:
    dp_hasPOUCode = AG["dp_hasPOUCode"]
    code_lit = next(graph.objects(pou_uri, dp_hasPOUCode), None)
    if trace:
        if code_lit:
            s = str(code_lit)
            trace.log(f"[KG] dp_hasPOUCode found for {pou_uri} (length={len(s)} chars)")
        else:
            trace.log(f"[KG] No dp_hasPOUCode for {pou_uri}")
    return str(code_lit) if code_lit else None


def get_port_by_name(graph: Graph, pou_uri: URIRef, port_name: str, trace: Optional[Tracer] = None) -> Optional[URIRef]:
    op_hasPort = AG["op_hasPort"]
    dp_hasPortName = AG["dp_hasPortName"]
    ports = list(graph.objects(pou_uri, op_hasPort))
    if trace:
        trace.log(f"[KG] POU {pou_uri} has {len(ports)} ports. Searching for port_name='{port_name}'")
    for port in ports:
        name = next(graph.objects(port, dp_hasPortName), None)
        if str(name) == port_name:
            if trace:
                trace.log(f"[KG] Found port '{port_name}' -> {port}")
            return port
    if trace:
        trace.log(f"[KG] Port '{port_name}' not found in POU {pou_uri}")
    return None


def resolve_input_to_upstream_output(
    graph: Graph,
    pou_uri: URIRef,
    input_port_name: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    op_assignsToPort = AG["op_assignsToPort"]
    op_assignsFrom = AG["op_assignsFrom"]
    op_instantiatesPort = AG["op_instantiatesPort"]
    op_hasPort = AG["op_hasPort"]

    dp_hasPOUName = AG["dp_hasPOUName"]
    dp_hasPOUCode = AG["dp_hasPOUCode"]
    dp_hasPortName = AG["dp_hasPortName"]
    dp_hasPortDirection = AG["dp_hasPortDirection"]

    if trace:
        trace.log(f"[KG] Resolving input '{input_port_name}' in POU {pou_uri}")

    port = get_port_by_name(graph, pou_uri, input_port_name, trace=trace)
    if not port:
        return {"error": f"Port '{input_port_name}' nicht in POU {pou_uri} gefunden."}

    port_dir = next(graph.objects(port, dp_hasPortDirection), None)
    if trace:
        trace.log(f"[KG] Input port direction: {port_dir}")

    assigns = list(graph.subjects(op_assignsToPort, port))
    if trace:
        trace.log(f"[KG] op_assignsToPort count: {len(assigns)}")

    if not assigns:
        return {
            "pou_uri": str(pou_uri),
            "input_port": str(port),
            "input_port_dir": str(port_dir) if port_dir else None,
            "note": "Kein op_assignsToPort gefunden. Port könnte unverdrahtet oder aus Default kommen.",
        }

    assignment = assigns[0]
    if trace:
        trace.log(f"[KG] Using assignment: {assignment}")

    caller_port_instance = next(graph.objects(assignment, op_assignsFrom), None)
    if trace:
        trace.log(f"[KG] op_assignsFrom -> port_instance: {caller_port_instance}")
    if not caller_port_instance:
        return {"error": f"Assignment {assignment} hat kein op_assignsFrom."}

    caller_port = next(graph.objects(caller_port_instance, op_instantiatesPort), None)
    if trace:
        trace.log(f"[KG] port_instance op_instantiatesPort -> caller_port: {caller_port}")
    if not caller_port:
        return {"error": f"PortInstance {caller_port_instance} hat kein op_instantiatesPort."}

    caller_port_name = next(graph.objects(caller_port, dp_hasPortName), None)
    caller_port_dir = next(graph.objects(caller_port, dp_hasPortDirection), None)
    if trace:
        trace.log(f"[KG] caller_port_name={caller_port_name} caller_port_dir={caller_port_dir}")

    caller_pous = list(graph.subjects(op_hasPort, caller_port))
    if trace:
        trace.log(f"[KG] caller_pous owning caller_port: {len(caller_pous)}")
    if not caller_pous:
        return {"error": f"Kein CallerPOU gefunden, das caller_port {caller_port} besitzt."}

    caller_pou = caller_pous[0]
    caller_pou_name = next(graph.objects(caller_pou, dp_hasPOUName), None)
    caller_code = next(graph.objects(caller_pou, dp_hasPOUCode), None)
    if trace:
        trace.log(f"[KG] caller_pou={caller_pou} name={caller_pou_name}")
        trace.log(f"[KG] caller_code length={len(str(caller_code)) if caller_code else 0}")

    return {
        "pou_uri": str(pou_uri),
        "input_port_name": input_port_name,
        "input_port_dir": str(port_dir) if port_dir else None,
        "assignment": str(assignment),
        "caller_port_name": str(caller_port_name) if caller_port_name else None,
        "caller_port_dir": str(caller_port_dir) if caller_port_dir else None,
        "caller_pou_uri": str(caller_pou),
        "caller_pou_name": str(caller_pou_name) if caller_pou_name else None,
        "caller_code": str(caller_code) if caller_code else None,
    }

# -----------------------------
# 2b) GEMMA States + Auswahl-Logik für OR/AND per last_state
# -----------------------------

def gemma_state_list() -> List[str]:
    """
    Klassische GEMMA Zustände:
      A1..A7, F1..F6, D1..D3
    Quelle: diverse GEMMA-Unterlagen / Papers.
    """
    A = [f"A{i}" for i in range(1, 8)]
    F = [f"F{i}" for i in range(1, 7)]
    D = [f"D{i}" for i in range(1, 4)]
    return A + F + D


def _is_internal_v(var: str) -> bool:
    return bool(re.match(r"^V_\d+$", var))


def _collect_leaf_tokens(node: Dict[str, Any]) -> List[str]:
    """
    Sammelt alle leaf 'var' Tokens aus deinem to_dict()-Tree.
    """
    out: List[str] = []
    if "func" not in node:
        out.append(node["var"])
        return out
    for a in node.get("args", []) or []:
        out.extend(_collect_leaf_tokens(a))
    return out


def _collect_gemma_states_in_subtree(node: Dict[str, Any], gemma_states: set) -> set:
    leaves = _collect_leaf_tokens(node)
    return {x for x in leaves if x in gemma_states}


def _find_first_or_node(node: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """
    Findet den ersten OR_* Knoten in DFS-Reihenfolge.
    """
    func = node.get("func")
    if func and func.startswith("OR_"):
        return node
    for a in node.get("args", []) or []:
        found = _find_first_or_node(a)
        if found:
            return found
    return None


def _node_to_expr(node: Dict[str, Any]) -> str:
    """
    String-Rekonstruktion: AND_5(Auto_Stoerung, F1)
    """
    if "func" not in node:
        return node["var"]
    args = ", ".join(_node_to_expr(a) for a in (node.get("args") or []))
    return f"{node['func']}({args})"


def infer_suspected_input_port_from_last_state(
    d2_trace: Dict[str, Any],
    last_state: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    """
    Erwartet d2_trace aus trace_d2_set_reset_from_fbd(...)
    Wählt deterministisch OR-Ast anhand last_state und gibt die weiter zu verfolgende Variable zurück.
    """
    gemma_states = set(gemma_state_list())

    if "error" in d2_trace:
        return {"error": d2_trace["error"]}

    set_tree = d2_trace.get("set_tree")
    if not set_tree:
        return {"error": "Kein set_tree im d2_trace vorhanden."}

    # 1) OR-Stelle finden
    or_node = _find_first_or_node(set_tree)
    if not or_node:
        return {
            "note": "Kein OR_* in set_tree gefunden. Keine automatische Branch-Auswahl implementiert.",
            "set_tree_expr": _node_to_expr(set_tree),
        }

    # 2) Alle OR-Äste ausgeben
    branches = or_node.get("args") or []
    branch_infos = []
    for idx, b in enumerate(branches, start=1):
        states_in_branch = _collect_gemma_states_in_subtree(b, gemma_states)
        branch_infos.append(
            {
                "idx": idx,
                "expr": _node_to_expr(b),
                "gemma_states_in_branch": sorted(states_in_branch),
                "contains_last_state": (last_state in states_in_branch),
            }
        )

    if trace:
        trace.log(f"[AUTO] Found OR node: {_node_to_expr(or_node)}")
        trace.log("[AUTO] OR branches:")
        for bi in branch_infos:
            trace.log(f"  - b{bi['idx']}: {bi['expr']}")
            trace.log(f"      states={bi['gemma_states_in_branch']} contains_last={bi['contains_last_state']}")

    # 3) deterministisch per last_state auswählen
    matching = [bi for bi in branch_infos if bi["contains_last_state"]]
    if len(matching) != 1:
        return {
            "or_branches": branch_infos,
            "error": f"Nicht deterministisch: {len(matching)} passende OR-Äste für last_state='{last_state}'.",
        }

    chosen = matching[0]
    chosen_node = branches[chosen["idx"] - 1]

    # 4) Kandidatenvariablen extrahieren:
    #    Leaf tokens im gewählten Ast minus GEMMA-States minus interne V_* (optional)
    leaves = _collect_leaf_tokens(chosen_node)
    candidates = []
    for tok in leaves:
        if tok in gemma_states:
            continue
        if _is_internal_v(tok):
            continue
        # kleine Filter: TRUE/FALSE o.ä. (falls auftauchen)
        if tok.upper() in ("TRUE", "FALSE"):
            continue
        candidates.append(tok)

    # uniq, deterministisch
    uniq: List[str] = []
    for c in candidates:
        if c not in uniq:
            uniq.append(c)

    if trace:
        trace.log(f"[AUTO] Chosen branch: {chosen['expr']}")
        trace.log(f"[AUTO] Next trace candidates (non-GEMMA leaves): {uniq}")

    return {
        "or_branches": branch_infos,
        "chosen_branch": chosen,
        "next_trace_candidates": uniq,
    }

# -----------------------------
# 3) ST Analyse: "var := TRUE;"
# -----------------------------

def extract_true_set_lines_st(
    st_code: str,
    var_name: str,
    with_line_numbers: bool = True,
    trace: Optional[Tracer] = None,
) -> List[Union[str, Tuple[int, str]]]:
    rx = re.compile(rf"^\s*{re.escape(var_name)}\s*:=\s*TRUE\s*;?\s*$")

    out: List[Union[str, Tuple[int, str]]] = []
    for i, line in enumerate(st_code.splitlines(), start=1):
        if rx.match(line):
            out.append((i, line) if with_line_numbers else line)

    if trace:
        trace.log(f"[ST] Searching TRUE assignments for '{var_name}' -> found {len(out)} lines")
        for item in out:
            if isinstance(item, tuple):
                trace.log(f"[ST] match line {item[0]}: {item[1].strip()}")
            else:
                trace.log(f"[ST] match: {item.strip()}")

    return out

# -----------------------------
# 3b) ST Analyse: IF-Bedingungen zu "var := TRUE;"
# -----------------------------

def _strip_st_comments(st: str) -> str:
    """
    Entfernt:
    - // line comments
    - (* block comments *)
    """
    st = re.sub(r"\(\*.*?\*\)", "", st, flags=re.S)
    st = re.sub(r"//.*?$", "", st, flags=re.M)
    return st


def extract_set_conditions_st(
    st_code: str,
    var_name: str,
    value_literal: str,  # "TRUE" oder "FALSE"
    trace: Optional[Tracer] = None,
    ) -> List[Dict[str, Any]]:
    """
    Findet Zeilen "var_name := <value_literal>;" und gibt die dazu aktiven IF-Bedingungen (Stack) zurück.
    """

    clean = _strip_st_comments(st_code)
    lines = clean.splitlines()

    rx_if    = re.compile(r"^\s*IF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_elsif = re.compile(r"^\s*ELSIF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_else  = re.compile(r"^\s*ELSE\s*$", flags=re.I)
    rx_end   = re.compile(r"^\s*END_IF\s*;?\s*$", flags=re.I)

    rx_assign = re.compile(
        rf"^\s*{re.escape(var_name)}\s*:=\s*{re.escape(value_literal)}\s*;\s*$",
        flags=re.I,
    )

    if_stack: List[Dict[str, Any]] = []
    results: List[Dict[str, Any]] = []

    def _stack_repr() -> List[str]:
        rep: List[str] = []
        for e in if_stack:
            if e["branch"] == "IF":
                rep.append(f"IF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSIF":
                rep.append(f"ELSIF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSE":
                rep.append(f"ELSE@{e['if_start_line']} (zu IF: {e['if_cond']})")
        return rep

    for i, raw in enumerate(lines, start=1):
        line = raw.strip()
        if not line:
            continue

        m_if = rx_if.match(line)
        if m_if:
            cond = m_if.group(1).strip()
            if_stack.append({"branch": "IF", "cond": cond, "if_start_line": i, "if_cond": cond})
            continue

        m_elsif = rx_elsif.match(line)
        if m_elsif:
            cond = m_elsif.group(1).strip()
            if not if_stack:
                if_stack.append({"branch": "ELSIF", "cond": cond, "if_start_line": i, "if_cond": cond})
            else:
                top = if_stack[-1]
                top["branch"] = "ELSIF"
                top["cond"] = cond
            continue

        if rx_else.match(line):
            if if_stack:
                top = if_stack[-1]
                top["branch"] = "ELSE"
                top["cond"] = ""
            continue

        if rx_end.match(line):
            if if_stack:
                if_stack.pop()
            continue

        if rx_assign.match(line):
            snapshot = _stack_repr()
            results.append(
                {
                    "line_no": i,
                    "assignment": raw.rstrip(),
                    "conditions": snapshot,
                    "conditions_conjunction": " AND ".join(snapshot) if snapshot else "(keine IF-Bedingung im Scope)",
                }
            )

    if trace:
        trace.log(f"[ST] IF-trace for '{var_name} := {value_literal}' -> {len(results)} hits")
        for r in results:
            trace.log(f"[ST] line {r['line_no']}: {r['assignment'].strip()}")
            trace.log(f"[ST]   conditions: {r['conditions_conjunction']}")

    return results

def analyze_var_assignments_st(
    st_code: str,
    var_name: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    """
    Baut eine deterministische Liste aller Zuweisungen var := TRUE/FALSE mit IF-Pfaden.
    Zusätzlich: sortiert nach Zeilennummer und zeigt dir "last write wins"-Sicht.
    """

    true_hits = extract_set_conditions_st(st_code, var_name, "TRUE", trace=None)
    false_hits = extract_set_conditions_st(st_code, var_name, "FALSE", trace=None)

    # Merge + markieren
    merged: List[Dict[str, Any]] = []
    for h in true_hits:
        merged.append({**h, "value": "TRUE"})
    for h in false_hits:
        merged.append({**h, "value": "FALSE"})

    merged.sort(key=lambda x: x["line_no"])  # deterministisch nach Code-Reihenfolge

    # "Kann überschrieben werden?" rein syntaktisch: gibt es eine spätere Zuweisung?
    for idx, item in enumerate(merged):
        item["has_later_assignment"] = (idx < len(merged) - 1)
        item["later_assignments"] = [
            {"line_no": m["line_no"], "value": m["value"]}
            for m in merged[idx + 1 :]
        ]

    summary = {
        "var_name": var_name,
        "assignment_count": len(merged),
        "last_assignment_in_code": merged[-1] if merged else None,
    }

    if trace:
        trace.log(f"[ST] analyze_var_assignments_st('{var_name}') -> {len(merged)} assignments")
        for m in merged:
            trace.log(f"[ST]   line {m['line_no']}: {var_name} := {m['value']};")
            trace.log(f"[ST]     path: {m['conditions_conjunction']}")
        if merged:
            last = merged[-1]
            trace.log(f"[ST] last assignment in code: line {last['line_no']} -> {last['value']}")

    return {"summary": summary, "assignments": merged}


# -----------------------------
# 4) High-Level Helper mit Log Kette
# -----------------------------

def find_st_true_set_lines_for_d2_path(
    ttl_path: str,
    gemma_pou_name: str = "",     # kann leer bleiben -> auto
    state_name: str = "",
    suspected_input_port: str = "",
    last_gemma_state_before_failure: str = "",
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    if trace is None:
        trace = Tracer(enabled=True, print_live=True)

    trace.log("[RUN] Starting query chain")

    g = load_graph(ttl_path, trace=trace)

    # --- GEMMA POU dynamisch ermitteln ---
    if not gemma_pou_name:
        trace.log("[RUN] gemma_pou_name not provided -> auto-detect GEMMA POU via KG flags")
        gemma_candidates = find_gemma_pous(g, trace=trace)
        if not gemma_candidates:
            trace.log("[RUN] Abort: No GEMMA CustomFBType found")
            return {"error": "Kein GEMMA CustomFBType mit dp_isGEMMAStateMachine=true gefunden.", "trace_log": trace.lines}

        # deterministisch: nimm den ersten nach Name sortiert
        pou_uri, auto_name = gemma_candidates[0]
        gemma_pou_name = auto_name
        trace.log(f"[RUN] Selected GEMMA POU deterministically: {gemma_pou_name}")
    else:
        pou_uri = find_pou_by_name(g, gemma_pou_name, trace=trace)
        if not pou_uri:
            trace.log("[RUN] Abort: GEMMA POU not found by name")
            return {"error": f"Konnte GEMMA POU '{gemma_pou_name}' nicht im KG finden.", "trace_log": trace.lines}

    fbd_code = get_pou_code(g, pou_uri, trace=trace)
    if not fbd_code:
        trace.log("[RUN] Abort: No dp_hasPOUCode for GEMMA POU")
        return {"error": f"Konnte dp_hasPOUCode für '{gemma_pou_name}' nicht lesen.", "trace_log": trace.lines}

    trace.log(f"[RUN] Tracing state '{state_name}' in FBD Python export")
    d2_trace = trace_d2_set_reset_from_fbd(fbd_code, state_name=state_name, trace=trace)

    if not last_gemma_state_before_failure:
        return {
            "d2_trace": d2_trace,
            "error": "last_gemma_state_before_failure ist leer. Bitte übergeben (z.B. 'F1').",
            "trace_log": trace.lines,
        }

    auto = infer_suspected_input_port_from_last_state(
        d2_trace,
        last_state=last_gemma_state_before_failure,
        trace=trace,
    )

    if "error" in auto:
        return {"d2_trace": d2_trace, "auto_port": auto, "error": auto["error"], "trace_log": trace.lines}

    cands = auto.get("next_trace_candidates", [])
    if not cands:
        return {"d2_trace": d2_trace, "auto_port": auto, "error": "Keine Kandidatenvariable gefunden.", "trace_log": trace.lines}

    suspected_input_port = cands[0]  # deterministisch: erster Kandidat
    trace.log(f"[RUN] Auto-detected suspected_input_port = {suspected_input_port}")

    trace.log(f"[RUN] Resolving input '{suspected_input_port}' upstream in KG")
    origin = resolve_input_to_upstream_output(g, pou_uri, suspected_input_port, trace=trace)

    if "caller_code" not in origin or not origin.get("caller_code"):
        trace.log("[RUN] Abort: No caller_code found for upstream resolution")
        return {"d2_trace": d2_trace, "origin": origin, "error": "Kein caller_code gefunden.", "trace_log": trace.lines}

    caller_port_name = origin.get("caller_port_name")
    if not caller_port_name:
        trace.log("[RUN] Abort: caller_port_name missing from origin")
        return {
            "d2_trace": d2_trace,
            "origin": origin,
            "error": "caller_port_name fehlt. Kann keine ST-Assignments suchen.",
            "trace_log": trace.lines,
        }
    
    st_code = origin["caller_code"]

    trace.log(f"[RUN] Extracting TRUE set lines in caller ST for variable '{caller_port_name}'")
    true_lines = extract_true_set_lines_st(st_code, caller_port_name, with_line_numbers=True, trace=trace)

    trace.log(f"[RUN] Extracting IF-conditions leading to '{caller_port_name} := TRUE;'")
    true_conditions = extract_set_conditions_st(
        st_code,
        var_name=caller_port_name,
        value_literal="TRUE",
        trace=trace,
    )

    trace.log(f"[RUN] Extracting IF-conditions leading to '{caller_port_name} := FALSE;'")
    false_conditions = extract_set_conditions_st(
        st_code,
        var_name=caller_port_name,
        value_literal="FALSE",
        trace=trace,
    )
    trace.log(f"[RUN] Building merged assignment order for '{caller_port_name}' (TRUE/FALSE + last-write view)")
    assignment_analysis = analyze_var_assignments_st(
        st_code,
        var_name=caller_port_name,
        trace=trace,
    )

    trace.log("[RUN] Done")

    return {
        "gemma_pou_name": gemma_pou_name,
        "d2_trace": d2_trace,
        "origin": origin,
        "true_set_lines": true_lines,
        "true_set_conditions": true_conditions,
        "trace_log": trace.lines,
        "last_gemma_state_before_failure": last_gemma_state_before_failure,
        "auto_port": auto,
        "suspected_input_port": suspected_input_port,
        "false_set_conditions": false_conditions,
        "assignment_analysis": assignment_analysis,
    }


# -----------------------------
# Ausführung
# -----------------------------

tr = Tracer(enabled=True, print_live=True)

result = find_st_true_set_lines_for_d2_path(
    ttl_path=r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl",
    gemma_pou_name="",   # auto
    state_name="D2",
    suspected_input_port="",  # wird ja auto-detected
    last_gemma_state_before_failure="F1",  # <-- HIER
    trace=tr,
)

print("\n=== TRUE Set Lines ===")
for item in result.get("true_set_lines", []):
    print(item)

print("\n=== TRUE Set Conditions (IF paths) ===")
for r in result.get("true_set_conditions", []):
    print(f"Line {r['line_no']}: {r['assignment'].strip()}")
    print(f"  Conditions: {r['conditions_conjunction']}")

print("\n=== FALSE Set Conditions (IF paths) ===")
for r in result.get("false_set_conditions", []):
    print(f"Line {r['line_no']}: {r['assignment'].strip()}")
    print(f"  Conditions: {r['conditions_conjunction']}")

print("\n=== Assignment order (merged TRUE/FALSE) ===")
analysis = result.get("assignment_analysis", {})
for a in analysis.get("assignments", []):
    print(f"Line {a['line_no']}: {result['origin']['caller_port_name']} := {a['value']};")
    print(f"  Path: {a['conditions_conjunction']}")
    if a["has_later_assignment"]:
        print(f"  Can be overwritten later by: {a['later_assignments']}")


[RUN] Starting query chain
[KG] Loading TTL: D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl
[KG] Loaded graph with 2530 triples
[RUN] gemma_pou_name not provided -> auto-detect GEMMA POU via KG flags
[KG] GEMMA candidates (CustomFBType + isGEMMAStateMachine=true): 1
[KG]   GEMMA POU: FB_Betriebsarten -> http://www.semanticweb.org/AgentProgramParams/FBType_FB_Betriebsarten
[RUN] Selected GEMMA POU deterministically: FB_Betriebsarten
[KG] dp_hasPOUCode found for http://www.semanticweb.org/AgentProgramParams/FBType_FB_Betriebsarten (length=6448 chars)
[RUN] Tracing state 'D2' in FBD Python export
[FBD] Parsed call assignments: 35 entries (V_* = FUNC(args...))
[FBD] Found final state assignment: D2 = V_40000000017
[FBD] V_40000000017 is computed by RS_4 with args ['V_40000000010', 'V_40000000016']
[FBD] Interpreting RS_4(SetCond, ResetCond) -> set=V_40000000010, reset=V_40000000016
[AUTO] Found OR node: OR_5(AND_5(Auto_Stoerung, F1), AND_6(GVL.DiagnoseRequested, D1, GVL.NotStopp)

In [20]:
from __future__ import annotations

import re
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Set

from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS

AG = Namespace("http://www.semanticweb.org/AgentProgramParams/")

# ---------
# Config
# ---------
TTL_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl"  # <--- dein Pfad
FBTYPE_URI = URIRef("http://www.semanticweb.org/AgentProgramParams/FBType_FB_Automatikbetrieb_F1")
START_EXPR = "pPer.Q"  # <--- darauf fokussieren wir uns

# ---------
# Load KG + ST code
# ---------
g = Graph()
g.parse(TTL_PATH, format="turtle")

st_code_lit = next(g.objects(FBTYPE_URI, AG["dp_hasPOUCode"]), None)
if not st_code_lit:
    raise RuntimeError(f"Kein dp_hasPOUCode für {FBTYPE_URI} gefunden.")
ST_CODE = str(st_code_lit)


# -----------------------------
# Helpers: KG
# -----------------------------
def _first(obj_iter):
    return next(iter(obj_iter), None)

def find_internal_variable_uri(fbtype_uri: URIRef, var_name: str) -> Optional[URIRef]:
    """Findet Var_* im FBType über op_hasInternalVariable/op_usesVariable + dp_hasVariableName=var_name."""
    for pred in (AG["op_hasInternalVariable"], AG["op_usesVariable"]):
        for v in g.objects(fbtype_uri, pred):
            name = _first(g.objects(v, AG["dp_hasVariableName"]))
            if name and str(name) == var_name:
                return v
    return None

def describe_internal_variable(fbtype_uri: URIRef, var_name: str) -> Dict[str, Any]:
    """
    pPer -> Var_* -> dp_hasVariableType (z.B. TP)
         -> op_representsFBInstance -> FBInst_*
         -> op_isInstanceOfFBType -> StandardFBType_TP
         -> rdfs:comment -> Beschreibung
    """
    var_uri = find_internal_variable_uri(fbtype_uri, var_name)
    if not var_uri:
        return {"error": f"Interne Variable '{var_name}' nicht gefunden (kein Var_* via op_hasInternalVariable/op_usesVariable)."}

    var_type = _first(g.objects(var_uri, AG["dp_hasVariableType"]))
    fbinst = _first(g.objects(var_uri, AG["op_representsFBInstance"]))
    fbinst_type = _first(g.objects(fbinst, AG["op_isInstanceOfFBType"])) if fbinst else None
    descr = _first(g.objects(fbinst_type, RDFS.comment)) if fbinst_type else None

    return {
        "var_name": var_name,
        "var_uri": str(var_uri),
        "var_type": str(var_type) if var_type else None,
        "fbinst_uri": str(fbinst) if fbinst else None,
        "fbinst_type_uri": str(fbinst_type) if fbinst_type else None,
        "fbinst_type_comment": str(descr) if descr else None,
    }

def find_port_uri(fbtype_uri: URIRef, port_name: str) -> Optional[URIRef]:
    for p in g.objects(fbtype_uri, AG["op_hasPort"]):
        n = _first(g.objects(p, AG["dp_hasPortName"]))
        if n and str(n) == port_name:
            return p
    return None

def port_default_and_wiring(fbtype_uri: URIRef, port_name: str) -> Dict[str, Any]:
    port_uri = find_port_uri(fbtype_uri, port_name)
    if not port_uri:
        return {"note": f"'{port_name}' ist kein Port von {fbtype_uri}."}

    default_val = _first(g.objects(port_uri, AG["dp_hasDefaultPortValue"]))
    assigns = list(g.subjects(AG["op_assignsToPort"], port_uri))

    return {
        "port_name": port_name,
        "port_uri": str(port_uri),
        "default_value": str(default_val) if default_val else None,
        "is_wired": bool(assigns),
        "assignments_count": len(assigns),
    }


# -----------------------------
# Helpers: ST parse (FB Calls)
# -----------------------------
def _split_top_level_commas(s: str) -> List[str]:
    parts = []
    buf = []
    depth = 0
    in_str = False

    i = 0
    while i < len(s):
        ch = s[i]
        if ch == "'" and (i == 0 or s[i-1] != "\\"):
            in_str = not in_str
            buf.append(ch)
        elif not in_str:
            if ch == "(":
                depth += 1
                buf.append(ch)
            elif ch == ")":
                depth = max(0, depth - 1)
                buf.append(ch)
            elif ch == "," and depth == 0:
                parts.append("".join(buf).strip())
                buf = []
            else:
                buf.append(ch)
        else:
            buf.append(ch)
        i += 1

    tail = "".join(buf).strip()
    if tail:
        parts.append(tail)
    return parts

def find_fb_call_lines(st_code: str, inst_name: str) -> List[Tuple[int, str]]:
    rx = re.compile(rf"^\s*{re.escape(inst_name)}\s*\((.*?)\)\s*;\s*$")
    hits = []
    for i, line in enumerate(st_code.splitlines(), start=1):
        m = rx.match(line)
        if m:
            hits.append((i, line.rstrip()))
    return hits

def parse_fb_call_args(call_line: str, inst_name: str) -> Dict[str, str]:
    # call_line z.B. "pPer(IN := rPer.Q, PT := FaultPulse);"
    m = re.match(rf"^\s*{re.escape(inst_name)}\s*\((.*?)\)\s*;\s*$", call_line)
    if not m:
        return {}
    argstr = m.group(1).strip()
    if not argstr:
        return {}

    args = _split_top_level_commas(argstr)
    mapping: Dict[str, str] = {}
    for a in args:
        # unterstützt ":=" und "=>"
        m_in = re.match(r"^\s*([A-Za-z_]\w*)\s*:=\s*(.+?)\s*$", a)
        m_out = re.match(r"^\s*([A-Za-z_]\w*)\s*=>\s*(.+?)\s*$", a)
        if m_in:
            mapping[m_in.group(1)] = m_in.group(2).strip()
        elif m_out:
            mapping[m_out.group(1)] = m_out.group(2).strip()
    return mapping

def extract_dotted_tokens(expr: str) -> List[str]:
    # findet "rPer.Q", "tPer.Q" etc.
    return re.findall(r"\b[A-Za-z_]\w*\.[A-Za-z_]\w*\b", expr)


# -----------------------------
# Trace Engine: expr wie "pPer.Q"
# -----------------------------
@dataclass
class TraceLine:
    indent: int
    text: str

def trace_expr(expr: str, fbtype_uri: URIRef, st_code: str, *, max_depth: int = 10) -> List[TraceLine]:
    lines: List[TraceLine] = []
    visited: Set[str] = set()

    def rec(e: str, depth: int):
        key = e.strip()
        if key in visited:
            lines.append(TraceLine(depth, f"(loop detected) {key}"))
            return
        visited.add(key)

        if depth > max_depth:
            lines.append(TraceLine(depth, f"(max depth) {key}"))
            return

        # dotted?
        if "." in key:
            inst, port = key.split(".", 1)
            desc = describe_internal_variable(fbtype_uri, inst)
            if "error" in desc:
                lines.append(TraceLine(depth, f"{key}: {desc['error']}"))
                return

            lines.append(TraceLine(depth, f"{key} ist Port '{port}' der FB-Instanz '{inst}' (Typ={desc['var_type']})"))
            if desc.get("fbinst_type_comment"):
                lines.append(TraceLine(depth + 1, f"FBType-Beschreibung: {desc['fbinst_type_comment']}"))

            call_hits = find_fb_call_lines(st_code, inst)
            if not call_hits:
                lines.append(TraceLine(depth + 1, f"Keine Call-Line '{inst}(...)' im ST-Code gefunden."))
                return

            # deterministisch: nimm erste Call-Line
            call_line_no, call_line = call_hits[0]
            lines.append(TraceLine(depth + 1, f"Call-Line @ {call_line_no}: {call_line.strip()}"))
            args = parse_fb_call_args(call_line, inst)

            # Für Output Q können wir nicht "ausrechnen", aber wir können die Eingänge zeigen
            if args:
                for k_arg, v_expr in args.items():
                    lines.append(TraceLine(depth + 2, f"Param {k_arg} = {v_expr}"))
                    for dt in extract_dotted_tokens(v_expr):
                        rec(dt, depth + 3)

                    # plain identifier: prüfen ob es Input-Port des aktuellen FB ist, dann Default/Wiring ausgeben
                    plain = v_expr.strip()
                    if re.fullmatch(r"[A-Za-z_]\w*", plain):
                        port_info = port_default_and_wiring(fbtype_uri, plain)
                        if "port_uri" in port_info:
                            dv = port_info.get("default_value")
                            wired = port_info.get("is_wired")
                            lines.append(TraceLine(depth + 3, f"'{plain}' ist Input-Port von FB_Automatikbetrieb_F1"))
                            lines.append(TraceLine(depth + 4, f"default_value = {dv}"))
                            lines.append(TraceLine(depth + 4, f"is_wired = {wired} (assignments={port_info.get('assignments_count')})"))
                            if not wired:
                                lines.append(TraceLine(depth + 4, "=> nicht verdrahtet, daher wird DefaultValue verwendet (Trace endet hier)."))
            return

        # plain identifier: Port Default/Wiring check
        if re.fullmatch(r"[A-Za-z_]\w*", key):
            port_info = port_default_and_wiring(fbtype_uri, key)
            if "port_uri" in port_info:
                dv = port_info.get("default_value")
                wired = port_info.get("is_wired")
                lines.append(TraceLine(depth, f"'{key}' ist Input-Port von FB_Automatikbetrieb_F1"))
                lines.append(TraceLine(depth + 1, f"default_value = {dv}"))
                lines.append(TraceLine(depth + 1, f"is_wired = {wired} (assignments={port_info.get('assignments_count')})"))
                if not wired:
                    lines.append(TraceLine(depth + 1, "=> nicht verdrahtet, daher wird DefaultValue verwendet (Trace endet hier)."))
            else:
                lines.append(TraceLine(depth, f"'{key}' ist kein (erkennbare) Input-Port hier."))
            return

        lines.append(TraceLine(depth, f"Unbekanntes Token: {key}"))

    rec(expr, 0)
    return lines


# -----------------------------
# Run trace for pPer.Q
# -----------------------------
trace_lines = trace_expr(START_EXPR, FBTYPE_URI, ST_CODE, max_depth=10)

print("=== TRACE START ===")
for tl in trace_lines:
    print("  " * tl.indent + tl.text)
print("=== TRACE END ===")


=== TRACE START ===
pPer.Q ist Port 'Q' der FB-Instanz 'pPer' (Typ=TP)
  FBType-Beschreibung: TP ist ein Impulsbaustein. Eine steigende Flanke an IN startet einen Impuls. Q wird für genau die Dauer PT TRUE, unabhängig davon wie lange IN TRUE bleibt. ET zeigt die verstrichene Impulszeit.
  Call-Line @ 74: pPer(IN := rPer.Q, PT := FaultPulse);
    Param IN = rPer.Q
      rPer.Q ist Port 'Q' der FB-Instanz 'rPer' (Typ=R_TRIG)
        FBType-Beschreibung: R_TRIG erkennt eine steigende Flanke an CLK. Q wird genau für einen Programmscan TRUE, wenn CLK von FALSE auf TRUE wechselt. Damit lassen sich aus einem Signal einzelne Ereignisimpulse ableiten.
        Call-Line @ 73: rPer(CLK := tPer.Q);
          Param CLK = tPer.Q
            tPer.Q ist Port 'Q' der FB-Instanz 'tPer' (Typ=TON)
              FBType-Beschreibung: TON ist eine Einschaltverzögerung. Wenn IN TRUE wird, startet die Zeitmessung. ET zählt die verstrichene Zeit hoch. Sobald ET die Vorgabezeit PT erreicht und IN weiterhin TRUE 

In [22]:
from __future__ import annotations
#Zelle 3
import re
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Union, Set, Callable

from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import RDF, RDFS, XSD

AG = Namespace("http://www.semanticweb.org/AgentProgramParams/")


# -----------------------------
# Tracer
# -----------------------------
@dataclass
class Tracer:
    enabled: bool = True
    print_live: bool = True
    lines: List[str] = None

    def __post_init__(self):
        if self.lines is None:
            self.lines = []

    def log(self, msg: str) -> None:
        if not self.enabled:
            return
        self.lines.append(msg)
        if self.print_live:
            print(msg)


# ==========================================
# TEIL 1: DEIN ORIGINAL-CODE (FBD & ST Analyse)
# ==========================================

@dataclass
class FbdNode:
    var: str
    func: Optional[str] = None
    args: Optional[List["FbdNode"]] = None

    def to_dict(self) -> Dict[str, Any]:
        if not self.func:
            return {"var": self.var}
        return {"var": self.var, "func": self.func, "args": [a.to_dict() for a in (self.args or [])]}

def _parse_fbd_call_assignments(fbd_py_code: str, trace: Optional[Tracer] = None) -> Dict[str, Tuple[str, List[str]]]:
    pattern = r"^\s*(V_\d+)\s*=\s*([A-Za-z_][A-Za-z0-9_]*)\(([^)]*)\)\s*$"
    m: Dict[str, Tuple[str, List[str]]] = {}
    for mm in re.finditer(pattern, fbd_py_code, flags=re.M):
        var, func, argstr = mm.group(1), mm.group(2), mm.group(3)
        args = [a.strip() for a in argstr.split(",") if a.strip()]
        m[var] = (func, args)
    return m

def _find_final_state_assignment_var(fbd_py_code: str, state_name: str = "D2", trace: Optional[Tracer] = None) -> Optional[str]:
    m = re.search(rf"^\s*{re.escape(state_name)}\s*=\s*(V_\d+)\s*$", fbd_py_code, flags=re.M)
    return m.group(1) if m else None

def _build_fbd_tree(var: str, assigns: Dict[str, Tuple[str, List[str]]], depth: int = 0, max_depth: int = 25) -> FbdNode:
    if depth >= max_depth: return FbdNode(var=var, func="MAX_DEPTH", args=[])
    if var not in assigns: return FbdNode(var=var)
    func, args = assigns[var]
    return FbdNode(var=var, func=func, args=[_build_fbd_tree(a, assigns, depth + 1, max_depth) for a in args])

def trace_d2_set_reset_from_fbd(fbd_py_code: str, state_name: str = "D2", trace: Optional[Tracer] = None) -> Dict[str, Any]:
    assigns = _parse_fbd_call_assignments(fbd_py_code, trace=trace)
    d2_from = _find_final_state_assignment_var(fbd_py_code, state_name=state_name, trace=trace)
    if not d2_from: return {"error": f"Konnte '{state_name} = V_...' im Code nicht finden."}
    if d2_from not in assigns: return {"error": f"'{state_name}' kommt von '{d2_from}', aber '{d2_from}' hat keine Call-Zuweisung im Code."}

    func, args = assigns[d2_from]
    if not (func.startswith("RS_") and len(args) >= 2):
        full_tree = _build_fbd_tree(d2_from, assigns).to_dict()
        return {"state": state_name, "assigned_from": d2_from, "tree": full_tree}

    set_var, reset_var = args[0], args[1]
    return {
        "state": state_name, "assigned_from": d2_from, "rs_block": func,
        "set_var": set_var, "reset_var": reset_var,
        "set_tree": _build_fbd_tree(set_var, assigns).to_dict(),
        "reset_tree": _build_fbd_tree(reset_var, assigns).to_dict(),
    }

def load_graph(ttl_path: str, trace: Optional[Tracer] = None) -> Graph:
    g = Graph()
    g.parse(ttl_path, format="turtle")
    return g

def find_pou_by_name(graph: Graph, pou_name: str, trace: Optional[Tracer] = None) -> Optional[URIRef]:
    matches = list(graph.subjects(AG["dp_hasPOUName"], Literal(pou_name)))
    return matches[0] if matches else None

def _literal_is_true(lit: Optional[Literal]) -> bool:
    if lit is None: return False
    try:
        if isinstance(lit.toPython(), bool): return bool(lit.toPython())
    except Exception: pass
    return str(lit).strip().lower() in ("true", "1", "yes")

def find_gemma_pous(graph: Graph, trace: Optional[Tracer] = None) -> List[Tuple[URIRef, str]]:
    """
    Entspricht SPARQL:
      ?pou a ag:class_CustomFBType ;
           ag:dp_isGEMMAStateMachine true .
      optional: ?pou ag:dp_hasPOUName ?name .
    Gibt Liste (pou_uri, pou_name) zurück, deterministisch sortiert nach Name.
    """
    dp_is_gemma = AG["dp_isGEMMAStateMachine"]
    dp_has_name = AG["dp_hasPOUName"]
    cls_custom = AG["class_CustomFBType"]

    hits: List[Tuple[URIRef, str]] = []
    for pou in graph.subjects(RDF.type, cls_custom):
        lit = next(graph.objects(pou, dp_is_gemma), None)
        if _literal_is_true(lit):
            name_lit = next(graph.objects(pou, dp_has_name), None)
            name = str(name_lit) if name_lit else str(pou)
            hits.append((pou, name))

    hits.sort(key=lambda x: x[1])  # deterministisch
    if trace:
        trace.log(f"[KG] GEMMA candidates (CustomFBType + isGEMMAStateMachine=true): {len(hits)}")
        for pou, name in hits[:10]:
            trace.log(f"[KG]   GEMMA POU: {name} -> {pou}")
    return hits

def get_pou_code(graph: Graph, pou_uri: URIRef, trace: Optional[Tracer] = None) -> Optional[str]:
    dp_hasPOUCode = AG["dp_hasPOUCode"]
    code_lit = next(graph.objects(pou_uri, dp_hasPOUCode), None)
    if trace:
        if code_lit:
            s = str(code_lit)
            trace.log(f"[KG] dp_hasPOUCode found for {pou_uri} (length={len(s)} chars)")
        else:
            trace.log(f"[KG] No dp_hasPOUCode for {pou_uri}")
    return str(code_lit) if code_lit else None

def get_port_by_name(graph: Graph, pou_uri: URIRef, port_name: str, trace: Optional[Tracer] = None) -> Optional[URIRef]:
    op_hasPort = AG["op_hasPort"]
    dp_hasPortName = AG["dp_hasPortName"]
    ports = list(graph.objects(pou_uri, op_hasPort))
    if trace:
        trace.log(f"[KG] POU {pou_uri} has {len(ports)} ports. Searching for port_name='{port_name}'")
    for port in ports:
        name = next(graph.objects(port, dp_hasPortName), None)
        if str(name) == port_name:
            if trace:
                trace.log(f"[KG] Found port '{port_name}' -> {port}")
            return port
    if trace:
        trace.log(f"[KG] Port '{port_name}' not found in POU {pou_uri}")
    return None

def resolve_input_to_upstream_output(graph: Graph, pou_uri: URIRef, input_port_name: str, trace: Optional[Tracer] = None) -> Dict[str, Any]:
    op_assignsToPort = AG["op_assignsToPort"]
    op_assignsFrom = AG["op_assignsFrom"]
    op_instantiatesPort = AG["op_instantiatesPort"]
    op_hasPort = AG["op_hasPort"]

    dp_hasPOUName = AG["dp_hasPOUName"]
    dp_hasPOUCode = AG["dp_hasPOUCode"]
    dp_hasPortName = AG["dp_hasPortName"]
    dp_hasPortDirection = AG["dp_hasPortDirection"]

    if trace:
        trace.log(f"[KG] Resolving input '{input_port_name}' in POU {pou_uri}")

    port = get_port_by_name(graph, pou_uri, input_port_name, trace=trace)
    if not port:
        return {"error": f"Port '{input_port_name}' nicht in POU {pou_uri} gefunden."}

    port_dir = next(graph.objects(port, dp_hasPortDirection), None)
    if trace:
        trace.log(f"[KG] Input port direction: {port_dir}")

    assigns = list(graph.subjects(op_assignsToPort, port))
    if trace:
        trace.log(f"[KG] op_assignsToPort count: {len(assigns)}")

    if not assigns:
        return {
            "pou_uri": str(pou_uri),
            "input_port": str(port),
            "input_port_dir": str(port_dir) if port_dir else None,
            "note": "Kein op_assignsToPort gefunden. Port könnte unverdrahtet oder aus Default kommen.",
        }

    assignment = assigns[0]
    if trace:
        trace.log(f"[KG] Using assignment: {assignment}")

    caller_port_instance = next(graph.objects(assignment, op_assignsFrom), None)
    if trace:
        trace.log(f"[KG] op_assignsFrom -> port_instance: {caller_port_instance}")
    if not caller_port_instance:
        return {"error": f"Assignment {assignment} hat kein op_assignsFrom."}

    caller_port = next(graph.objects(caller_port_instance, op_instantiatesPort), None)
    if trace:
        trace.log(f"[KG] port_instance op_instantiatesPort -> caller_port: {caller_port}")
    if not caller_port:
        return {"error": f"PortInstance {caller_port_instance} hat kein op_instantiatesPort."}

    caller_port_name = next(graph.objects(caller_port, dp_hasPortName), None)
    caller_port_dir = next(graph.objects(caller_port, dp_hasPortDirection), None)
    if trace:
        trace.log(f"[KG] caller_port_name={caller_port_name} caller_port_dir={caller_port_dir}")

    caller_pous = list(graph.subjects(op_hasPort, caller_port))
    if trace:
        trace.log(f"[KG] caller_pous owning caller_port: {len(caller_pous)}")
    if not caller_pous:
        return {"error": f"Kein CallerPOU gefunden, das caller_port {caller_port} besitzt."}

    caller_pou = caller_pous[0]
    caller_pou_name = next(graph.objects(caller_pou, dp_hasPOUName), None)
    caller_code = next(graph.objects(caller_pou, dp_hasPOUCode), None)
    if trace:
        trace.log(f"[KG] caller_pou={caller_pou} name={caller_pou_name}")
        trace.log(f"[KG] caller_code length={len(str(caller_code)) if caller_code else 0}")

    return {
        "pou_uri": str(pou_uri),
        "input_port_name": input_port_name,
        "input_port_dir": str(port_dir) if port_dir else None,
        "assignment": str(assignment),
        "caller_port_name": str(caller_port_name) if caller_port_name else None,
        "caller_port_dir": str(caller_port_dir) if caller_port_dir else None,
        "caller_pou_uri": str(caller_pou),
        "caller_pou_name": str(caller_pou_name) if caller_pou_name else None,
        "caller_code": str(caller_code) if caller_code else None,
    }

def gemma_state_list() -> List[str]:
    return [f"A{i}" for i in range(1, 8)] + [f"F{i}" for i in range(1, 7)] + [f"D{i}" for i in range(1, 4)]

def _is_internal_v(var: str) -> bool: return bool(re.match(r"^V_\d+$", var))

def _collect_leaf_tokens(node: Dict[str, Any]) -> List[str]:
    if "func" not in node: return [node["var"]]
    out = []
    for a in node.get("args", []) or []: out.extend(_collect_leaf_tokens(a))
    return out

def _collect_gemma_states_in_subtree(node: Dict[str, Any], gemma_states: set) -> set:
    return {x for x in _collect_leaf_tokens(node) if x in gemma_states}

def _find_first_or_node(node: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    if node.get("func") and node["func"].startswith("OR_"): return node
    for a in node.get("args", []) or []:
        if found := _find_first_or_node(a): return found
    return None

def _node_to_expr(node: Dict[str, Any]) -> str:
    if "func" not in node: return node["var"]
    return f"{node['func']}({', '.join(_node_to_expr(a) for a in (node.get('args') or []))})"

def infer_suspected_input_port_from_last_state(
    d2_trace: Dict[str, Any],
    last_state: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    """
    Erwartet d2_trace aus trace_d2_set_reset_from_fbd(...)
    W?hlt deterministisch OR-Ast anhand last_state und gibt die weiter zu verfolgende Variable zur?ck.
    """
    gemma_states = set(gemma_state_list())

    if "error" in d2_trace:
        return {"error": d2_trace["error"]}

    set_tree = d2_trace.get("set_tree")
    if not set_tree:
        return {"error": "Kein set_tree im d2_trace vorhanden."}

    # 1) OR-Stelle finden
    or_node = _find_first_or_node(set_tree)
    if not or_node:
        return {
            "note": "Kein OR_* in set_tree gefunden. Keine automatische Branch-Auswahl implementiert.",
            "set_tree_expr": _node_to_expr(set_tree),
        }

    # 2) Alle OR-?ste ausgeben
    branches = or_node.get("args") or []
    branch_infos = []
    for idx, b in enumerate(branches, start=1):
        states_in_branch = _collect_gemma_states_in_subtree(b, gemma_states)
        branch_infos.append(
            {
                "idx": idx,
                "expr": _node_to_expr(b),
                "gemma_states_in_branch": sorted(states_in_branch),
                "contains_last_state": (last_state in states_in_branch),
            }
        )

    if trace:
        trace.log(f"[AUTO] Found OR node: {_node_to_expr(or_node)}")
        trace.log("[AUTO] OR branches:")
        for bi in branch_infos:
            trace.log(f"  - b{bi['idx']}: {bi['expr']}")
            trace.log(f"      states={bi['gemma_states_in_branch']} contains_last={bi['contains_last_state']}")

    # 3) deterministisch per last_state ausw?hlen
    matching = [bi for bi in branch_infos if bi["contains_last_state"]]
    if len(matching) != 1:
        return {
            "or_branches": branch_infos,
            "error": f"Nicht deterministisch: {len(matching)} passende OR-?ste f?r last_state='{last_state}'.",
        }

    chosen = matching[0]
    chosen_node = branches[chosen["idx"] - 1]

    # 4) Kandidatenvariablen extrahieren
    leaves = _collect_leaf_tokens(chosen_node)
    candidates = []
    for tok in leaves:
        if tok in gemma_states:
            continue
        if _is_internal_v(tok):
            continue
        if tok.upper() in ("TRUE", "FALSE"):
            continue
        candidates.append(tok)

    # uniq, deterministisch
    uniq: List[str] = []
    for c in candidates:
        if c not in uniq:
            uniq.append(c)

    if trace:
        trace.log(f"[AUTO] Chosen branch: {chosen['expr']}")
        trace.log(f"[AUTO] Next trace candidates (non-GEMMA leaves): {uniq}")

    return {
        "or_branches": branch_infos,
        "chosen_branch": chosen,
        "next_trace_candidates": uniq,
    }


def extract_true_set_lines_st(
    st_code: str,
    var_name: str,
    with_line_numbers: bool = True,
    trace: Optional[Tracer] = None,
) -> List[Union[str, Tuple[int, str]]]:
    rx = re.compile(rf"^\s*{re.escape(var_name)}\s*:=\s*TRUE\s*;?\s*$")

    out: List[Union[str, Tuple[int, str]]] = []
    for i, line in enumerate(st_code.splitlines(), start=1):
        if rx.match(line):
            out.append((i, line) if with_line_numbers else line)

    if trace:
        trace.log(f"[ST] Searching TRUE assignments for '{var_name}' -> found {len(out)} lines")
        for item in out:
            if isinstance(item, tuple):
                trace.log(f"[ST] match line {item[0]}: {item[1].strip()}")
            else:
                trace.log(f"[ST] match: {item.strip()}")

    return out


def _strip_st_comments(st: str) -> str:
    st = re.sub(r"\(\*.*?\*\)", "", st, flags=re.S)
    return re.sub(r"//.*?$", "", st, flags=re.M)

def extract_set_conditions_st(
    st_code: str,
    var_name: str,
    value_literal: str,  # "TRUE" oder "FALSE"
    trace: Optional[Tracer] = None,
) -> List[Dict[str, Any]]:
    """
    Findet Zeilen "var_name := <value_literal>;" und gibt die dazu aktiven IF-Bedingungen (Stack) zur?ck.
    """

    clean = _strip_st_comments(st_code)
    lines = clean.splitlines()

    rx_if    = re.compile(r"^\s*IF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_elsif = re.compile(r"^\s*ELSIF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_else  = re.compile(r"^\s*ELSE\s*$", flags=re.I)
    rx_end   = re.compile(r"^\s*END_IF\s*;?\s*$", flags=re.I)

    rx_assign = re.compile(
        rf"^\s*{re.escape(var_name)}\s*:=\s*{re.escape(value_literal)}\s*;\s*$",
        flags=re.I,
    )

    if_stack: List[Dict[str, Any]] = []
    results: List[Dict[str, Any]] = []

    def _stack_repr() -> List[str]:
        rep: List[str] = []
        for e in if_stack:
            if e["branch"] == "IF":
                rep.append(f"IF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSIF":
                rep.append(f"ELSIF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSE":
                rep.append(f"ELSE@{e['if_start_line']} (zu IF: {e['if_cond']})")
        return rep

    for i, raw in enumerate(lines, start=1):
        line = raw.strip()
        if not line:
            continue

        m_if = rx_if.match(line)
        if m_if:
            cond = m_if.group(1).strip()
            if_stack.append({"branch": "IF", "cond": cond, "if_start_line": i, "if_cond": cond})
            continue

        m_elsif = rx_elsif.match(line)
        if m_elsif:
            cond = m_elsif.group(1).strip()
            if not if_stack:
                if_stack.append({"branch": "ELSIF", "cond": cond, "if_start_line": i, "if_cond": cond})
            else:
                top = if_stack[-1]
                top["branch"] = "ELSIF"
                top["cond"] = cond
            continue

        if rx_else.match(line):
            if if_stack:
                top = if_stack[-1]
                top["branch"] = "ELSE"
                top["cond"] = ""
            continue

        if rx_end.match(line):
            if if_stack:
                if_stack.pop()
            continue

        if rx_assign.match(line):
            snapshot = _stack_repr()
            results.append(
                {
                    "line_no": i,
                    "assignment": raw.rstrip(),
                    "conditions": snapshot,
                    "conditions_conjunction": " AND ".join(snapshot) if snapshot else "(keine IF-Bedingung im Scope)",
                }
            )

    if trace:
        trace.log(f"[ST] IF-trace for '{var_name} := {value_literal}' -> {len(results)} hits")
        for r in results:
            trace.log(f"[ST] line {r['line_no']}: {r['assignment'].strip()}")
            trace.log(f"[ST]   conditions: {r['conditions_conjunction']}")

    return results


def analyze_var_assignments_st(
    st_code: str,
    var_name: str,
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    """
    Baut eine deterministische Liste aller Zuweisungen var := TRUE/FALSE mit IF-Pfaden.
    Zus?tzlich: sortiert nach Zeilennummer und zeigt dir "last write wins"-Sicht.
    """

    true_hits = extract_set_conditions_st(st_code, var_name, "TRUE", trace=None)
    false_hits = extract_set_conditions_st(st_code, var_name, "FALSE", trace=None)

    merged: List[Dict[str, Any]] = []
    for h in true_hits:
        merged.append({**h, "value": "TRUE"})
    for h in false_hits:
        merged.append({**h, "value": "FALSE"})

    merged.sort(key=lambda x: x["line_no"])  # deterministisch nach Code-Reihenfolge

    for idx, item in enumerate(merged):
        item["has_later_assignment"] = (idx < len(merged) - 1)
        item["later_assignments"] = [{"line_no": m["line_no"], "value": m["value"]} for m in merged[idx + 1 :]]

    summary = {
        "var_name": var_name,
        "assignment_count": len(merged),
        "last_assignment_in_code": merged[-1] if merged else None,
    }

    if trace:
        trace.log(f"[ST] analyze_var_assignments_st('{var_name}') -> {len(merged)} assignments")
        for m in merged:
            trace.log(f"[ST]   line {m['line_no']}: {var_name} := {m['value']};")
            trace.log(f"[ST]     path: {m['conditions_conjunction']}")
        if merged:
            last = merged[-1]
            trace.log(f"[ST] last assignment in code: line {last['line_no']} -> {last['value']}")

    return {"summary": summary, "assignments": merged}


def find_st_true_set_lines_for_d2_path(
    ttl_path: str,
    gemma_pou_name: str = "",     # kann leer bleiben -> auto
    state_name: str = "",
    suspected_input_port: str = "",
    last_gemma_state_before_failure: str = "",
    trace: Optional[Tracer] = None,
) -> Dict[str, Any]:
    if trace is None:
        trace = Tracer(enabled=True, print_live=True)

    trace.log("[RUN] Starting query chain")

    g = load_graph(ttl_path, trace=trace)

    # --- GEMMA POU dynamisch ermitteln ---
    if not gemma_pou_name:
        trace.log("[RUN] gemma_pou_name not provided -> auto-detect GEMMA POU via KG flags")
        gemma_candidates = find_gemma_pous(g, trace=trace)
        if not gemma_candidates:
            trace.log("[RUN] Abort: No GEMMA CustomFBType found")
            return {"error": "Kein GEMMA CustomFBType mit dp_isGEMMAStateMachine=true gefunden.", "trace_log": trace.lines}

        pou_uri, auto_name = gemma_candidates[0]
        gemma_pou_name = auto_name
        trace.log(f"[RUN] Selected GEMMA POU deterministically: {gemma_pou_name}")
    else:
        pou_uri = find_pou_by_name(g, gemma_pou_name, trace=trace)
        if not pou_uri:
            trace.log("[RUN] Abort: GEMMA POU not found by name")
            return {"error": f"Konnte GEMMA POU '{gemma_pou_name}' nicht im KG finden.", "trace_log": trace.lines}

    fbd_code = get_pou_code(g, pou_uri, trace=trace)
    if not fbd_code:
        trace.log("[RUN] Abort: No dp_hasPOUCode for GEMMA POU")
        return {"error": f"Konnte dp_hasPOUCode f?r '{gemma_pou_name}' nicht lesen.", "trace_log": trace.lines}

    trace.log(f"[RUN] Tracing state '{state_name}' in FBD Python export")
    d2_trace = trace_d2_set_reset_from_fbd(fbd_code, state_name=state_name, trace=trace)

    if not last_gemma_state_before_failure:
        return {
            "d2_trace": d2_trace,
            "error": "last_gemma_state_before_failure ist leer. Bitte ?bergeben (z.B. 'F1').",
            "trace_log": trace.lines,
        }

    auto = infer_suspected_input_port_from_last_state(
        d2_trace,
        last_state=last_gemma_state_before_failure,
        trace=trace,
    )

    if "error" in auto:
        return {"d2_trace": d2_trace, "auto_port": auto, "error": auto["error"], "trace_log": trace.lines}

    cands = auto.get("next_trace_candidates", [])
    if not cands:
        return {"d2_trace": d2_trace, "auto_port": auto, "error": "Keine Kandidatenvariable gefunden.", "trace_log": trace.lines}

    suspected_input_port = cands[0]  # deterministisch: erster Kandidat
    trace.log(f"[RUN] Auto-detected suspected_input_port = {suspected_input_port}")

    trace.log(f"[RUN] Resolving input '{suspected_input_port}' upstream in KG")
    origin = resolve_input_to_upstream_output(g, pou_uri, suspected_input_port, trace=trace)

    if "caller_code" not in origin or not origin.get("caller_code"):
        trace.log("[RUN] Abort: No caller_code found for upstream resolution")
        return {"d2_trace": d2_trace, "origin": origin, "error": "Kein caller_code gefunden.", "trace_log": trace.lines}

    caller_port_name = origin.get("caller_port_name")
    if not caller_port_name:
        trace.log("[RUN] Abort: caller_port_name missing from origin")
        return {
            "d2_trace": d2_trace,
            "origin": origin,
            "error": "caller_port_name fehlt. Kann keine ST-Assignments suchen.",
            "trace_log": trace.lines,
        }

    st_code = origin["caller_code"]

    trace.log(f"[RUN] Extracting TRUE set lines in caller ST for variable '{caller_port_name}'")
    true_lines = extract_true_set_lines_st(st_code, caller_port_name, with_line_numbers=True, trace=trace)

    trace.log(f"[RUN] Extracting IF-conditions leading to '{caller_port_name} := TRUE;'")
    true_conditions = extract_set_conditions_st(st_code, var_name=caller_port_name, value_literal="TRUE", trace=trace)

    trace.log(f"[RUN] Extracting IF-conditions leading to '{caller_port_name} := FALSE;'")
    false_conditions = extract_set_conditions_st(st_code, var_name=caller_port_name, value_literal="FALSE", trace=trace)

    trace.log(f"[RUN] Building merged assignment order for '{caller_port_name}' (TRUE/FALSE + last-write view)")
    assignment_analysis = analyze_var_assignments_st(st_code, var_name=caller_port_name, trace=trace)

    trace.log("[RUN] Done")

    return {
        "graph": g,
        "gemma_pou_name": gemma_pou_name,
        "d2_trace": d2_trace,
        "origin": origin,
        "true_set_lines": true_lines,
        "true_set_conditions": true_conditions,
        "trace_log": trace.lines,
        "last_gemma_state_before_failure": last_gemma_state_before_failure,
        "auto_port": auto,
        "suspected_input_port": suspected_input_port,
        "false_set_conditions": false_conditions,
        "assignment_analysis": assignment_analysis,
    }


# ==========================================
# TEIL 2: DEIN ORIGINAL-CODE (Deep Trace)
# ==========================================

def _first(obj_iter): return next(iter(obj_iter), None)

def find_internal_variable_uri(graph: Graph, fbtype_uri: URIRef, var_name: str) -> Optional[URIRef]:
    for pred in (AG["op_hasInternalVariable"], AG["op_usesVariable"]):
        for v in graph.objects(fbtype_uri, pred):
            name = _first(graph.objects(v, AG["dp_hasVariableName"]))
            if name and str(name) == var_name: return v
    return None

def describe_internal_variable(graph: Graph, fbtype_uri: URIRef, var_name: str) -> Dict[str, Any]:
    var_uri = find_internal_variable_uri(graph, fbtype_uri, var_name)
    if not var_uri: return {"error": f"Interne Variable '{var_name}' nicht gefunden."}
    var_type = _first(graph.objects(var_uri, AG["dp_hasVariableType"]))
    fbinst = _first(graph.objects(var_uri, AG["op_representsFBInstance"]))
    fbinst_type = _first(graph.objects(fbinst, AG["op_isInstanceOfFBType"])) if fbinst else None
    descr = _first(graph.objects(fbinst_type, RDFS.comment)) if fbinst_type else None
    lang = _first(graph.objects(fbinst_type, AG["dp_hasPOULanguage"])) if fbinst_type else None
    return {
        "var_name": var_name, "var_uri": str(var_uri), "var_type": str(var_type) if var_type else None,
        "fbinst_uri": str(fbinst) if fbinst else None, "fbinst_type_uri": str(fbinst_type) if fbinst_type else None,
        "fbinst_type_comment": str(descr) if descr else None,
        "fbinst_type_language": str(lang) if lang else None,
    }

def find_port_uri(graph: Graph, fbtype_uri: URIRef, port_name: str) -> Optional[URIRef]:
    for p in graph.objects(fbtype_uri, AG["op_hasPort"]):
        n = _first(graph.objects(p, AG["dp_hasPortName"]))
        if n and str(n) == port_name: return p
    return None

def port_default_and_wiring(graph: Graph, fbtype_uri: URIRef, port_name: str) -> Dict[str, Any]:
    port_uri = find_port_uri(graph, fbtype_uri, port_name)
    if not port_uri: return {"note": f"'{port_name}' ist kein Port von {fbtype_uri}."}
    default_val = _first(graph.objects(port_uri, AG["dp_hasDefaultPortValue"]))
    assigns = list(graph.subjects(AG["op_assignsToPort"], port_uri))
    return {
        "port_name": port_name, "port_uri": str(port_uri), "default_value": str(default_val) if default_val else None,
        "is_wired": bool(assigns), "assignments_count": len(assigns), "assignment_uris": [str(a) for a in assigns],
    }

def get_fb_port_direction(graph: Graph, fbtype_uri: URIRef, port_name: str) -> Optional[str]:
    port_uri = find_port_uri(graph, fbtype_uri, port_name)
    if not port_uri:
        return None
    direction = _first(graph.objects(port_uri, AG["dp_hasPortDirection"]))
    return str(direction) if direction else None


def _find_symbol_roles_in_pou(graph: Graph, pou_uri: URIRef, symbol_name: str) -> Dict[str, Any]:
    """
    Best-effort \"Was ist dieses Symbol im Kontext des POU?\".

    Motivation (dein Beispiel):
      - 'Automatikbetrieb_Starten' wird im ST-Code wie eine Variable referenziert,
        ist aber im KG ein Input-Port von FB_Automatikbetrieb_F1.

    Wichtiges Kriterium:
      - Wenn op_hasPort existiert => Port-Rolle gewinnt (auch wenn es zusätzlich Var_* gibt).
    """
    port_uri = find_port_uri(graph, pou_uri, symbol_name)
    var_uri = find_internal_variable_uri(graph, pou_uri, symbol_name)

    # SPARQL-analog: <pou_uri> ?property ?searchObj . ?searchObj ?property2 "name"^^xsd:string
    # (Wir machen es ohne SPARQL, aber protokollierbar)
    hits: List[Dict[str, Any]] = []
    for prop, obj in graph.predicate_objects(pou_uri):
        if not isinstance(obj, URIRef):
            continue
        for prop2, lit in graph.predicate_objects(obj):
            if isinstance(lit, Literal) and str(lit) == symbol_name:
                hits.append({"property": str(prop), "searchObj": str(obj), "property2": str(prop2), "value": str(lit)})

    return {
        "symbol": symbol_name,
        "is_port": bool(port_uri),
        "port_uri": str(port_uri) if port_uri else None,
        "is_internal_var": bool(var_uri),
        "var_uri": str(var_uri) if var_uri else None,
        "hits": hits,
    }


def _port_assignments_to_upstream_exprs(graph: Graph, assignment_uris: List[str]) -> List[Dict[str, Any]]:
    """
    Aus op_assignsToPort-Assignments im KG einen upstream-\"Ausdruck\" erzeugen.

    Beispiel:
      Assign_MAIN_fbAuto_Automatikbetrieb_Starten_1 assignsFrom PortInstance_FBInst_MAIN_edgeF1_Q
      -> expr 'edgeF1.Q' im Kontext von Program_MAIN
    """
    out: List[Dict[str, Any]] = []
    op_assignsFrom = AG["op_assignsFrom"]
    op_instantiatesPort = AG["op_instantiatesPort"]
    op_isPortOfInstance = AG["op_isPortOfInstance"]
    dp_hasPortName = AG["dp_hasPortName"]
    dp_hasVariableName = AG["dp_hasVariableName"]
    op_representsFBInstance = AG["op_representsFBInstance"]
    op_hasAssignment = AG["op_hasAssignment"]
    op_containsPOUCall = AG["op_containsPOUCall"]

    for au in assignment_uris:
        assign_uri = URIRef(au)
        assigns_from = _first(graph.objects(assign_uri, op_assignsFrom))
        if not assigns_from:
            out.append({"assignment": au, "note": "Kein op_assignsFrom."})
            continue

        # Caller-Context via POUCall
        pou_call = _first(graph.subjects(op_hasAssignment, assign_uri))
        caller_pou = _first(graph.subjects(op_containsPOUCall, pou_call)) if pou_call else None

        # 1) PortInstance -> FBInst + PortName
        inst_port = _first(graph.objects(assigns_from, op_instantiatesPort))
        fbinst = _first(graph.objects(assigns_from, op_isPortOfInstance))
        port_name = str(_first(graph.objects(inst_port, dp_hasPortName))) if inst_port else None

        inst_var_name = None
        if fbinst:
            var = _first(graph.subjects(op_representsFBInstance, fbinst))
            inst_var_name = str(_first(graph.objects(var, dp_hasVariableName))) if var else None

        if inst_var_name and port_name:
            out.append(
                {
                    "assignment": au,
                    "caller_pou_uri": str(caller_pou) if caller_pou else None,
                    "expr": f"{inst_var_name}.{port_name}",
                    "assigns_from": str(assigns_from),
                }
            )
            continue

        # 2) Var_* (z.B. Var_MAIN_Start_eff)
        var_name_lit = _first(graph.objects(assigns_from, dp_hasVariableName))
        if var_name_lit:
            out.append(
                {
                    "assignment": au,
                    "caller_pou_uri": str(caller_pou) if caller_pou else None,
                    "expr": str(var_name_lit),
                    "assigns_from": str(assigns_from),
                }
            )
            continue

        out.append(
            {
                "assignment": au,
                "caller_pou_uri": str(caller_pou) if caller_pou else None,
                "note": "assignsFrom ist weder PortInstance noch Var_* mit dp_hasVariableName (best-effort).",
                "assigns_from": str(assigns_from),
            }
        )

    return out

def _split_top_level_commas(s: str) -> List[str]:
    parts, buf, depth, in_str = [], [], 0, False
    for i, ch in enumerate(s):
        if ch == "'" and (i == 0 or s[i-1] != "\\"): in_str = not in_str
        if not in_str:
            if ch == "(": depth += 1
            elif ch == ")": depth = max(0, depth - 1)
            elif ch == "," and depth == 0:
                parts.append("".join(buf).strip()); buf = []; continue
        buf.append(ch)
    if tail := "".join(buf).strip(): parts.append(tail)
    return parts

def _count_parens_delta(line: str) -> int:
    """
    Zählt Klammerdelta für ST-Call-Blöcke, ignoriert einfache String-Literale.
    """
    delta = 0
    in_str = False
    for i, ch in enumerate(line):
        if ch == "'" and (i == 0 or line[i - 1] != "\\"):
            in_str = not in_str
            continue
        if in_str:
            continue
        if ch == "(":
            delta += 1
        elif ch == ")":
            delta -= 1
    return delta

def find_fb_call_blocks(st_code: str, inst_name: str) -> List[Tuple[int, str]]:
    """
    Findet inst( ... ); über mehrere Zeilen.
    Rückgabe: [(start_line, full_call_block_text), ...]
    """
    clean = _strip_st_comments(st_code)
    lines = clean.splitlines()
    start_rx = re.compile(rf"^\s*{re.escape(inst_name)}\s*\(")

    blocks: List[Tuple[int, str]] = []
    collecting = False
    start_line = -1
    depth = 0
    buffer: List[str] = []

    for i, line in enumerate(lines, start=1):
        if not collecting:
            if not start_rx.match(line):
                continue
            collecting = True
            start_line = i
            depth = 0
            buffer = []

        buffer.append(line.rstrip())
        depth += _count_parens_delta(line)

        if collecting and depth <= 0 and ";" in line:
            blocks.append((start_line, "\n".join(buffer)))
            collecting = False
            start_line = -1
            depth = 0
            buffer = []

    return blocks

def parse_fb_call_args_from_block(call_block: str, inst_name: str) -> Dict[str, str]:
    m = re.search(rf"{re.escape(inst_name)}\s*\((.*?)\)\s*;", call_block, flags=re.S)
    if not m or not m.group(1).strip():
        return {}

    arg_text = m.group(1).strip()
    mapping = {}
    for a in _split_top_level_commas(arg_text):
        m_in = re.match(r"^\s*([A-Za-z_]\w*)\s*:=\s*(.+?)\s*$", a)
        m_out = re.match(r"^\s*([A-Za-z_]\w*)\s*=>\s*(.+?)\s*$", a)
        if m_in: mapping[m_in.group(1)] = m_in.group(2).strip()
        elif m_out: mapping[m_out.group(1)] = m_out.group(2).strip()
    return mapping

def find_fb_call_lines(st_code: str, inst_name: str) -> List[Tuple[int, str]]:
    # Backward compatibility für bestehende Aufrufer: Einzeiler aus Call-Block ableiten.
    out: List[Tuple[int, str]] = []
    for line_no, block in find_fb_call_blocks(st_code, inst_name):
        first = block.splitlines()[0].rstrip()
        out.append((line_no, first))
    return out

def parse_fb_call_args(call_line: str, inst_name: str) -> Dict[str, str]:
    # Backward compatibility: alter Parsername.
    return parse_fb_call_args_from_block(call_line, inst_name)

def _is_internal_v_token(var: str) -> bool:
    return bool(re.match(r"^V_\d+$", var))

def _build_fbd_true_requirement_tree(
    var: str,
    assigns: Dict[str, Tuple[str, List[str]]],
    depth: int = 0,
    max_depth: int = 25,
) -> FbdNode:
    if depth >= max_depth:
        return FbdNode(var=var, func="MAX_DEPTH", args=[])
    if var not in assigns:
        return FbdNode(var=var)

    func, args = assigns[var]
    if func.startswith("RS_") and len(args) >= 1:
        return FbdNode(
            var=var,
            func=f"{func}[TRUE->SET]",
            args=[_build_fbd_true_requirement_tree(args[0], assigns, depth + 1, max_depth)],
        )
    return FbdNode(
        var=var,
        func=func,
        args=[_build_fbd_true_requirement_tree(a, assigns, depth + 1, max_depth) for a in args],
    )

def trace_symbol_from_fbd(fbd_py_code: str, symbol_name: str, target_value: str = "TRUE") -> Dict[str, Any]:
    assigns = _parse_fbd_call_assignments(fbd_py_code, trace=None)

    root_var = None
    m = re.search(rf"^\s*{re.escape(symbol_name)}\s*=\s*(V_\d+)\s*$", fbd_py_code, flags=re.M)
    if m:
        root_var = m.group(1)
    elif symbol_name in assigns:
        root_var = symbol_name

    if not root_var:
        return {"error": f"Konnte Output-Symbol '{symbol_name}' nicht auf V_* auflösen."}

    tree = _build_fbd_tree(root_var, assigns).to_dict()
    true_tree = _build_fbd_true_requirement_tree(root_var, assigns).to_dict() if target_value.upper() == "TRUE" else tree
    return {
        "symbol": symbol_name,
        "assigned_from": root_var,
        "tree": tree,
        "target_value": target_value.upper(),
        "true_tree": true_tree,
    }

def _render_fbd_tree_lines(node: Dict[str, Any], indent: int = 0) -> List[TraceLine]:
    out: List[TraceLine] = []
    if "func" not in node:
        out.append(TraceLine(indent, f"{node.get('var')}"))
        return out
    out.append(TraceLine(indent, f"{node.get('var')} = {node.get('func')}("))
    for a in node.get("args", []) or []:
        out.extend(_render_fbd_tree_lines(a, indent + 1))
    out.append(TraceLine(indent, ")"))
    return out

def _collect_fbd_leaf_tokens(node: Dict[str, Any]) -> List[str]:
    out: List[str] = []
    if "func" not in node:
        val = str(node.get("var"))
        if val:
            out.append(val)
        return out
    for a in node.get("args", []) or []:
        out.extend(_collect_fbd_leaf_tokens(a))
    return out

def _infer_st_output_dependency_params(inst_type_name: Optional[str], output_port: str, arg_keys: List[str]) -> List[str]:
    """
    Heuristik: Wenn ein ST-Standard-FB keine expliziten Zuweisungen fuer einen Output liefert,
    leite die relevanten Input-Parameter fuer den Trace ab.
    """
    output = (output_port or "").upper()
    type_name = (inst_type_name or "").upper()

    preferred: List[str] = []
    if output == "Q":
        if type_name in ("TON", "TP", "TOF"):
            preferred = ["IN", "PT", "CLK", "S", "R", "SET", "RESET"]
        elif type_name in ("R_TRIG", "F_TRIG"):
            preferred = ["CLK", "IN"]
        elif type_name in ("RS", "SR"):
            preferred = ["S", "R", "SET", "RESET", "IN"]
        else:
            preferred = ["CLK", "IN"]

    keys_upper = {k.upper(): k for k in arg_keys}
    resolved: List[str] = []
    for p in preferred:
        if p in keys_upper and keys_upper[p] not in resolved:
            resolved.append(keys_upper[p])

    if resolved:
        return resolved
    return list(arg_keys)

def _is_global_variable_uri(graph: Graph, var_uri: URIRef) -> bool:
    scope = _first(graph.objects(var_uri, AG["dp_hasVariableScope"]))
    if scope and str(scope).strip().lower() == "global":
        return True
    if _first(graph.subjects(AG["op_listsGlobalVariable"], var_uri)):
        return True
    return False

def describe_global_variable(graph: Graph, symbol_name: str) -> Optional[Dict[str, Any]]:
    candidate_uris: List[URIRef] = []
    for v in graph.subjects(AG["dp_hasVariableName"], Literal(symbol_name)):
        if isinstance(v, URIRef) and v not in candidate_uris:
            candidate_uris.append(v)

    # Fallback: bei dotted Namen auch Kurzname pruefen (z.B. GVL.DiagnoseRequested -> DiagnoseRequested)
    if "." in symbol_name:
        short_name = symbol_name.split(".", 1)[1].strip()
        for v in graph.subjects(AG["dp_hasVariableName"], Literal(short_name)):
            if isinstance(v, URIRef) and v not in candidate_uris:
                candidate_uris.append(v)

    var_uri = None
    for v in candidate_uris:
        if _is_global_variable_uri(graph, v):
            var_uri = v
            break
    if not var_uri:
        return None

    var_type = _first(graph.objects(var_uri, AG["dp_hasVariableType"]))
    scope = _first(graph.objects(var_uri, AG["dp_hasVariableScope"]))
    bound_ports = list(graph.objects(var_uri, AG["op_isBoundToPort"]))
    bound_port_names: List[str] = []
    for p in bound_ports:
        pn = _first(graph.objects(p, AG["dp_hasPortName"]))
        if pn:
            bound_port_names.append(str(pn))

    users = list(graph.subjects(AG["op_usesVariable"], var_uri))
    user_names: List[str] = []
    for u in users:
        n = _first(graph.objects(u, AG["dp_hasPOUName"]))
        user_names.append(str(n) if n else str(u))

    list_uri = _first(graph.subjects(AG["op_listsGlobalVariable"], var_uri))
    list_name = _first(graph.objects(list_uri, AG["dp_hasGlobalVariableListName"])) if list_uri else None

    return {
        "var_uri": str(var_uri),
        "var_name": str(_first(graph.objects(var_uri, AG["dp_hasVariableName"])) or symbol_name),
        "scope": str(scope) if scope else None,
        "var_type": str(var_type) if var_type else None,
        "bound_port_names": bound_port_names,
        "used_by_pou_names": user_names,
        "global_list_uri": str(list_uri) if list_uri else None,
        "global_list_name": str(list_name) if list_name else None,
    }

def find_all_fb_call_blocks(st_code: str) -> List[Tuple[int, str, str]]:
    """
    Findet alle FB-Call-Bloecke in ST:
      [(start_line, inst_name, full_call_block), ...]
    """
    clean = _strip_st_comments(st_code)
    lines = clean.splitlines()
    start_rx = re.compile(r"^\s*([A-Za-z_]\w*)\s*\(")

    blocks: List[Tuple[int, str, str]] = []
    collecting = False
    current_inst = ""
    start_line = -1
    depth = 0
    buffer: List[str] = []

    for i, line in enumerate(lines, start=1):
        if not collecting:
            m = start_rx.match(line)
            if not m:
                continue
            collecting = True
            current_inst = m.group(1)
            start_line = i
            depth = 0
            buffer = []

        buffer.append(line.rstrip())
        depth += _count_parens_delta(line)

        if collecting and depth <= 0 and ";" in line:
            blocks.append((start_line, current_inst, "\n".join(buffer)))
            collecting = False
            current_inst = ""
            start_line = -1
            depth = 0
            buffer = []

    return blocks

def parse_fb_call_bindings_from_block(call_block: str, inst_name: str) -> List[Dict[str, str]]:
    m = re.search(rf"{re.escape(inst_name)}\s*\((.*?)\)\s*;", call_block, flags=re.S)
    if not m or not m.group(1).strip():
        return []
    arg_text = m.group(1).strip()

    out: List[Dict[str, str]] = []
    for a in _split_top_level_commas(arg_text):
        m_in = re.match(r"^\s*([A-Za-z_]\w*)\s*:=\s*(.+?)\s*$", a)
        m_out = re.match(r"^\s*([A-Za-z_]\w*)\s*=>\s*(.+?)\s*$", a)
        if m_in:
            out.append({"param": m_in.group(1), "mode": ":=", "expr": m_in.group(2).strip()})
        elif m_out:
            out.append({"param": m_out.group(1), "mode": "=>", "expr": m_out.group(2).strip()})
    return out

def find_fb_output_writers_for_variable(st_code: str, var_name: str) -> List[Dict[str, Any]]:
    hits: List[Dict[str, Any]] = []
    for line_no, inst_name, block in find_all_fb_call_blocks(st_code):
        for b in parse_fb_call_bindings_from_block(block, inst_name):
            if b["mode"] == "=>" and b["expr"].strip() == var_name:
                hits.append(
                    {
                        "line_no": line_no,
                        "inst_name": inst_name,
                        "port_name": b["param"],
                        "source_token": f"{inst_name}.{b['param']}",
                    }
                )
    return hits

def inspect_variable_access_metadata(graph: Graph, var_uri: Optional[URIRef]) -> Dict[str, Any]:
    if not var_uri:
        return {
            "has_var_uri": False,
            "dp_hasOPCUADataAccess": None,
            "dp_hasOPCUAWriteAccess": None,
            "dp_hasHardwareAddress": None,
        }

    opcua_data = _first(graph.objects(var_uri, AG["dp_hasOPCUADataAccess"]))
    opcua_write = _first(graph.objects(var_uri, AG["dp_hasOPCUAWriteAccess"]))
    hw_addr = _first(graph.objects(var_uri, AG["dp_hasHardwareAddress"]))
    return {
        "has_var_uri": True,
        "dp_hasOPCUADataAccess": str(opcua_data) if opcua_data is not None else None,
        "dp_hasOPCUAWriteAccess": str(opcua_write) if opcua_write is not None else None,
        "dp_hasHardwareAddress": str(hw_addr) if hw_addr is not None else None,
    }

def classify_symbol_in_context(graph: Graph, token: str, ctx_pou_uri: URIRef) -> Dict[str, Any]:
    """
    Zentraler Klassifikator fuer ein Token im Kontext eines POU.

    Reihenfolge:
      1) dotted token (x.y): zuerst Instanz-Port, bei Fehlschlag globale Variable
      2) plain token: Port (mit direction) > interne Variable > globale Variable
    """
    key = token.strip()

    if "." in key:
        inst, port = key.split(".", 1)
        desc = describe_internal_variable(graph, ctx_pou_uri, inst)
        if "error" not in desc:
            callee_uri = URIRef(desc["fbinst_type_uri"]) if desc.get("fbinst_type_uri") else None
            port_direction = get_fb_port_direction(graph, callee_uri, port) if callee_uri else None
            return {
                "kind": "instance_port",
                "token": key,
                "instance": inst,
                "port": port,
                "instance_desc": desc,
                "callee_uri": callee_uri,
                "callee_lang": (desc.get("fbinst_type_language") or "").upper(),
                "port_direction": port_direction,
                "reason": "dot_token->instance_port",
            }

        gvar = describe_global_variable(graph, key)
        if gvar:
            return {
                "kind": "global_var",
                "token": key,
                "global": gvar,
                "reason": "dot_token->global_var_fallback",
            }

        return {"kind": "unknown", "token": key, "reason": "dot_token->unresolved"}

    port_uri = find_port_uri(graph, ctx_pou_uri, key)
    if port_uri:
        direction = _first(graph.objects(port_uri, AG["dp_hasPortDirection"]))
        return {
            "kind": "pou_port",
            "token": key,
            "port_uri": str(port_uri),
            "port_direction": str(direction) if direction else None,
            "reason": "plain_token->pou_port",
        }

    var_uri = find_internal_variable_uri(graph, ctx_pou_uri, key)
    if var_uri:
        return {
            "kind": "internal_var",
            "token": key,
            "var_uri": str(var_uri),
            "reason": "plain_token->internal_var",
        }

    gvar = describe_global_variable(graph, key)
    if gvar:
        return {
            "kind": "global_var",
            "token": key,
            "global": gvar,
            "reason": "plain_token->global_var",
        }

    return {"kind": "unknown", "token": key, "reason": "plain_token->unresolved"}

def is_betriebsarten_layer_pou(graph: Graph, pou_uri: URIRef) -> bool:
    """
    True, wenn der aktuelle Kontext der GEMMA-Betriebsarten-Layer ist.
    """
    if not pou_uri:
        return False
    if str(pou_uri).endswith("FBType_FB_Betriebsarten"):
        return True
    name = _first(graph.objects(pou_uri, AG["dp_hasPOUName"]))
    return str(name) == "FB_Betriebsarten"

def extract_dotted_tokens(expr: str) -> List[str]:
    return re.findall(r"\b[A-Za-z_]\w*\.[A-Za-z_]\w*\b", expr)

@dataclass
class TraceLine:
    indent: int
    text: str

def trace_expr(
    graph: Graph,
    expr: str,
    fbtype_uri: URIRef,
    st_code: str,
    *,
    max_depth: int = 10,
    assumed_false_states_in_betriebsarten: Optional[Set[str]] = None,
) -> List[TraceLine]:
    lines: List[TraceLine] = []
    visited: Set[str] = set()
    assumed_false = {s.upper() for s in (assumed_false_states_in_betriebsarten or {"D1", "D2", "D3"})}

    def _last_true_assignment(var_name: str) -> Optional[Dict[str, Any]]:
        aa = analyze_var_assignments_st(st_code, var_name, trace=None)
        trues = [a for a in aa.get("assignments", []) if a.get("value") == "TRUE"]
        return trues[-1] if trues else None

    def _innermost_if_expr(conditions: List[str]) -> Optional[str]:
        rx = re.compile(r"^(IF|ELSIF)@\d+:\s*(.*)\s*$", flags=re.I)
        for raw in reversed(conditions or []):
            m = rx.match(raw.strip())
            if m:
                return m.group(2).strip()
        return None

    def rec(e: str, depth: int, ctx_pou_uri: URIRef, ctx_code: str):
        key = e.strip()
        visit_key = f"{ctx_pou_uri}|{key}"
        if visit_key in visited:
            lines.append(TraceLine(depth, f"(loop detected) {key}"))
            return
        visited.add(visit_key)

        if depth > max_depth:
            lines.append(TraceLine(depth, f"(max depth) {key}"))
            return

        if is_betriebsarten_layer_pou(graph, ctx_pou_uri) and key.upper() in assumed_false:
            lines.append(
                TraceLine(
                    depth,
                    f"[ASSUME] {key}=FALSE im GEMMA-Betriebsarten-Layer -> Zweig wird nicht weiter analysiert.",
                )
            )
            return

        cls = classify_symbol_in_context(graph, key, ctx_pou_uri)
        lines.append(TraceLine(depth, f"[CLS] token='{key}' kind={cls.get('kind')} reason={cls.get('reason')}"))

        if cls.get("kind") == "global_var":
            gvar = cls.get("global", {})
            lines.append(TraceLine(depth, f"{key} ist globale Variable (scope={gvar.get('scope')}, type={gvar.get('var_type')})."))
            if gvar.get("bound_port_names"):
                lines.append(TraceLine(depth + 1, f"op_isBoundToPort -> {gvar.get('bound_port_names')}"))
            lines.append(TraceLine(depth + 1, f"op_usesVariable -> {gvar.get('used_by_pou_names')}"))
            lines.append(TraceLine(depth + 1, "Trace endet hier (globale Signalquelle)."))
            return

        if cls.get("kind") == "instance_port":
            inst = cls["instance"]
            port = cls["port"]
            desc = cls["instance_desc"]

            lines.append(TraceLine(depth, f"{key} ist Port '{port}' der FB-Instanz '{inst}' (Typ={desc['var_type']})"))
            if desc.get("fbinst_type_comment"):
                lines.append(TraceLine(depth + 1, f"FBType-Beschreibung: {desc['fbinst_type_comment']}"))

            call_hits = find_fb_call_blocks(ctx_code, inst)
            if not call_hits:
                lines.append(TraceLine(depth + 1, f"Keine Call-Block '{inst}(...)' im ST-Code gefunden."))
                return

            call_line_no, call_block = call_hits[0]
            first_line = call_block.splitlines()[0].strip() if call_block else f"{inst}(...)"
            lines.append(TraceLine(depth + 1, f"Call-Block @ {call_line_no}: {first_line}"))
            args = parse_fb_call_args_from_block(call_block, inst)

            callee_uri = URIRef(desc["fbinst_type_uri"]) if desc.get("fbinst_type_uri") else None
            callee_lang = (desc.get("fbinst_type_language") or "").upper()
            if callee_uri:
                lines.append(TraceLine(depth + 1, f"[KG] Instanz-Typ: {callee_uri} (lang={callee_lang or 'unbekannt'})"))
            callee_port_dir = get_fb_port_direction(graph, callee_uri, port) if callee_uri else None
            lines.append(TraceLine(depth + 1, f"[KG] Port '{port}' direction im callee: {callee_port_dir}"))

            # Input-Port des callee: Argument aus Call verwenden.
            if callee_port_dir and callee_port_dir.lower() == "input":
                mapped = args.get(port)
                if mapped:
                    lines.append(TraceLine(depth + 2, f"[ST-MAP] Input-Port '{port}' <= {mapped}"))
                    for dt in extract_dotted_tokens(mapped):
                        rec(dt, depth + 3, ctx_pou_uri, ctx_code)
                    plain = mapped.strip()
                    if re.fullmatch(r"[A-Za-z_]\w*", plain):
                        rec(plain, depth + 3, ctx_pou_uri, ctx_code)
                else:
                    lines.append(TraceLine(depth + 2, f"[ST-MAP] Kein Argument für Input-Port '{port}' im Call gefunden."))
                return

            # Output-Port des callee: Sprache entscheidet den Trace-Algorithmus.
            if callee_uri and callee_lang == "FBD":
                callee_code = get_pou_code(graph, callee_uri, trace=None)
                lines.append(TraceLine(depth + 1, f"[FBD] Trace Output '{port}' im callee (FBD-Export)"))
                if not callee_code:
                    lines.append(TraceLine(depth + 2, "Kein dp_hasPOUCode im callee gefunden."))
                    return
                out_trace = trace_symbol_from_fbd(callee_code, port, target_value="TRUE")
                if "error" in out_trace:
                    lines.append(TraceLine(depth + 2, out_trace["error"]))
                    return
                lines.append(TraceLine(depth + 2, f"[FBD] {port} assigned_from {out_trace.get('assigned_from')}"))
                for tl in _render_fbd_tree_lines(out_trace["tree"], indent=depth + 3):
                    lines.append(tl)

                focus_tree = out_trace.get("true_tree") or out_trace["tree"]
                if focus_tree is not out_trace["tree"]:
                    lines.append(TraceLine(depth + 2, f"[FBD] TRUE-Pfad fuer '{port}' (RS -> SetCond):"))
                    for tl in _render_fbd_tree_lines(focus_tree, indent=depth + 3):
                        lines.append(tl)

                leaves = _collect_fbd_leaf_tokens(focus_tree)
                uniq_leaves: List[str] = []
                for tok in leaves:
                    if tok not in uniq_leaves:
                        uniq_leaves.append(tok)
                lines.append(TraceLine(depth + 2, f"[FBD] Leaf tokens (uniq): {uniq_leaves}"))

                callee_code_ctx = callee_code or ""
                for leaf in uniq_leaves:
                    leaf_dir = get_fb_port_direction(graph, callee_uri, leaf)
                    if leaf_dir and leaf_dir.lower() == "input":
                        mapped_leaf = args.get(leaf)
                        if mapped_leaf:
                            lines.append(TraceLine(depth + 2, f"[FBD->CALLMAP] {leaf} <= {mapped_leaf}"))
                            for dt in extract_dotted_tokens(mapped_leaf):
                                rec(dt, depth + 3, ctx_pou_uri, ctx_code)
                            plain = mapped_leaf.strip()
                            if re.fullmatch(r"[A-Za-z_]\w*", plain):
                                rec(plain, depth + 3, ctx_pou_uri, ctx_code)
                        else:
                            lines.append(TraceLine(depth + 2, f"[FBD->CALLMAP] Kein Call-Argument für Input '{leaf}' gefunden."))
                        continue

                    if _is_internal_v_token(leaf):
                        continue
                    if "." in leaf:
                        rec(leaf, depth + 2, callee_uri, callee_code_ctx)
                    elif re.fullmatch(r"[A-Za-z_]\w*", leaf):
                        rec(leaf, depth + 2, callee_uri, callee_code_ctx)
                    else:
                        lines.append(TraceLine(depth + 2, f"[FBD] Atom: {leaf}"))
                return

            if callee_uri and callee_lang == "ST":
                callee_code = get_pou_code(graph, callee_uri, trace=None) or ""
                lines.append(TraceLine(depth + 1, f"[ST] Trace Output '{port}' im callee (ST)"))
                aa_callee = analyze_var_assignments_st(callee_code, port, trace=None)
                summ = aa_callee.get("summary", {})
                lines.append(TraceLine(depth + 2, f"Assignments im callee ST-Code: {summ.get('assignment_count')}"))
                trues = [a for a in aa_callee.get("assignments", []) if a.get("value") == "TRUE"]
                last_true = trues[-1] if trues else None
                if last_true:
                    inn = _innermost_if_expr(last_true.get("conditions", []))
                    if inn:
                        lines.append(TraceLine(depth + 2, f"Letzte TRUE-Zuweisung @ {last_true['line_no']} (innere IF): {inn}"))
                        for tok in extract_variables_from_condition(inn):
                            rec(tok, depth + 3, callee_uri, callee_code)
                        return

                dep_params = _infer_st_output_dependency_params(desc.get("var_type"), port, list(args.keys()))
                lines.append(TraceLine(depth + 2, f"[ST-FALLBACK] Keine direkte TRUE-Zuweisung fuer Output '{port}'."))
                lines.append(TraceLine(depth + 2, f"[ST-FALLBACK] Verfolge Input-Parameter: {dep_params}"))
                for param in dep_params:
                    mapped = args.get(param)
                    if not mapped:
                        continue
                    lines.append(TraceLine(depth + 3, f"[ST-FALLBACK] {port} <= {param} <= {mapped}"))
                    for dt in extract_dotted_tokens(mapped):
                        rec(dt, depth + 4, ctx_pou_uri, ctx_code)
                    plain = mapped.strip()
                    if re.fullmatch(r"[A-Za-z_]\w*", plain):
                        rec(plain, depth + 4, ctx_pou_uri, ctx_code)
                return

            if args:
                for k_arg, v_expr in args.items():
                    lines.append(TraceLine(depth + 2, f"Param {k_arg} = {v_expr}"))
                    for dt in extract_dotted_tokens(v_expr):
                        rec(dt, depth + 3, ctx_pou_uri, ctx_code)

                    plain = v_expr.strip()
                    if re.fullmatch(r"[A-Za-z_]\w*", plain):
                        roles = _find_symbol_roles_in_pou(graph, ctx_pou_uri, plain)
                        if roles.get("hits"):
                            lines.append(TraceLine(depth + 3, f"[KG-ROLE] lookup '{plain}' -> hits={len(roles['hits'])} is_port={roles['is_port']} is_internal_var={roles['is_internal_var']}"))

                        # Port gewinnt, falls vorhanden
                        if roles.get("is_port"):
                            port_info = port_default_and_wiring(graph, ctx_pou_uri, plain)
                            dv = port_info.get("default_value")
                            wired = port_info.get("is_wired")
                            lines.append(TraceLine(depth + 3, f"'{plain}' ist Input-Port von {str(ctx_pou_uri).split('_')[-1]}"))
                            lines.append(TraceLine(depth + 4, f"default_value = {dv}"))
                            lines.append(TraceLine(depth + 4, f"is_wired = {wired} (assignments={port_info.get('assignments_count')})"))

                            if wired:
                                ups = _port_assignments_to_upstream_exprs(graph, port_info.get("assignment_uris", []))
                                for u in ups:
                                    up_expr = u.get("expr")
                                    caller_pou_uri = u.get("caller_pou_uri")
                                    lines.append(TraceLine(depth + 4, f"[KG-WIRE] {plain} <= {up_expr} (caller_pou={caller_pou_uri})"))
                                    if up_expr and caller_pou_uri:
                                        caller_uri = URIRef(caller_pou_uri)
                                        caller_code = get_pou_code(graph, caller_uri, trace=None) or ""
                                        rec(up_expr, depth + 5, caller_uri, caller_code)
                            else:
                                lines.append(TraceLine(depth + 4, "=> nicht verdrahtet, daher wird DefaultValue verwendet (Trace endet hier)."))
                        elif roles.get("is_internal_var"):
                            lines.append(TraceLine(depth + 3, f"'{plain}' ist interne Variable (wird im ST gesetzt)."))
                            aa = analyze_var_assignments_st(ctx_code, plain, trace=None)
                            summ = aa.get("summary", {})
                            lines.append(TraceLine(depth + 4, f"Assignments im ST-Code: {summ.get('assignment_count')}"))
            return

        if re.fullmatch(r"[A-Za-z_]\w*", key):
            roles = _find_symbol_roles_in_pou(graph, ctx_pou_uri, key)
            if roles.get("hits"):
                lines.append(TraceLine(depth, f"[KG-ROLE] lookup '{key}' -> hits={len(roles['hits'])} is_port={roles['is_port']} is_internal_var={roles['is_internal_var']}"))

            if cls.get("kind") == "pou_port":
                direction = (cls.get("port_direction") or "unknown").strip()
                port_info = port_default_and_wiring(graph, ctx_pou_uri, key)
                dv = port_info.get("default_value")
                wired = port_info.get("is_wired")
                lines.append(TraceLine(depth, f"'{key}' ist {direction}-Port von {str(ctx_pou_uri).split('_')[-1]}"))
                if direction.lower() == "input":
                    lines.append(TraceLine(depth + 1, f"default_value = {dv}"))
                    lines.append(TraceLine(depth + 1, f"is_wired = {wired} (assignments={port_info.get('assignments_count')})"))
                    if wired:
                        ups = _port_assignments_to_upstream_exprs(graph, port_info.get("assignment_uris", []))
                        for u in ups:
                            up_expr = u.get("expr")
                            caller_pou_uri = u.get("caller_pou_uri")
                            lines.append(TraceLine(depth + 1, f"[KG-WIRE] {key} <= {up_expr} (caller_pou={caller_pou_uri})"))
                            if up_expr and caller_pou_uri:
                                caller_uri = URIRef(caller_pou_uri)
                                caller_code = get_pou_code(graph, caller_uri, trace=None) or ""
                                rec(up_expr, depth + 2, caller_uri, caller_code)
                    else:
                        lines.append(TraceLine(depth + 1, "=> nicht verdrahtet, daher wird DefaultValue verwendet (Trace endet hier)."))
                else:
                    lines.append(TraceLine(depth + 1, "Output-Port: Wert wird intern im POU berechnet."))

                    pou_lang_lit = _first(graph.objects(ctx_pou_uri, AG["dp_hasPOULanguage"]))
                    pou_lang = str(pou_lang_lit).upper() if pou_lang_lit else ""
                    is_custom_fb = (ctx_pou_uri, RDF.type, AG["class_CustomFBType"]) in graph
                    lines.append(TraceLine(depth + 1, f"[POU-OUT] owning_pou={ctx_pou_uri} lang={pou_lang or 'unbekannt'} is_custom={is_custom_fb}"))

                    owning_code = ctx_code or (get_pou_code(graph, ctx_pou_uri, trace=None) or "")
                    if not is_custom_fb:
                        lines.append(TraceLine(depth + 1, "[POU-OUT] Kein CustomFBType. Verwende Fallback und beende hier."))
                        return
                    if not owning_code:
                        lines.append(TraceLine(depth + 1, "[POU-OUT] Kein dp_hasPOUCode gefunden."))
                        return

                    if pou_lang == "FBD":
                        lines.append(TraceLine(depth + 1, f"[POU-OUT] Rekursiver FBD-Trace fuer Output '{key}'"))
                        out_trace = trace_symbol_from_fbd(owning_code, key, target_value="TRUE")
                        if "error" in out_trace:
                            lines.append(TraceLine(depth + 2, out_trace["error"]))
                            return

                        lines.append(TraceLine(depth + 2, f"[POU-OUT][FBD] {key} assigned_from {out_trace.get('assigned_from')}"))
                        focus_tree = out_trace.get("true_tree") or out_trace.get("tree")
                        if focus_tree is not out_trace.get("tree"):
                            lines.append(TraceLine(depth + 2, f"[POU-OUT][FBD] TRUE-Pfad fuer '{key}' (RS -> SetCond):"))
                        for tl in _render_fbd_tree_lines(focus_tree, indent=depth + 3):
                            lines.append(tl)

                        leaves = _collect_fbd_leaf_tokens(focus_tree)
                        uniq_leaves: List[str] = []
                        for tok in leaves:
                            if tok not in uniq_leaves:
                                uniq_leaves.append(tok)
                        lines.append(TraceLine(depth + 2, f"[POU-OUT][FBD] Leaf tokens (uniq): {uniq_leaves}"))

                        for leaf in uniq_leaves:
                            if leaf == key or _is_internal_v_token(leaf):
                                continue
                            if "." in leaf:
                                rec(leaf, depth + 3, ctx_pou_uri, owning_code)
                            elif re.fullmatch(r"[A-Za-z_]\w*", leaf):
                                rec(leaf, depth + 3, ctx_pou_uri, owning_code)
                            else:
                                lines.append(TraceLine(depth + 3, f"[POU-OUT][FBD] Atom: {leaf}"))
                        return

                    if pou_lang == "ST":
                        lines.append(TraceLine(depth + 1, f"[POU-OUT] Rekursiver ST-Trace fuer Output '{key}'"))
                        aa_out = analyze_var_assignments_st(owning_code, key, trace=None)
                        summ_out = aa_out.get("summary", {})
                        lines.append(TraceLine(depth + 2, f"Assignments im owning ST-Code: {summ_out.get('assignment_count')}"))
                        trues_out = [a for a in aa_out.get("assignments", []) if a.get("value") == "TRUE"]
                        last_true_out = trues_out[-1] if trues_out else None
                        if last_true_out:
                            inn_out = _innermost_if_expr(last_true_out.get("conditions", []))
                            if inn_out:
                                lines.append(TraceLine(depth + 2, f"Letzte TRUE-Zuweisung @ {last_true_out['line_no']} (innere IF): {inn_out}"))
                                for tok in extract_variables_from_condition(inn_out):
                                    rec(tok, depth + 3, ctx_pou_uri, owning_code)
                        return

                    lines.append(TraceLine(depth + 1, "[POU-OUT] Unbekannte POU-Sprache. Keine weitere Rekursion."))
                    return
            elif cls.get("kind") == "internal_var":
                lines.append(TraceLine(depth, f"'{key}' ist interne Variable in {str(ctx_pou_uri).split('_')[-1]}"))
                aa = analyze_var_assignments_st(ctx_code, key, trace=None)
                summ = aa.get("summary", {})
                assign_count = int(summ.get("assignment_count") or 0)
                lines.append(TraceLine(depth + 1, f"Assignments im ST-Code: {assign_count}"))

                if assign_count == 0:
                    writer_hits = find_fb_output_writers_for_variable(ctx_code, key)
                    if writer_hits:
                        lines.append(TraceLine(depth + 1, f"[ST-FALLBACK] Output-Mappings auf '{key}': {len(writer_hits)}"))
                        for wh in writer_hits:
                            source_token = wh["source_token"]
                            lines.append(
                                TraceLine(
                                    depth + 2,
                                    f"[ST-FALLBACK] line {wh['line_no']}: {key} <= {source_token}",
                                )
                            )
                            rec(source_token, depth + 3, ctx_pou_uri, ctx_code)
                    else:
                        var_uri = URIRef(cls["var_uri"]) if cls.get("var_uri") else None
                        default_val = _first(graph.objects(var_uri, AG["dp_hasDefaultVariableValue"])) if var_uri else None
                        meta = inspect_variable_access_metadata(graph, var_uri)
                        lines.append(TraceLine(depth + 1, "[META] Keine weiteren Aufrufer gefunden. Prüfe Zugriffs-Metadaten:"))
                        lines.append(TraceLine(depth + 2, f"dp_hasOPCUADataAccess = {meta.get('dp_hasOPCUADataAccess')}"))
                        lines.append(TraceLine(depth + 2, f"dp_hasOPCUAWriteAccess = {meta.get('dp_hasOPCUAWriteAccess')}"))
                        lines.append(TraceLine(depth + 2, f"dp_hasHardwareAddress = {meta.get('dp_hasHardwareAddress')}"))
                        if default_val is not None:
                            lines.append(TraceLine(depth + 1, f"[ST-FALLBACK] Kein Writer gefunden. Default-Wert = {default_val}"))
                        else:
                            lines.append(TraceLine(depth + 1, "[ST-FALLBACK] Kein Writer und kein Default-Wert im KG gefunden."))
                    return

                last_true = _last_true_assignment(key)
                if last_true:
                    inn = _innermost_if_expr(last_true.get("conditions", []))
                    if inn:
                        lines.append(TraceLine(depth + 2, f"Letzte TRUE-Zuweisung @ {last_true['line_no']} (innere IF): {inn}"))
                        for tok in extract_variables_from_condition(inn):
                            rec(tok, depth + 3, ctx_pou_uri, ctx_code)
                else:
                    all_assignments = aa.get("assignments", [])
                    if all_assignments:
                        last_any = all_assignments[-1]
                        inn_any = _innermost_if_expr(last_any.get("conditions", []))
                        lines.append(TraceLine(depth + 2, f"Keine TRUE-Zuweisung. Letzte Zuweisung @ {last_any['line_no']} = {last_any['value']}"))
                        if inn_any:
                            lines.append(TraceLine(depth + 2, f"Innere IF der letzten Zuweisung: {inn_any}"))
                            for tok in extract_variables_from_condition(inn_any):
                                rec(tok, depth + 3, ctx_pou_uri, ctx_code)

                tail = aa.get("assignments", [])[-6:]
                if tail:
                    lines.append(TraceLine(depth + 1, "Letzte Assignments (Auszug):"))
                    for a in tail:
                        lines.append(TraceLine(depth + 2, f"@{a['line_no']} {key} := {a['value']} | Path: {a['conditions_conjunction']}"))
            else:
                lines.append(TraceLine(depth, f"'{key}' ist im Kontext nicht klassifizierbar (kind={cls.get('kind')})."))
            return

        lines.append(TraceLine(depth, f"Unbekanntes Token: {key}"))

    rec(expr, 0, fbtype_uri, st_code)
    return lines


_TIMER_FB_TYPES = {"TON", "TOF", "TP", "R_TRIG", "F_TRIG"}


def summarize_trace_characteristics(trace_lines: List[TraceLine]) -> Dict[str, Any]:
    """
    Verdichtet den freien Trace-Text in technische Fakten:
    - verwendete FB-Typen (Timer/Trigger vs. sonstige)
    - erkannte HW/OPCUA-Metadaten
    - unverdrahtete Input-Ports + Default-Werte
    - Loop-Hinweise
    """
    fb_types: List[str] = []
    non_timer_fb_types: List[str] = []
    loop_tokens: List[str] = []
    hw_values: List[str] = []
    opcua_data_values: List[str] = []
    opcua_write_values: List[str] = []
    default_values_by_token: Dict[str, str] = {}
    unwired_tokens: List[str] = []
    current_input_token: Optional[str] = None

    rx_fb_type = re.compile(r"Typ=([^)]+)\)")
    rx_input_token = re.compile(r"^'([^']+)' ist Input-Port\b")
    rx_default = re.compile(r"^default_value\s*=\s*(.*)$")
    rx_loop = re.compile(r"^\(loop detected\)\s+(.+)$")
    rx_hw = re.compile(r"^dp_hasHardwareAddress\s*=\s*(.*)$")
    rx_op_data = re.compile(r"^dp_hasOPCUADataAccess\s*=\s*(.*)$")
    rx_op_write = re.compile(r"^dp_hasOPCUAWriteAccess\s*=\s*(.*)$")

    def _append_unique(lst: List[str], val: Optional[str]) -> None:
        if not val:
            return
        if val not in lst:
            lst.append(val)

    for tl in trace_lines:
        txt = tl.text.strip()
        if not txt:
            continue

        m_fb = rx_fb_type.search(txt)
        if m_fb:
            fb_t = m_fb.group(1).strip()
            _append_unique(fb_types, fb_t)
            if fb_t.upper() not in _TIMER_FB_TYPES:
                _append_unique(non_timer_fb_types, fb_t)

        m_in = rx_input_token.match(txt)
        if m_in:
            current_input_token = m_in.group(1).strip()

        m_dv = rx_default.match(txt)
        if m_dv:
            dv = m_dv.group(1).strip()
            if current_input_token:
                default_values_by_token[current_input_token] = dv

        if "is_wired = False" in txt and current_input_token:
            _append_unique(unwired_tokens, current_input_token)

        m_loop = rx_loop.match(txt)
        if m_loop:
            _append_unique(loop_tokens, m_loop.group(1).strip())

        m_hw = rx_hw.match(txt)
        if m_hw:
            val = m_hw.group(1).strip()
            if val and val.upper() != "NONE":
                _append_unique(hw_values, val)

        m_od = rx_op_data.match(txt)
        if m_od:
            val = m_od.group(1).strip()
            if val and val.upper() != "NONE":
                _append_unique(opcua_data_values, val)

        m_ow = rx_op_write.match(txt)
        if m_ow:
            val = m_ow.group(1).strip()
            if val and val.upper() != "NONE":
                _append_unique(opcua_write_values, val)

    default_time_literals: List[str] = []
    for token, dv in default_values_by_token.items():
        if isinstance(dv, str) and re.search(r"\bT#[0-9A-Za-z_]+\b", dv, flags=re.I):
            default_time_literals.append(f"{token}={dv}")

    has_hw_or_opcua = bool(hw_values or opcua_data_values or opcua_write_values)
    only_timer_fb = bool(fb_types) and not non_timer_fb_types
    custom_fb_types = [t for t in non_timer_fb_types if t.upper().startswith("FB_")]
    timer_or_custom_logic_only = bool(fb_types) and len(custom_fb_types) == len(non_timer_fb_types)
    forced_candidates = [f"{tok} (default={default_values_by_token.get(tok)})" for tok in unwired_tokens]

    return {
        "fb_types": fb_types,
        "non_timer_fb_types": non_timer_fb_types,
        "custom_fb_types": custom_fb_types,
        "only_timer_fb": only_timer_fb,
        "timer_or_custom_logic_only": timer_or_custom_logic_only,
        "loop_tokens": loop_tokens,
        "unwired_input_tokens": unwired_tokens,
        "default_values_by_token": default_values_by_token,
        "default_time_literals": default_time_literals,
        "forced_candidates": forced_candidates,
        "hardware_addresses": hw_values,
        "opcua_data_access": opcua_data_values,
        "opcua_write_access": opcua_write_values,
        "has_hw_or_opcua": has_hw_or_opcua,
    }


def log_trace_characteristics(
    trace: Tracer,
    root_token: str,
    summary: Dict[str, Any],
) -> None:
    trace.log(f"[EVAL] Zusammenfassung '{root_token}':")
    trace.log(f"[EVAL]   FB-Typen: {summary.get('fb_types')}")
    trace.log(f"[EVAL]   Nur Timer/Trigger-Kette: {summary.get('only_timer_fb')}")
    if summary.get("custom_fb_types"):
        trace.log(f"[EVAL]   Zusätzliche Custom-FBs: {summary.get('custom_fb_types')}")
    trace.log(f"[EVAL]   Loops: {summary.get('loop_tokens')}")
    trace.log(f"[EVAL]   Unverdrahtete Inputs: {summary.get('unwired_input_tokens')}")
    if summary.get("default_time_literals"):
        trace.log(f"[EVAL]   Zeit-Defaults erkannt: {summary.get('default_time_literals')}")

    trace.log(
        f"[EVAL]   HW/OPCUA vorhanden: {summary.get('has_hw_or_opcua')} "
        f"(HW={summary.get('hardware_addresses')}, "
        f"OPCUA-Data={summary.get('opcua_data_access')}, "
        f"OPCUA-Write={summary.get('opcua_write_access')})"
    )
    if summary.get("forced_candidates"):
        trace.log(f"[EVAL]   Force/Config-Kandidaten: {summary.get('forced_candidates')}")

    if summary.get("only_timer_fb") and not summary.get("has_hw_or_opcua"):
        trace.log("[EVAL]   Interpretation: Kette ist zeit-/logikgetrieben (kein HW/OPCUA-Auslöser im Trace).")
    elif summary.get("timer_or_custom_logic_only") and not summary.get("has_hw_or_opcua"):
        trace.log("[EVAL]   Interpretation: Kette besteht aus Timern + Custom-Logik, ohne HW/OPCUA-Auslöser.")


def _append_unique_text(items: List[str], value: Optional[str]) -> None:
    if not value:
        return
    if value not in items:
        items.append(value)


def collect_trace_markers(trace_lines: List[TraceLine]) -> Dict[str, List[str]]:
    """
    Extrahiert zentrale Marker aus den freien Trace-Lines:
    - IF-Verzweigungen
    - Knoten (Port/Var/Global)
    - Call-/Wiring-Kette
    - Terminale Hinweise
    """
    markers = {
        "classifications": [],
        "if_branches": [],
        "nodes": [],
        "call_and_wire": [],
        "terminals": [],
    }

    for tl in trace_lines:
        txt = tl.text.strip()
        if not txt:
            continue

        if txt.startswith("[CLS]"):
            _append_unique_text(markers["classifications"], txt)

        if "Letzte TRUE-Zuweisung @" in txt or txt.startswith("IF@") or "Innere IF" in txt:
            _append_unique_text(markers["if_branches"], txt)

        if (
            " ist Port '" in txt
            or " ist interne Variable" in txt
            or " ist globale Variable" in txt
            or " ist Input-Port " in txt
            or " ist Output-Port " in txt
        ):
            _append_unique_text(markers["nodes"], txt)

        if (
            "Call-Block @" in txt
            or "Call-Line @" in txt
            or txt.startswith("[KG-WIRE]")
            or txt.startswith("[ST-MAP]")
            or txt.startswith("[FBD->CALLMAP]")
            or txt.startswith("[ST-FALLBACK]")
            or txt.startswith("[POU-OUT]")
            or txt.startswith("[FBD]")
        ):
            _append_unique_text(markers["call_and_wire"], txt)

        if (
            "Trace endet hier" in txt
            or "Trace endet" in txt
            or "=> nicht verdrahtet" in txt
            or "(loop detected)" in txt
            or "(max depth)" in txt
            or txt.startswith("[ASSUME]")
            or txt.startswith("[META]")
        ):
            _append_unique_text(markers["terminals"], txt)

    return markers


def build_token_trace_report(
    token: str,
    clause_raw: str,
    is_negated: bool,
    trace_lines: List[TraceLine],
    summary: Dict[str, Any],
) -> Dict[str, Any]:
    markers = collect_trace_markers(trace_lines)
    first_cls = markers["classifications"][0] if markers["classifications"] else None
    return {
        "token": token,
        "clause": clause_raw,
        "negated": is_negated,
        "summary": summary,
        "markers": markers,
        "first_classification": first_cls,
    }


def log_detailed_true_condition_diagnosis(
    trace: Tracer,
    *,
    target_var: str,
    target_pou_uri: URIRef,
    dominant_assignment: Dict[str, Any],
    inner_if: Dict[str, Any],
    truth_paths: List[List[str]],
    path_reports: List[Dict[str, Any]],
) -> None:
    """
    Ausführliche Diagnose:
    - welche Bool-Pfade untersucht wurden
    - welche Tokens je Pfad durchlaufen wurden
    - welche IF-Verzweigungen/Knoten/Call-Wires passiert wurden
    - wie die Pfadenden (Loop/Default/Global/Meta) aussehen
    """
    trace.log("\n--- Diagnosebericht: warum wurde Stoerung_erkannt = TRUE? ---")
    trace.log(f"[DIAG] Zielvariable: {target_var}")
    trace.log(f"[DIAG] Kontext-POU: {target_pou_uri}")
    trace.log(f"[DIAG] Dominante TRUE-Zuweisung: line {dominant_assignment.get('line_no')} -> {dominant_assignment.get('assignment')}")
    trace.log(f"[DIAG] Aktive innere IF: IF@{inner_if.get('if_line')}: {inner_if.get('expr')}")
    trace.log(f"[DIAG] Untersuchte Bool-Pfade ({len(truth_paths)}):")
    for idx, p in enumerate(truth_paths, start=1):
        trace.log(f"[DIAG]   Path #{idx}: " + "  AND  ".join(p))

    for pr in path_reports:
        path_idx = pr.get("path_index")
        clauses = pr.get("clauses", [])
        trace.log(f"\n[DIAG][PATH #{path_idx}] " + "  AND  ".join(clauses))

        token_reports = pr.get("token_reports", [])
        if not token_reports:
            trace.log("[DIAG][PATH]   Keine Token-Reports vorhanden.")
            continue

        for trp in token_reports:
            token = trp.get("token")
            neg_tag = " (NOT-Kontext)" if trp.get("negated") else ""
            trace.log(f"[DIAG][TOKEN] {token}{neg_tag}")
            if trp.get("first_classification"):
                trace.log(f"[DIAG][TOKEN]   Klassifikation: {trp.get('first_classification')}")

            mk = trp.get("markers", {})
            ifs = mk.get("if_branches", [])
            nodes = mk.get("nodes", [])
            flow = mk.get("call_and_wire", [])
            terms = mk.get("terminals", [])

            trace.log(f"[DIAG][TOKEN]   IF-Verzweigungen ({len(ifs)}):")
            for item in ifs[:12]:
                trace.log(f"[DIAG][TOKEN]     - {item}")

            trace.log(f"[DIAG][TOKEN]   Knoten ({len(nodes)}):")
            for item in nodes[:16]:
                trace.log(f"[DIAG][TOKEN]     - {item}")

            trace.log(f"[DIAG][TOKEN]   Call/Wire-Kette ({len(flow)}):")
            for item in flow[:20]:
                trace.log(f"[DIAG][TOKEN]     - {item}")

            trace.log(f"[DIAG][TOKEN]   Terminale Hinweise ({len(terms)}):")
            for item in terms[:12]:
                trace.log(f"[DIAG][TOKEN]     - {item}")

            summ = trp.get("summary", {})
            trace.log(
                "[DIAG][TOKEN]   Bewertung: "
                f"timer_only={summ.get('only_timer_fb')}, "
                f"timer_or_custom={summ.get('timer_or_custom_logic_only')}, "
                f"loops={summ.get('loop_tokens')}, "
                f"defaults={summ.get('default_values_by_token')}, "
                f"hw/opcua={summ.get('has_hw_or_opcua')}"
            )


# ==========================================
# TEIL 3: NEU - IF-BEDINGUNG ZERLEGEN & ORCHESTRIEREN
# ==========================================

def extract_variables_from_condition(cond_str: str) -> List[str]:
    """
    Extrahierte SPS-Variablen/Ports aus einem IF-Pfad (ignoriert Keywords).
    Beispiel: "IF@77: (Schritt1 OR Schritt2) AND pPer.Q" -> ['Schritt1', 'Schritt2', 'pPer.Q']
    """
    # 1. Entferne Tracer-Tags (z.B. IF@77:)
    clean_str = re.sub(r'(IF|ELSIF|ELSE)@\d+:\s*', '', cond_str)
    
    # 2. Finde alle Bezeichner (inklusive Punkte für Instanz.Port)
    tokens = re.findall(r'\b[A-Za-z_][A-Za-z0-9_]*(?:\.[A-Za-z_][A-Za-z0-9_]*)*\b', clean_str)
    
    # 3. Filtere ST-Keywords heraus
    keywords = {"AND", "OR", "NOT", "XOR", "MOD", "TRUE", "FALSE", "THEN", "ELSE", "ELSIF"}
    
    # Duplikate entfernen, Reihenfolge beibehalten
    return list(dict.fromkeys([t for t in tokens if t.upper() not in keywords]))


# ==========================================
# TEIL 3b: Bool-Analyse der IF-Bedingung
# ==========================================

@dataclass(frozen=True)
class BoolExpr:
    kind: str  # 'atom' | 'and' | 'or' | 'not'
    value: Optional[str] = None
    children: Tuple["BoolExpr", ...] = ()


def _strip_outer_parens(s: str) -> str:
    s = s.strip()
    if not (s.startswith("(") and s.endswith(")")):
        return s
    depth = 0
    in_str = False
    for i, ch in enumerate(s):
        if ch == "'" and (i == 0 or s[i - 1] != "\\"):
            in_str = not in_str
        if in_str:
            continue
        if ch == "(":
            depth += 1
        elif ch == ")":
            depth -= 1
            if depth == 0 and i != len(s) - 1:
                return s
    return s[1:-1].strip() if depth == 0 else s


def _split_top_level_bool(s: str, op: str) -> List[str]:
    op_up = op.upper()
    parts: List[str] = []
    buf: List[str] = []
    depth = 0
    in_str = False
    i = 0
    while i < len(s):
        ch = s[i]
        if ch == "'" and (i == 0 or s[i - 1] != "\\"):
            in_str = not in_str
            buf.append(ch)
            i += 1
            continue
        if not in_str:
            if ch == "(":
                depth += 1
            elif ch == ")":
                depth = max(0, depth - 1)
            if depth == 0 and s[i:].upper().startswith(op_up):
                before_ok = i == 0 or not (s[i - 1].isalnum() or s[i - 1] == "_")
                after_ok = (i + len(op_up) >= len(s)) or not (s[i + len(op_up)].isalnum() or s[i + len(op_up)] == "_")
                if before_ok and after_ok:
                    part = "".join(buf).strip()
                    if part:
                        parts.append(part)
                    buf = []
                    i += len(op_up)
                    continue
        buf.append(ch)
        i += 1
    tail = "".join(buf).strip()
    if tail:
        parts.append(tail)
    return parts


def parse_bool_expr(s: str) -> BoolExpr:
    """Sehr einfacher Parser: NOT > AND > OR; Rest wird als Atom behandelt."""
    s = _strip_outer_parens(s.strip())
    or_parts = _split_top_level_bool(s, "OR")
    if len(or_parts) > 1:
        return BoolExpr("or", children=tuple(parse_bool_expr(p) for p in or_parts))
    and_parts = _split_top_level_bool(s, "AND")
    if len(and_parts) > 1:
        return BoolExpr("and", children=tuple(parse_bool_expr(p) for p in and_parts))
    if s.upper().startswith("NOT "):
        return BoolExpr("not", children=(parse_bool_expr(s[4:].strip()),))
    return BoolExpr("atom", value=s)


def render_bool_expr(expr: BoolExpr) -> str:
    if expr.kind == "atom":
        return expr.value or ""
    if expr.kind == "not":
        return f"NOT ({render_bool_expr(expr.children[0])})"
    joiner = " AND " if expr.kind == "and" else " OR "
    return "(" + joiner.join(render_bool_expr(c) for c in expr.children) + ")"


def required_truth_paths(expr: BoolExpr, *, limit: int = 20) -> List[List[str]]:
    """Alternative Minimal-Sets an Teilbedingungen, die TRUE sein müssen.

    Beispiel: (A OR B) AND C -> [[A, C], [B, C]]
    """
    if expr.kind == "atom":
        return [[expr.value or ""]]
    if expr.kind == "not":
        inner = expr.children[0]
        if inner.kind == "atom":
            return [[f"NOT {inner.value}"]]
        return [[f"NOT ({render_bool_expr(inner)})"]]
    if expr.kind == "and":
        paths: List[List[str]] = [[]]
        for c in expr.children:
            child_paths = required_truth_paths(c, limit=limit)
            new_paths: List[List[str]] = []
            for base in paths:
                for add in child_paths:
                    new_paths.append(base + add)
                    if len(new_paths) >= limit:
                        break
                if len(new_paths) >= limit:
                    break
            paths = new_paths
        return paths[:limit]
    if expr.kind == "or":
        out: List[List[str]] = []
        for c in expr.children:
            out.extend(required_truth_paths(c, limit=limit))
            if len(out) >= limit:
                break
        return out[:limit]
    return [[render_bool_expr(expr)]]


def _extract_innermost_if_condition_from_assignment(assign: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    rx = re.compile(r"^(IF|ELSIF)@(\d+):\s*(.*)\s*$", flags=re.I)
    for raw in reversed(assign.get("conditions", []) or []):
        m = rx.match(raw.strip())
        if m:
            return {"tag": m.group(1).upper(), "if_line": int(m.group(2)), "expr": m.group(3).strip(), "raw": raw}
    return None


def run_unified_set_and_condition_trace(
    *,
    ttl_path: str,
    entrypoint: str,
    pou_name: str = "",
    target_var: str = "",
    last_gemma_state_before_failure: str = "",
    state_name: str = "D2",
    gemma_pou_name: str = "",
    max_depth: int = 12,
    max_truth_paths: int = 12,
    trace_each_truth_path: bool = True,
) -> None:
    """
    Vereint Set-Trace + Deep-Trace für die *innere* (letzte) IF-Bedingung, die zu
    `target_var := TRUE;` führt.

    entrypoint:
      - 'port' / 'st': Analysiere ST-Code direkt in `pou_name` für `target_var`
      - 'fbd_state'  : nutze FBD-Export + last_gemma_state -> resolve Upstream-Port -> ST-Analyse
    """

    tr = Tracer(enabled=True, print_live=True)
    tr.log("\n=======================================================")
    tr.log("START: Unified Set-Trace + IF-Condition Deep-Trace")
    tr.log("=======================================================\n")
    tr.log(f"[CFG] ttl_path={ttl_path}")
    tr.log(f"[CFG] entrypoint={entrypoint}")

    g = load_graph(ttl_path, trace=tr)

    context_pou_uri: Optional[URIRef] = None
    context_code: Optional[str] = None
    context_target: Optional[str] = None

    if entrypoint.lower() in ("port", "st"):
        if not pou_name or not target_var:
            tr.log("[ABBRUCH] Für entrypoint='port'/'st' werden pou_name und target_var benötigt.")
            return
        pou_uri = find_pou_by_name(g, pou_name, trace=tr)
        if not pou_uri:
            tr.log(f"[ABBRUCH] POU '{pou_name}' nicht im KG gefunden.")
            return
        st_code = get_pou_code(g, pou_uri, trace=tr)
        if not st_code:
            tr.log(f"[ABBRUCH] Kein dp_hasPOUCode für '{pou_name}'.")
            return

        if entrypoint.lower() == "port":
            port_uri = get_port_by_name(g, pou_uri, target_var, trace=tr)
            if port_uri:
                direction = next(g.objects(port_uri, AG["dp_hasPortDirection"]), None)
                tr.log(f"[CTX] target_var '{target_var}' ist Port (direction={direction})")
            else:
                tr.log(f"[WARN] '{target_var}' ist als Port nicht erkennbar; fahre fort (ST-Analyse).")
        else:
            iv = find_internal_variable_uri(g, pou_uri, target_var)
            if iv:
                tr.log(f"[CTX] target_var '{target_var}' ist interne Variable (Var_* URI gefunden)")
            else:
                tr.log(f"[WARN] '{target_var}' ist als interne Variable nicht erkennbar; fahre fort.")

        context_pou_uri, context_code, context_target = pou_uri, st_code, target_var

    elif entrypoint.lower() == "fbd_state":
        if not last_gemma_state_before_failure:
            tr.log("[ABBRUCH] entrypoint='fbd_state' benötigt last_gemma_state_before_failure.")
            return
        result = find_st_true_set_lines_for_d2_path(
            ttl_path=ttl_path,
            gemma_pou_name=gemma_pou_name,
            state_name=state_name,
            suspected_input_port="",
            last_gemma_state_before_failure=last_gemma_state_before_failure,
            trace=tr,
        )
        if "error" in result:
            tr.log(f"[ABBRUCH] {result['error']}")
            return
        context_pou_uri = URIRef(result["origin"]["caller_pou_uri"])
        context_code = result["origin"]["caller_code"]
        context_target = result["origin"]["caller_port_name"]
        tr.log(f"[CTX] fbd_state resolved -> target_var='{context_target}' (caller_pou_uri={context_pou_uri})")

    else:
        tr.log(f"[ABBRUCH] Unbekannter entrypoint: {entrypoint}")
        return

    tr.log("\n--- ST: Assignment Analyse ---")
    aa = analyze_var_assignments_st(context_code, context_target, trace=tr)
    true_assignments = [a for a in aa.get("assignments", []) if a.get("value") == "TRUE"]
    dominant_assignment = true_assignments[-1] if true_assignments else None
    if not dominant_assignment:
        if entrypoint.lower() == "port":
            tr.log(f"[INFO] Keine TRUE-Zuweisung für '{context_target}' im ST-Code gefunden.")
            tr.log(f"[INFO] ENTRYPOINT='port' -> starte Deep-Trace direkt auf '{context_target}'.")
            tr.log(f"\n=== TRACE START: {context_target} ===")
            trace_lines = trace_expr(g, context_target, context_pou_uri, context_code, max_depth=max_depth)
            for tl in trace_lines:
                tr.log("  " * tl.indent + tl.text)
            diag = summarize_trace_characteristics(trace_lines)
            log_trace_characteristics(tr, context_target, diag)
            tr.log(f"=== TRACE END: {context_target} ===")
            return

        tr.log(f"[ABBRUCH] Keine TRUE-Zuweisung für '{context_target}' im ST-Code gefunden.")
        return

    tr.log(f"[ST] Dominante (letzte) TRUE-Zuweisung @ {dominant_assignment['line_no']}: {dominant_assignment['assignment']}")
    tr.log(f"[ST] Path: {dominant_assignment['conditions_conjunction']}")

    inn = _extract_innermost_if_condition_from_assignment(dominant_assignment)
    if not inn:
        tr.log("[ABBRUCH] Konnte die innere IF-Bedingung nicht extrahieren.")
        return

    tr.log("\n--- Innere IF-Bedingung (Set-Trace Fokus) ---")
    tr.log(f"IF@{inn['if_line']}: {inn['expr']}")

    tr.log("\n--- Bool-Analyse (\"was musste TRUE sein?\") ---")
    bexpr = parse_bool_expr(inn["expr"])
    tr.log(f"Parsed: {render_bool_expr(bexpr)}")
    truth_paths = required_truth_paths(bexpr, limit=max_truth_paths)
    for idx, p in enumerate(truth_paths, start=1):
        tr.log(f"Path #{idx}: " + "  AND  ".join(p))

    path_reports: List[Dict[str, Any]] = []

    if trace_each_truth_path and truth_paths:
        tr.log("\n--- Deep Trace: je Bool-Pfad (Schritte einzeln) ---")
        for idx, p in enumerate(truth_paths, start=1):
            tr.log(f"\n>>> PATH TRACE #{idx}: " + "  AND  ".join(p))
            seen_in_path: Set[str] = set()
            path_report: Dict[str, Any] = {
                "path_index": idx,
                "clauses": p,
                "token_reports": [],
            }
            for clause in p:
                clause_raw = clause.strip()
                is_negated = clause_raw.upper().startswith("NOT ")
                clause_expr = clause_raw[4:].strip() if is_negated else clause_raw
                clause_expr = _strip_outer_parens(clause_expr)
                step_tokens = extract_variables_from_condition(clause_expr)
                if not step_tokens and re.fullmatch(r"[A-Za-z_]\w*(?:\.[A-Za-z_]\w*)*", clause_expr):
                    step_tokens = [clause_expr]

                if not step_tokens:
                    tr.log(f"[STEP] {clause_raw} -> keine Tokens extrahiert")
                    continue

                tr.log(f"[STEP] {clause_raw} -> Tokens: {step_tokens}")
                for tok in step_tokens:
                    if tok in seen_in_path:
                        continue
                    seen_in_path.add(tok)
                    tag = " (NOT-Kontext)" if is_negated else ""
                    tr.log(f"  === STEP TRACE START: {tok}{tag} ===")
                    step_trace_lines = trace_expr(g, tok, context_pou_uri, context_code, max_depth=max_depth)
                    for tl in step_trace_lines:
                        tr.log("    " + ("  " * tl.indent) + tl.text)
                    diag_step = summarize_trace_characteristics(step_trace_lines)
                    log_trace_characteristics(tr, tok, diag_step)
                    token_report = build_token_trace_report(
                        token=tok,
                        clause_raw=clause_raw,
                        is_negated=is_negated,
                        trace_lines=step_trace_lines,
                        summary=diag_step,
                    )
                    path_report["token_reports"].append(token_report)
                    tr.log(f"  === STEP TRACE END: {tok} ===")
            path_reports.append(path_report)

    if path_reports:
        log_detailed_true_condition_diagnosis(
            tr,
            target_var=context_target,
            target_pou_uri=context_pou_uri,
            dominant_assignment=dominant_assignment,
            inner_if=inn,
            truth_paths=truth_paths,
            path_reports=path_reports,
        )

    tr.log("\n--- Deep Trace: Variablen/Ports aus der IF-Bedingung ---")
    vars_to_trace = extract_variables_from_condition(inn["expr"])
    tr.log(f"Extrahierte Tokens: {vars_to_trace}")

    for var in vars_to_trace:
        tr.log(f"\n=== TRACE START: {var} ===")
        trace_lines = trace_expr(g, var, context_pou_uri, context_code, max_depth=max_depth)
        for tl in trace_lines:
            tr.log("  " * tl.indent + tl.text)
        diag = summarize_trace_characteristics(trace_lines)
        log_trace_characteristics(tr, var, diag)
        tr.log(f"=== TRACE END: {var} ===")


def run_full_automated_trace(ttl_path: str, last_gemma_state: str, target_state: str = "D2"):
    """
    Die Hauptmethode, die Skript 1 und Skript 2 vereint.
    """
    tr = Tracer(enabled=True, print_live=True)
    
    tr.log(f"\n=======================================================")
    tr.log(f"STARTE AUTOMATISCHEN GESAMT-TRACE ({target_state} nach {last_gemma_state})")
    tr.log(f"=======================================================\n")

    # 1. Methode 1 ausführen (Findet den Upstream-Port und die Zuweisungen)
    result = find_st_true_set_lines_for_d2_path(
        ttl_path=ttl_path,
        state_name=target_state,
        last_gemma_state_before_failure=last_gemma_state,
        trace=tr
    )

    if "error" in result:
        tr.log(f"\n[ABBRUCH] Es gab einen Fehler in Methode 1: {result['error']}")
        return

    # 2. Letzte "TRUE" Zuweisung ("Last write wins") finden
    assignments = result.get("assignment_analysis", {}).get("assignments", [])
    true_assignments = [a for a in assignments if a["value"] == "TRUE"]
    
    if not true_assignments:
        tr.log("\n[ABBRUCH] Es konnte keine TRUE-Zuweisung im Code gefunden werden.")
        return

    # Wir nehmen die chronologisch allerletzte TRUE Zuweisung im Code
    dominant_assignment = true_assignments[-1]
    dominant_condition = dominant_assignment["conditions_conjunction"]
    
    tr.log(f"\n--- ZERLEGUNG DER DOMINANTEN BEDINGUNG ---")
    tr.log(f"Letzte Zuweisung in Zeile {dominant_assignment['line_no']}: {result['origin']['caller_port_name']} := TRUE")
    tr.log(f"Bedingungspfad: {dominant_condition}")

    # 3. Bedingungen zerlegen
    vars_to_trace = extract_variables_from_condition(dominant_condition)
    tr.log(f"Extrahierte Trace-Variablen: {vars_to_trace}")

    # 4. Methode 2 (Deep Trace) für jede Variable aufrufen
    graph = result["graph"]
    caller_pou_uri = URIRef(result["origin"]["caller_pou_uri"])
    st_code = result["origin"]["caller_code"]

    for var in vars_to_trace:
        tr.log(f"\n=== TRACE START: {var} ===")
        trace_lines = trace_expr(graph, var, caller_pou_uri, st_code, max_depth=10)
        for tl in trace_lines:
            tr.log("  " * tl.indent + tl.text)
        diag = summarize_trace_characteristics(trace_lines)
        log_trace_characteristics(tr, var, diag)
        tr.log(f"=== TRACE END: {var} ===")

# -----------------------------
# Ausführung
# -----------------------------
TTL_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl"
ENTRYPOINT = "fbd_state"  # 'port' | 'st' | 'fbd_state'

# Kontext für ENTRYPOINT='port'/'st'
POU_NAME = "FB_Automatikbetrieb_F1"
TARGET_VAR = "Stoerung_erkannt"

# Kontext für ENTRYPOINT='fbd_state' (optional)
GEMMA_POU_NAME = ""  # leer -> auto GEMMA POU via KG-Flag dp_isGEMMAStateMachine
STATE_NAME = "D2"
LAST_GEMMA_STATE_BEFORE_FAILURE = "F1"

run_unified_set_and_condition_trace(
    ttl_path=TTL_PATH,
    entrypoint=ENTRYPOINT,
    pou_name=POU_NAME,
    target_var=TARGET_VAR,
    gemma_pou_name=GEMMA_POU_NAME,
    state_name=STATE_NAME,
    last_gemma_state_before_failure=LAST_GEMMA_STATE_BEFORE_FAILURE,
    max_depth=200,
    max_truth_paths=200,
)



START: Unified Set-Trace + IF-Condition Deep-Trace

[CFG] ttl_path=D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl
[CFG] entrypoint=fbd_state
[RUN] Starting query chain
[RUN] gemma_pou_name not provided -> auto-detect GEMMA POU via KG flags
[KG] GEMMA candidates (CustomFBType + isGEMMAStateMachine=true): 1
[KG]   GEMMA POU: FB_Betriebsarten -> http://www.semanticweb.org/AgentProgramParams/FBType_FB_Betriebsarten
[RUN] Selected GEMMA POU deterministically: FB_Betriebsarten
[KG] dp_hasPOUCode found for http://www.semanticweb.org/AgentProgramParams/FBType_FB_Betriebsarten (length=6448 chars)
[RUN] Tracing state 'D2' in FBD Python export
[AUTO] Found OR node: OR_5(AND_5(Auto_Stoerung, F1), AND_6(GVL.DiagnoseRequested, D1, GVL.NotStopp), AND_7(D3, Alt_abort))
[AUTO] OR branches:
  - b1: AND_5(Auto_Stoerung, F1)
      states=['F1'] contains_last=True
  - b2: AND_6(GVL.DiagnoseRequested, D1, GVL.NotStopp)
      states=['D1'] contains_last=False
  - b3: AND_7(D3, Alt_abort)
      sta

In [20]:
#Zelle 4
# -----------------------------
# 4) Konkrete Pfad-Diagnose (D2 + Stoerung_erkannt)
# -----------------------------

from __future__ import annotations

import re
from typing import Any, Dict, List, Optional, Set, Tuple


def _pou_label_from_uri(uri: Optional[str]) -> str:
    if not uri:
        return "-"
    s = str(uri)
    tail = s.split("/")[-1]
    for prefix in ("FBType_", "Program_"):
        if tail.startswith(prefix):
            return tail[len(prefix):]
    return tail


def _normalize_pou_label(raw_pou: Optional[str], default_pou: str) -> str:
    """
    Normalisiert kurze/uneinheitliche POU-Labels aus Freitext-Logs.
    Beispiel: 'F1' im Kontext von FB_Automatikbetrieb_F1 -> FB_Automatikbetrieb_F1.
    """
    if not raw_pou:
        return default_pou
    pou = str(raw_pou).strip()
    if not pou:
        return default_pou

    if pou in {"F1", "F2", "F3", "D1", "D2", "D3", "A1", "A2", "A3", "A4", "A5", "A6", "A7"}:
        if default_pou.upper().endswith(pou.upper()):
            return default_pou

    if pou == "Betriebsarten":
        return "FB_Betriebsarten"
    if pou == "MAIN":
        return "MAIN"

    return pou


def _phase_label(phase_idx: int) -> str:
    if phase_idx == 0:
        return "t0"
    if phase_idx < 0:
        return f"t{phase_idx}"
    return f"t+{phase_idx}"


def _clone_rows(rows: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    out: Dict[str, Dict[str, Any]] = {}
    for k, v in rows.items():
        out[k] = {
            "symbol": v["symbol"],
            "values": set(v.get("values", set())),
            "pous": set(v.get("pous", set())),
            "phases": set(v.get("phases", set())),
            "reasons": list(v.get("reasons", [])),
        }
    return out


def _upsert_row(
    rows: Dict[str, Dict[str, Any]],
    symbol: str,
    value: Optional[str] = None,
    pou: Optional[str] = None,
    phase: Optional[str] = None,
    reason: Optional[str] = None,
) -> None:
    key = symbol.strip()
    if not key:
        return
    if key not in rows:
        rows[key] = {
            "symbol": key,
            "values": set(),
            "pous": set(),
            "phases": set(),
            "reasons": [],
        }

    if value is not None:
        v = str(value).strip()
        if v:
            rows[key]["values"].add(v)

    if pou is not None:
        p = str(pou).strip()
        if p:
            rows[key]["pous"].add(p)

    if phase is not None and value is not None and str(value).strip():
        ph = str(phase).strip()
        if ph:
            rows[key]["phases"].add(ph)

    if reason:
        r = str(reason).strip()
        if r and r not in rows[key]["reasons"]:
            rows[key]["reasons"].append(r)


def _merge_rows(dst: Dict[str, Dict[str, Any]], src: Dict[str, Dict[str, Any]]) -> None:
    for sym, item in src.items():
        if sym not in dst:
            dst[sym] = {
                "symbol": sym,
                "values": set(),
                "pous": set(),
                "phases": set(),
                "reasons": [],
            }
        dst[sym]["values"].update(item.get("values", set()))
        dst[sym]["pous"].update(item.get("pous", set()))
        dst[sym]["phases"].update(item.get("phases", set()))
        for r in item.get("reasons", []):
            if r not in dst[sym]["reasons"]:
                dst[sym]["reasons"].append(r)


def _bool_invert(v: str) -> str:
    return "FALSE" if str(v).upper() == "TRUE" else "TRUE"


def _likely_bool_literal(expr: str) -> Optional[str]:
    e = expr.strip().upper()
    if e == "TRUE":
        return "TRUE"
    if e == "FALSE":
        return "FALSE"
    return None


def _extract_signal_tokens(expr: str) -> List[str]:
    """
    Extrahiert Token aus Ausdrücken, ignoriert dabei typische Zeitliterale (z.B. T#55S).
    """
    raw = expr.strip()
    if re.fullmatch(r"T#[0-9A-Za-z_]+", raw, flags=re.I):
        return []

    cleaned = re.sub(r"\bT#[0-9A-Za-z_]+\b", " ", raw, flags=re.I)
    tokens = extract_variables_from_condition(cleaned)

    out: List[str] = []
    for tok in tokens:
        t = tok.strip()
        if not t:
            continue
        if t.upper() == "T":
            continue
        if re.fullmatch(r"\d+(\.\d+)?", t):
            continue
        out.append(t)
    return out


def _collect_positive_atoms(expr: BoolExpr, polarity: int = 1) -> List[str]:
    out: List[str] = []
    if expr.kind == "atom":
        if polarity > 0 and expr.value:
            out.append(expr.value.strip())
        return out
    if expr.kind == "not":
        if expr.children:
            out.extend(_collect_positive_atoms(expr.children[0], -polarity))
        return out
    for child in expr.children:
        out.extend(_collect_positive_atoms(child, polarity))
    return out


def _positive_tokens_for_condition(cond_expr: str) -> Set[str]:
    """
    Liefert nur die positiv geforderten Tokens einer Bedingung.
    Beispiel:
      rStart.Q AND NOT (Schritt1 OR Schritt2) -> {rStart.Q}
      Schritt1 AND tSchritt1.Q                -> {Schritt1, tSchritt1.Q}
    """
    tokens: Set[str] = set()
    try:
        be = parse_bool_expr(cond_expr)
        atoms = _collect_positive_atoms(be, polarity=1)
        for a in atoms:
            for tok in _extract_signal_tokens(a):
                tokens.add(tok)
    except Exception:
        for tok in _extract_signal_tokens(cond_expr):
            tokens.add(tok)
    return tokens


def parse_trace_requirements_from_lines(
    trace_lines: List[TraceLine],
    *,
    default_pou: str,
    desired_value: str = "TRUE",
    base_phase_idx: int = 0,
) -> Tuple[Dict[str, Dict[str, Any]], Dict[str, List[str]]]:
    """
    Verdichtet freie Trace-Lines zu konkreten Wert-Anforderungen.
    Ziel: pro Port/Variable m?gliche Sollwerte + POU + Begr?ndung erfassen.
    """
    rows: Dict[str, Dict[str, Any]] = {}
    markers = collect_trace_markers(trace_lines) if "collect_trace_markers" in globals() else {
        "classifications": [], "if_branches": [], "nodes": [], "call_and_wire": [], "terminals": []
    }

    current_symbol: Optional[str] = None
    current_pou: Optional[str] = default_pou
    current_instance: Optional[str] = None
    current_instance_type: Optional[str] = None
    current_output_port: Optional[str] = None
    instance_ctx_by_indent: Dict[int, Tuple[str, str, str]] = {}
    assignment_ctx_by_indent: Dict[int, Set[str]] = {}
    active_phase_by_indent: Dict[int, int] = {-1: base_phase_idx}
    skip_branch_indent: Optional[int] = None
    wire_edges: List[Tuple[str, str, str]] = []  # (dest, src, caller_pou)

    rx_inst_port = re.compile(r"^([A-Za-z_]\w*(?:\.[A-Za-z_]\w*)?)\s+ist\s+Port\s+'([^']+)'\s+der\s+FB-Instanz\s+'([^']+)'\s+\(Typ=([^)]+)\)")
    rx_input_output_port = re.compile(r"^'([^']+)'\s+ist\s+(Input|Output)-Port\s+von\s+(.+)$")
    rx_internal_var = re.compile(r"^'([^']+)'\s+ist\s+interne\s+Variable\s+in\s+(.+)$")
    rx_global_var = re.compile(r"^([A-Za-z_]\w*(?:\.[A-Za-z_]\w*)?)\s+ist\s+globale\s+Variable")
    rx_default = re.compile(r"^default_value\s*=\s*(.*)$")
    rx_st_fallback_q = re.compile(r"^\[ST-FALLBACK\]\s*Q\s*<=\s*([A-Za-z_]+)\s*<=\s*(.+)$")
    rx_st_map = re.compile(r"^\[ST-MAP\]\s*Input-Port\s+'([^']+)'\s*<=\s*(.+)$")
    rx_fbd_map = re.compile(r"^\[FBD->CALLMAP\]\s*([A-Za-z_]\w*)\s*<=\s*(.+)$")
    rx_kg_wire = re.compile(r"^\[KG-WIRE\]\s*([A-Za-z_]\w*(?:\.[A-Za-z_]\w*)?)\s*<=\s*(.+?)\s*\(caller_pou=([^)]+)\)")
    rx_assume_false = re.compile(r"^\[ASSUME\]\s*([A-Za-z_]\w*)=FALSE")
    rx_loop = re.compile(r"^\(loop detected\)\s+(.+)$")
    rx_max_depth = re.compile(r"^\(max depth\)\s+(.+)$")
    rx_last_true_if = re.compile(r"^Letzte TRUE-Zuweisung @ \d+ \(innere IF\):\s*(.+)$")
    rx_cls_token = re.compile(r"^\[CLS\]\s+token='([^']+)'\s+kind=")

    positive_params = {"IN", "CLK", "S", "SET"}
    negative_params = {"R", "RESET"}

    for tl in trace_lines:
        line_indent = int(getattr(tl, "indent", 0) or 0)
        for k in list(instance_ctx_by_indent.keys()):
            if k > line_indent:
                instance_ctx_by_indent.pop(k, None)
        for k in list(assignment_ctx_by_indent.keys()):
            if k >= line_indent:
                assignment_ctx_by_indent.pop(k, None)
        for k in list(active_phase_by_indent.keys()):
            if k > line_indent:
                active_phase_by_indent.pop(k, None)

        if skip_branch_indent is not None:
            if line_indent > skip_branch_indent:
                continue
            skip_branch_indent = None

        phase_keys = [k for k in active_phase_by_indent.keys() if k <= line_indent]
        line_phase_idx = active_phase_by_indent[max(phase_keys)] if phase_keys else base_phase_idx
        line_phase = _phase_label(line_phase_idx)

        text = tl.text.strip()
        if not text:
            continue

        m_last_true = rx_last_true_if.match(text)
        if m_last_true:
            cond_expr = m_last_true.group(1).strip()
            assignment_ctx_by_indent[line_indent] = _positive_tokens_for_condition(cond_expr)
            active_phase_by_indent[line_indent + 1] = line_phase_idx - 1
            continue

        m_cls = rx_cls_token.match(text)
        if m_cls and assignment_ctx_by_indent:
            tok = m_cls.group(1).strip()
            parent_candidates = [i for i in assignment_ctx_by_indent.keys() if i < line_indent]
            if parent_candidates:
                parent_indent = max(parent_candidates)
                allowed = assignment_ctx_by_indent.get(parent_indent, set())
                if line_indent == parent_indent + 1 and allowed and tok not in allowed:
                    skip_branch_indent = parent_indent
                    continue

        m_inst = rx_inst_port.match(text)
        if m_inst:
            tok = m_inst.group(1).strip()
            port_name = m_inst.group(2).strip()
            inst = m_inst.group(3).strip()
            typ = m_inst.group(4).strip()
            current_symbol = tok
            current_pou = default_pou
            current_instance = inst
            current_instance_type = typ
            current_output_port = port_name
            instance_ctx_by_indent[line_indent] = (inst, typ, port_name)
            _upsert_row(rows, tok, None, default_pou, line_phase, f"Instanz-Port {inst}.{port_name} (Typ={typ})")
            continue

        m_iop = rx_input_output_port.match(text)
        if m_iop:
            sym = m_iop.group(1).strip()
            direction = m_iop.group(2).strip()
            pou = _normalize_pou_label(m_iop.group(3).strip(), default_pou)
            current_symbol = sym
            current_pou = pou
            _upsert_row(rows, sym, None, pou, line_phase, f"{direction}-Port")
            continue

        m_int = rx_internal_var.match(text)
        if m_int:
            sym = m_int.group(1).strip()
            pou = _normalize_pou_label(m_int.group(2).strip(), default_pou)
            current_symbol = sym
            current_pou = pou
            _upsert_row(rows, sym, None, pou, line_phase, "interne Variable")
            continue

        m_glob = rx_global_var.match(text)
        if m_glob:
            sym = m_glob.group(1).strip()
            current_symbol = sym
            current_pou = "global"
            _upsert_row(rows, sym, None, "global", line_phase, "globale Variable")
            continue

        m_def = rx_default.match(text)
        if m_def and current_symbol:
            dv = m_def.group(1).strip()
            _upsert_row(rows, current_symbol, dv, current_pou, line_phase, "Default-Wert aus KG")
            continue

        if "is_wired = False" in text and current_symbol:
            _upsert_row(rows, current_symbol, None, current_pou, line_phase, "nicht verdrahtet")
            continue

        m_st_fb = rx_st_fallback_q.match(text)
        if m_st_fb:
            param = m_st_fb.group(1).strip().upper()
            expr = m_st_fb.group(2).strip()
            expr_tokens = _extract_signal_tokens(expr)
            expr_upper = expr.upper().strip()
            local_expected: Optional[str] = None

            inst_ctx = None
            if instance_ctx_by_indent:
                candidates = [i for i in instance_ctx_by_indent.keys() if i <= line_indent]
                if candidates:
                    inst_ctx = instance_ctx_by_indent[max(candidates)]

            inst_name = inst_ctx[0] if inst_ctx else current_instance
            out_port = inst_ctx[2] if inst_ctx else current_output_port
            fb_port_symbol = f"{inst_name}.{param}" if inst_name else param
            dep_phase_idx = line_phase_idx - 1
            dep_phase = _phase_label(dep_phase_idx)

            if param in positive_params:
                local_expected = "TRUE" if desired_value.upper() == "TRUE" else "FALSE"
            elif param in negative_params:
                local_expected = "FALSE" if desired_value.upper() == "TRUE" else "TRUE"
            elif param == "PT":
                local_expected = None

            if param == "PT":
                # PT muss "abgelaufen" sein, damit Q TRUE wird.
                pt_window = f"{dep_phase}..{line_phase}"
                _upsert_row(
                    rows,
                    fb_port_symbol,
                    f"abgelaufen ({expr})",
                    default_pou,
                    pt_window,
                    f"{fb_port_symbol} bestimmt die Impuls-/Timerdauer für {inst_name}.{out_port or 'Q'}",
                )
                if expr_tokens:
                    for tok in expr_tokens:
                        _upsert_row(rows, tok, "abgelaufen", None, pt_window, f"PT-Vorgabe für {fb_port_symbol}: {expr}")
                else:
                    # Reines Literal (z.B. T#55S): nur am FB-Port vermerken, kein eigenes Symbol erzeugen.
                    _upsert_row(rows, fb_port_symbol, expr, default_pou, pt_window, "PT-Literal im Call")
                continue

            # Spezialfall: IN/CLK bekommt NOT X -> X muss invertiert sein
            if expr_upper.startswith("NOT ") and local_expected in ("TRUE", "FALSE"):
                local_expected = _bool_invert(local_expected)

            _upsert_row(
                rows,
                fb_port_symbol,
                local_expected,
                default_pou,
                dep_phase,
                f"{fb_port_symbol} treibt {inst_name}.{out_port or 'Q'}",
            )

            if expr_tokens:
                for tok in expr_tokens:
                    _upsert_row(rows, tok, local_expected, None, dep_phase, f"Q-Trigger über {param}: {expr}")
            else:
                lit = _likely_bool_literal(expr)
                if lit is not None:
                    _upsert_row(rows, expr, lit, None, dep_phase, f"Q-Trigger über {param}")
            continue

        m_st_map = rx_st_map.match(text)
        if m_st_map:
            param = m_st_map.group(1).strip()
            expr = m_st_map.group(2).strip()
            toks = _extract_signal_tokens(expr)
            for tok in toks:
                _upsert_row(rows, tok, None, None, line_phase, f"Input-Mapping {param} <= {expr}")
            continue

        m_fbd_map = rx_fbd_map.match(text)
        if m_fbd_map:
            left = m_fbd_map.group(1).strip()
            right = m_fbd_map.group(2).strip()
            toks = _extract_signal_tokens(right)
            _upsert_row(rows, left, None, None, line_phase, f"FBD Call-Mapping {left} <= {right}")
            for tok in toks:
                _upsert_row(rows, tok, None, None, line_phase, f"FBD Call-Mapping {left} <= {right}")
            continue

        m_wire = rx_kg_wire.match(text)
        if m_wire:
            dest = m_wire.group(1).strip()
            src_expr = m_wire.group(2).strip()
            caller_pou_uri = m_wire.group(3).strip()
            caller_pou = _pou_label_from_uri(caller_pou_uri)

            _upsert_row(rows, dest, None, None, line_phase, f"Verdrahtung: {dest} <= {src_expr}")
            src_tokens = _extract_signal_tokens(src_expr)
            for tok in src_tokens:
                _upsert_row(rows, tok, None, caller_pou, line_phase, f"Verdrahtung nach {dest}")
                wire_edges.append((dest, tok, caller_pou))
            continue

        m_assume = rx_assume_false.match(text)
        if m_assume:
            sym = m_assume.group(1).strip()
            _upsert_row(rows, sym, "FALSE", None, line_phase, "Annahme im GEMMA-Layer")
            continue

        m_loop = rx_loop.match(text)
        if m_loop:
            sym = m_loop.group(1).strip()
            _upsert_row(rows, sym, None, None, line_phase, "loop detected")
            continue

        m_md = rx_max_depth.match(text)
        if m_md:
            sym = m_md.group(1).strip()
            _upsert_row(rows, sym, None, None, line_phase, "max depth erreicht")
            continue

    # Verdrahtungs-Propagation: wenn Ziel bool-Wert hat, auf Quelle übertragen
    for dest, src, caller_pou in wire_edges:
        dest_values = rows.get(dest, {}).get("values", set())
        dest_phases = rows.get(dest, {}).get("phases", set())
        bool_vals = [v for v in dest_values if str(v).upper() in ("TRUE", "FALSE")]
        for bv in bool_vals:
            if dest_phases:
                for ph in dest_phases:
                    _upsert_row(rows, src, bv, caller_pou, ph, f"Wert über Verdrahtung von {dest}")
            else:
                _upsert_row(rows, src, bv, caller_pou, None, f"Wert über Verdrahtung von {dest}")

    return rows, markers


def _phase_sort_key(phase_text: str) -> Tuple[int, str]:
    p = str(phase_text).strip()
    if ".." in p:
        left = p.split("..", 1)[0].strip()
    else:
        left = p
    if left == "t0":
        return (0, p)
    m = re.match(r"^t(-?\d+)$", left)
    if m:
        return (int(m.group(1)), p)
    m2 = re.match(r"^t\+(\d+)$", left)
    if m2:
        return (int(m2.group(1)), p)
    return (999, p)


def rows_to_records(rows: Dict[str, Dict[str, Any]], max_reasons: int = 4) -> List[Dict[str, str]]:
    recs: List[Dict[str, str]] = []
    for sym in sorted(rows.keys()):
        item = rows[sym]
        vals = sorted(item.get("values", set()))
        pous = sorted(item.get("pous", set()))
        phases = sorted(item.get("phases", set()), key=_phase_sort_key)
        reasons = item.get("reasons", [])

        recs.append(
            {
                "Port/Variable": sym,
                "Wert": ", ".join(vals) if vals else "unbekannt",
                "POU": ", ".join(pous) if pous else "-",
                "Zeitphase": ", ".join(phases) if phases else "-",
                "Begr?ndung": " | ".join(reasons[:max_reasons]) if reasons else "-",
            }
        )
    return recs


def format_records_table(records: List[Dict[str, str]]) -> str:
    if not records:
        return "(keine Daten)"

    cols = ["Port/Variable", "Wert", "POU", "Zeitphase", "Begr?ndung"]
    widths = {c: len(c) for c in cols}

    for r in records:
        for c in cols:
            widths[c] = max(widths[c], len(str(r.get(c, ""))))

    def _line(ch: str = "-") -> str:
        return "+" + "+".join(ch * (widths[c] + 2) for c in cols) + "+"

    out: List[str] = []
    out.append(_line("-"))
    out.append("| " + " | ".join(c.ljust(widths[c]) for c in cols) + " |")
    out.append(_line("="))
    for r in records:
        out.append("| " + " | ".join(str(r.get(c, "")).ljust(widths[c]) for c in cols) + " |")
    out.append(_line("-"))
    return "\n".join(out)


def build_concrete_d2_error_paths(
    ttl_path: str,
    *,
    state_name: str = "D2",
    last_gemma_state_before_failure: str = "F1",
    gemma_pou_name: str = "",
    max_depth: int = 40,
    max_truth_paths: int = 8,
    verbose_core_trace: bool = False,
) -> Dict[str, Any]:
    tr = Tracer(enabled=verbose_core_trace, print_live=verbose_core_trace)

    result = find_st_true_set_lines_for_d2_path(
        ttl_path=ttl_path,
        gemma_pou_name=gemma_pou_name,
        state_name=state_name,
        suspected_input_port="",
        last_gemma_state_before_failure=last_gemma_state_before_failure,
        trace=tr,
    )
    if "error" in result:
        return {"error": result["error"], "trace_log": tr.lines}

    graph = result.get("graph") or load_graph(ttl_path, trace=None)
    caller_pou_uri = URIRef(result["origin"]["caller_pou_uri"])
    caller_pou_name = _pou_name(graph, caller_pou_uri) if "_pou_name" in globals() else _pou_label_from_uri(str(caller_pou_uri))
    st_code = result["origin"]["caller_code"]
    target_var = result["origin"]["caller_port_name"]

    aa = analyze_var_assignments_st(st_code, target_var, trace=None)
    true_assignments = [a for a in aa.get("assignments", []) if a.get("value") == "TRUE"]
    dominant_assignment = true_assignments[-1] if true_assignments else None
    if not dominant_assignment:
        return {"error": f"Keine TRUE-Zuweisung f?r '{target_var}' gefunden."}

    inner = _extract_innermost_if_condition_from_assignment(dominant_assignment)
    if not inner:
        return {"error": "Innere IF-Bedingung konnte nicht extrahiert werden."}

    bool_expr = parse_bool_expr(inner["expr"])
    truth_paths = required_truth_paths(bool_expr, limit=max_truth_paths)

    base_rows: Dict[str, Dict[str, Any]] = {}

    gemma_name = result.get("gemma_pou_name") or "FB_Betriebsarten"
    suspected_input = result.get("suspected_input_port")

    _upsert_row(
        base_rows,
        state_name,
        "TRUE",
        gemma_name,
        "t0",
        f"Zielzustand wurde gesetzt ({state_name}=TRUE)",
    )
    if last_gemma_state_before_failure:
        _upsert_row(
            base_rows,
            last_gemma_state_before_failure,
            "TRUE",
            gemma_name,
            "t-1",
            "vorheriger GEMMA-Zustand laut Fehlerkontext",
        )
    if suspected_input:
        _upsert_row(
            base_rows,
            suspected_input,
            "TRUE",
            gemma_name,
            "t0",
            f"Setzpfad f?r {state_name} ben?tigt Eingang '{suspected_input}'",
        )

    _upsert_row(
        base_rows,
        target_var,
        "TRUE",
        caller_pou_name,
        "t0",
        f"dominante TRUE-Zuweisung @ line {dominant_assignment['line_no']}",
    )

    path_reports: List[Dict[str, Any]] = []

    for idx, path in enumerate(truth_paths, start=1):
        rows = _clone_rows(base_rows)
        token_details: List[Dict[str, Any]] = []
        seen_tokens: Set[str] = set()

        for clause in path:
            clause_raw = clause.strip()
            is_negated = clause_raw.upper().startswith("NOT ")
            clause_expr = clause_raw[4:].strip() if is_negated else clause_raw
            clause_expr = _strip_outer_parens(clause_expr)

            step_tokens = extract_variables_from_condition(clause_expr)
            if not step_tokens and re.fullmatch(r"[A-Za-z_]\w*(?:\.[A-Za-z_]\w*)*", clause_expr):
                step_tokens = [clause_expr]

            clause_value = "FALSE" if is_negated else "TRUE"

            for tok in step_tokens:
                if tok in seen_tokens:
                    continue
                seen_tokens.add(tok)

                _upsert_row(rows, tok, clause_value, caller_pou_name, "t0", f"IF-Klausel: {clause_raw}")

                trace_lines = trace_expr(
                    graph,
                    tok,
                    caller_pou_uri,
                    st_code,
                    max_depth=max_depth,
                )
                parsed_rows, markers = parse_trace_requirements_from_lines(
                    trace_lines,
                    default_pou=caller_pou_name,
                    desired_value=clause_value,
                    base_phase_idx=0,
                )
                _merge_rows(rows, parsed_rows)

                token_details.append(
                    {
                        "token": tok,
                        "clause": clause_raw,
                        "expected_value": clause_value,
                        "markers": markers,
                    }
                )

        path_reports.append(
            {
                "path_index": idx,
                "clause_path": path,
                "rows": rows,
                "token_details": token_details,
            }
        )

    return {
        "context": {
            "ttl_path": ttl_path,
            "target_state": state_name,
            "target_var": target_var,
            "target_pou": caller_pou_name,
            "dominant_assignment_line": dominant_assignment["line_no"],
            "dominant_if": inner["expr"],
            "last_gemma_state_before_failure": last_gemma_state_before_failure,
            "suspected_input_for_state": suspected_input,
            "gemma_pou_name": gemma_name,
        },
        "truth_paths": truth_paths,
        "paths": path_reports,
    }


def print_concrete_d2_error_paths(
    report: Dict[str, Any],
    *,
    max_marker_lines: int = 8,
    max_reasons: int = 4,
) -> None:
    if "error" in report:
        print("[ERROR]", report["error"])
        return

    ctx = report["context"]
    print("\n=== KONTEXT ===")
    print("Zielzustand:", ctx["target_state"], "| GEMMA-POU:", ctx["gemma_pou_name"])
    print("Zielvariable:", ctx["target_var"], "| POU:", ctx["target_pou"])
    print("Dominante TRUE-Zuweisung Zeile:", ctx["dominant_assignment_line"])
    print("Innere IF:", ctx["dominant_if"])
    print("Last GEMMA State:", ctx["last_gemma_state_before_failure"])
    print("Setz-Eingang f?r Zustand:", ctx["suspected_input_for_state"])

    for p in report.get("paths", []):
        idx = p["path_index"]
        print(f"\n\n=== PFAD #{idx} ===")
        print("Bool-Pfad:", "  AND  ".join(p["clause_path"]))

        for td in p.get("token_details", []):
            print(f"\n[TOKEN] {td['token']} | erwarteter Wert: {td['expected_value']} | Klausel: {td['clause']}")
            markers = td.get("markers", {})

            ifs = markers.get("if_branches", [])
            nodes = markers.get("nodes", [])
            flow = markers.get("call_and_wire", [])
            terms = markers.get("terminals", [])

            print("  IF-Verzweigungen:")
            if ifs:
                for item in ifs[:max_marker_lines]:
                    print("   -", item)
            else:
                print("   - (keine)")

            print("  Knoten:")
            if nodes:
                for item in nodes[:max_marker_lines]:
                    print("   -", item)
            else:
                print("   - (keine)")

            print("  Call/Wire:")
            if flow:
                for item in flow[:max_marker_lines]:
                    print("   -", item)
            else:
                print("   - (keine)")

            print("  End-Hinweise:")
            if terms:
                for item in terms[:max_marker_lines]:
                    print("   -", item)
            else:
                print("   - (keine)")

        print("\nTabelle Pfadbedingungen (Port/Variable, Wert, POU):")
        records = rows_to_records(p["rows"], max_reasons=max_reasons)
        print(format_records_table(records))


# -----------------------------
# Ausf?hrung (konkrete Ketten)
# -----------------------------
TTL_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl"

concrete_report = build_concrete_d2_error_paths(
    ttl_path=TTL_PATH,
    state_name="D2",
    last_gemma_state_before_failure="F1",
    gemma_pou_name="",
    max_depth=40,
    max_truth_paths=8,
    verbose_core_trace=False,
)

print_concrete_d2_error_paths(
    concrete_report,
    max_marker_lines=10,
    max_reasons=5,
)




=== KONTEXT ===
Zielzustand: D2 | GEMMA-POU: FB_Betriebsarten
Zielvariable: Stoerung_erkannt | POU: FB_Automatikbetrieb_F1
Dominante TRUE-Zuweisung Zeile: 78
Innere IF: (Schritt1 OR Schritt2) AND pPer.Q
Last GEMMA State: F1
Setz-Eingang f?r Zustand: Auto_Stoerung


=== PFAD #1 ===
Bool-Pfad: Schritt1  AND  pPer.Q

[TOKEN] Schritt1 | erwarteter Wert: TRUE | Klausel: Schritt1
  IF-Verzweigungen:
   - Letzte TRUE-Zuweisung @ 40 (innere IF): rStart.Q AND NOT (Schritt1 OR Schritt2)
   - Letzte TRUE-Zuweisung @ 51 (innere IF): Schritt1 AND tSchritt1.Q
  Knoten:
   - 'Schritt1' ist interne Variable in F1
   - rStart.Q ist Port 'Q' der FB-Instanz 'rStart' (Typ=R_TRIG)
   - 'Automatikbetrieb_Starten' ist Input-Port von F1
   - edgeF1.Q ist Port 'Q' der FB-Instanz 'edgeF1' (Typ=R_TRIG)
   - fbBA.F1 ist Port 'F1' der FB-Instanz 'fbBA' (Typ=FB_Betriebsarten)
   - GVL.Start ist globale Variable (scope=global, type=BOOL).
   - 'A1' ist Output-Port von Betriebsarten
   - 'A6' ist Output-Port von Bet

In [23]:
#Zelle 5
# 5) Vollständige Signalkette inkl. Event (Trigger -> Skill -> D2)
from __future__ import annotations

import json
import re
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple
from rdflib import URIRef


def _require_global(name: str) -> None:
    if name not in globals():
        raise RuntimeError(f"Benötigte Funktion/Variable '{name}' fehlt. Bitte zuerst Zelle 3 und 4 ausführen.")


def _load_event_context(event_json_path: str) -> Dict[str, Any]:
    if not event_json_path:
        return {}
    p = Path(event_json_path)
    if not p.exists():
        return {"error": f"Event-Datei nicht gefunden: {event_json_path}"}
    obj = json.loads(p.read_text(encoding="utf-8"))
    payload = obj.get("payload", {}) if isinstance(obj, dict) else {}
    snap = payload.get("plcSnapshot", {}) if isinstance(payload, dict) else {}
    vars_list = snap.get("vars", []) if isinstance(snap, dict) else []

    vals: Dict[str, Any] = {}
    for it in vars_list:
        if isinstance(it, dict) and "id" in it:
            vals[str(it["id"])] = it.get("v")

    def _get(*names: str):
        for n in names:
            if n in vals:
                return vals[n]
        for n in names:
            for k, v in vals.items():
                if k.endswith(n):
                    return v
        return None

    return {
        "event_type": obj.get("type"),
        "trigger_event": payload.get("triggerEvent"),
        "process_name": payload.get("processName"),
        "summary": payload.get("summary"),
        "snapshot_vars": vals,
        "last_skill": _get("OPCUA.lastExecutedSkill", "lastExecutedSkill"),
        "last_process": _get("OPCUA.lastExecutedProcess", "lastExecutedProcess"),
        "last_gemma_state": _get("OPCUA.LastGEMMAStateBeforeFailure", "LastGEMMAStateBeforeFailure"),
        "trigger_d2": _get("OPCUA.TriggerD2", "TriggerD2"),
    }


def _snap(token: str, ctx: Dict[str, Any]) -> Any:
    m = ctx.get("snapshot_vars", {}) if isinstance(ctx, dict) else {}
    if not isinstance(m, dict):
        return None
    if token in m:
        return m[token]
    for k, v in m.items():
        if k.endswith(token):
            return v
    return None


def _strip_st_comments_local(st: str) -> str:
    st = re.sub(r"\(\*.*?\*\)", "", st, flags=re.S)
    st = re.sub(r"//.*?$", "", st, flags=re.M)
    return st


def _extract_var_assignments_any_st(st_code: str, var_name: str) -> List[Dict[str, Any]]:
    clean = _strip_st_comments_local(st_code)
    lines = clean.splitlines()

    rx_if = re.compile(r"^\s*IF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_elsif = re.compile(r"^\s*ELSIF\s+(.*?)\s+THEN\s*$", flags=re.I)
    rx_else = re.compile(r"^\s*ELSE\s*$", flags=re.I)
    rx_end = re.compile(r"^\s*END_IF\s*;?\s*$", flags=re.I)
    rx_assign = re.compile(rf"^\s*{re.escape(var_name)}\s*:=\s*(.+?)\s*;\s*$", flags=re.I)

    if_stack: List[Dict[str, Any]] = []
    results: List[Dict[str, Any]] = []

    def _stack_repr() -> List[str]:
        rep: List[str] = []
        for e in if_stack:
            if e["branch"] == "IF":
                rep.append(f"IF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSIF":
                rep.append(f"ELSIF@{e['if_start_line']}: {e['cond']}")
            elif e["branch"] == "ELSE":
                rep.append(f"ELSE@{e['if_start_line']} (zu IF: {e['if_cond']})")
        return rep

    for i, raw in enumerate(lines, start=1):
        line = raw.strip()
        if not line:
            continue
        m_if = rx_if.match(line)
        if m_if:
            cond = m_if.group(1).strip()
            if_stack.append({"branch": "IF", "cond": cond, "if_start_line": i, "if_cond": cond})
            continue
        m_elsif = rx_elsif.match(line)
        if m_elsif:
            cond = m_elsif.group(1).strip()
            if not if_stack:
                if_stack.append({"branch": "ELSIF", "cond": cond, "if_start_line": i, "if_cond": cond})
            else:
                top = if_stack[-1]
                top["branch"] = "ELSIF"
                top["cond"] = cond
            continue
        if rx_else.match(line):
            if if_stack:
                top = if_stack[-1]
                top["branch"] = "ELSE"
                top["cond"] = ""
            continue
        if rx_end.match(line):
            if if_stack:
                if_stack.pop()
            continue
        m_as = rx_assign.match(line)
        if m_as:
            rhs = m_as.group(1).strip()
            conds = _stack_repr()
            results.append(
                {
                    "line_no": i,
                    "assignment": raw.rstrip(),
                    "rhs": rhs,
                    "conditions": conds,
                    "conditions_conjunction": " AND ".join(conds) if conds else "(keine IF-Bedingung im Scope)",
                }
            )
    return results


def find_pou_setters_for_variable(graph, var_name: str) -> List[Dict[str, Any]]:
    _require_global("AG")
    out: List[Dict[str, Any]] = []
    for pou_uri, _, code_lit in graph.triples((None, AG["dp_hasPOUCode"], None)):
        code = str(code_lit)
        assigns = _extract_var_assignments_any_st(code, var_name)
        if assigns:
            name_lit = next(graph.objects(pou_uri, AG["dp_hasPOUName"]), None)
            out.append(
                {
                    "pou_uri": str(pou_uri),
                    "pou_name": str(name_lit) if name_lit else str(pou_uri),
                    "code": code,
                    "analysis": {
                        "summary": {
                            "var_name": var_name,
                            "assignment_count": len(assigns),
                            "last_assignment_in_code": assigns[-1] if assigns else None,
                        },
                        "assignments": assigns,
                    },
                }
            )
    return out

def _find_call_param_expr(st_code: str, inst: str, param: str) -> Optional[str]:
    rx_start = re.compile(rf"^\s*{re.escape(inst)}\s*\(")
    buf: List[str] = []
    collecting = False
    bal = 0
    for ln in st_code.splitlines():
        if not collecting and rx_start.search(ln):
            collecting = True
        if collecting:
            buf.append(ln.strip())
            bal += ln.count("(") - ln.count(")")
            if ln.strip().endswith(";") and bal <= 0:
                break
    if not buf:
        return None
    joined = " ".join(buf)
    m = re.search(r"\((.*)\)\s*;?$", joined)
    if not m:
        return None
    mm = re.search(rf"\b{re.escape(param)}\s*:=\s*(.+?)(?:,|$)", m.group(1))
    return mm.group(1).strip() if mm else None


def _extract_assignments_from_trace_lines(lines: List[str]) -> List[str]:
    out: List[str] = []
    rx_call = re.compile(r"^(Call-Block|Call-Line)\s*@\s*\d+\s*:\s*(.+)$")
    rx_wire = re.compile(r"^\[KG-WIRE\]\s*(.+?)\s*<=\s*(.+?)\s*\(caller_pou=(.+)\)")
    for raw in lines or []:
        t = str(raw).strip()
        m = rx_call.match(t)
        if m:
            out.append(m.group(2).strip())
            continue
        m = rx_wire.match(t)
        if m:
            out.append(f"{m.group(1).strip()} <= {m.group(2).strip()} (caller={m.group(3).strip()})")
    uniq: List[str] = []
    for x in out:
        if x not in uniq:
            uniq.append(x)
    return uniq


def _phase_to_int(phase: str) -> Optional[int]:
    p = str(phase or "").strip()
    if not p:
        return None
    if ".." in p:
        p = p.split("..", 1)[0].strip()
    if p == "t0":
        return 0
    m = re.match(r"^t(-?\d+)$", p)
    if m:
        return int(m.group(1))
    m = re.match(r"^t\+(\d+)$", p)
    if m:
        return int(m.group(1))
    return None


def _phase_sort_key_reverse(phase: str) -> Tuple[int, int, str]:
    v = _phase_to_int(phase)
    if v is None:
        return (3, 999, str(phase))
    if v == 0:
        return (0, 0, phase)
    if v < 0:
        return (1, abs(v), phase)
    return (2, v, phase)


def _extract_active_exclusive_token(path_clauses: List[str]) -> Optional[str]:
    active = {"Schritt1", "Schritt2"}
    for clause in path_clauses or []:
        c = str(clause).strip()
        if c.upper().startswith("NOT "):
            continue
        toks = extract_variables_from_condition(c) or [c]
        for t in toks:
            if t in active:
                return t
    return None


def _pick_value(symbol: str, values: List[str], *, path_name: str, path_clauses: List[str], state_name: str, last_state: str, event_ctx: Dict[str, Any]) -> str:
    vals = [str(v).strip().upper() for v in values if str(v).strip()]
    uniq = sorted(set(vals))
    if not uniq:
        return "unbekannt"
    if len(uniq) == 1:
        return uniq[0]

    pos = set()
    neg = set()
    for clause in path_clauses or []:
        c = str(clause).strip()
        is_neg = c.upper().startswith("NOT ")
        expr = c[4:].strip() if is_neg else c
        toks = extract_variables_from_condition(expr) or ([expr] if re.fullmatch(r"[A-Za-z_]\w*(?:\.[A-Za-z_]\w*)*", expr) else [])
        for t in toks:
            (neg if is_neg else pos).add(t)

    if symbol in pos:
        return "TRUE"
    if symbol in neg:
        return "FALSE"

    if path_name.startswith("state_path_"):
        if symbol == state_name:
            return "TRUE"
        if symbol in ("D1", "D2", "D3") and symbol != state_name:
            return "FALSE"
        active = _extract_active_exclusive_token(path_clauses)
        if active:
            if symbol in ("Schritt1", "Schritt2"):
                return "TRUE" if symbol == active else "FALSE"
        if symbol == last_state:
            return "TRUE"

    snap = _snap(symbol, event_ctx)
    if snap is not None:
        snap_txt = "TRUE" if str(snap).strip().lower() in ("true", "1", "yes") else ("FALSE" if str(snap).strip().lower() in ("false", "0", "no") else str(snap).strip().upper())
        if snap_txt in uniq:
            return snap_txt

    if "TRUE" in uniq:
        return "TRUE"
    if "FALSE" in uniq:
        return "FALSE"
    return uniq[0]


def _pick_phase(symbol: str, phases: List[str], *, path_name: str, value: str, state_name: str, last_state: str, path_clauses: List[str]) -> str:
    pos = set()
    neg = set()
    for clause in path_clauses or []:
        c = str(clause).strip()
        is_neg = c.upper().startswith("NOT ")
        expr = c[4:].strip() if is_neg else c
        toks = extract_variables_from_condition(expr) or ([expr] if re.fullmatch(r"[A-Za-z_]\w*(?:\.[A-Za-z_]\w*)*", expr) else [])
        for t in toks:
            (neg if is_neg else pos).add(t)

    if symbol in pos or symbol in neg:
        return "t0" if path_name != "skill_path" else "t-1"
    if path_name == "trigger_path":
        return "t0"
    if path_name == "skill_path":
        return "t-1"
    if path_name.startswith("state_path_"):
        if symbol == state_name:
            return "t0"
        if symbol == last_state and str(value).upper() == "TRUE":
            return "t-1"
        if symbol in ("D1", "D2", "D3") and str(value).upper() == "FALSE":
            return "t-1"
    if phases:
        pp = sorted({str(x).strip() for x in phases if str(x).strip()}, key=_phase_sort_key_reverse)
        if pp:
            return pp[0]
    return "t0"


def _rows_to_chain_records(
    rows: Dict[str, Dict[str, Any]],
    path: str,
    default_pou: str,
    event_ctx: Dict[str, Any],
    *,
    state_name: str,
    last_state: str,
    path_clauses: List[str],
) -> List[Dict[str, str]]:
    recs: List[Dict[str, str]] = []
    for sym in sorted(rows.keys()):
        item = rows[sym]
        vals = sorted(item.get("values", set()))
        pous = sorted(item.get("pous", set()))
        reasons = item.get("reasons", [])
        phases = sorted(item.get("phases", set()), key=_phase_sort_key_reverse)

        assignment = "-"
        for r in reasons:
            rr = str(r).strip()
            if "<=" in rr or "Call-" in rr or ":=" in rr:
                assignment = rr
                break

        selected_value = _pick_value(
            sym,
            vals,
            path_name=path,
            path_clauses=path_clauses,
            state_name=state_name,
            last_state=last_state,
            event_ctx=event_ctx,
        )
        selected_phase = _pick_phase(
            sym,
            phases,
            path_name=path,
            value=selected_value,
            state_name=state_name,
            last_state=last_state,
            path_clauses=path_clauses,
        )

        recs.append({
            "Pfad": path,
            "Zeitphase": selected_phase,
            "Port/Variable": sym,
            "Muss-Wert": selected_value,
            "Snapshot-Wert": str(_snap(sym, event_ctx) if _snap(sym, event_ctx) is not None else "-"),
            "POU": ", ".join(pous) if pous else default_pou,
            "Assignment": assignment,
            "Herkunft": " | ".join(reasons[:4]) if reasons else "-",
        })
    return recs

def _format_table(records: List[Dict[str, str]], cols: List[str]) -> str:
    if not records:
        return "(keine Daten)"
    w = {c: len(c) for c in cols}
    for r in records:
        for c in cols:
            w[c] = max(w[c], len(str(r.get(c, ""))))
    def line(ch="-"):
        return "+" + "+".join(ch * (w[c] + 2) for c in cols) + "+"
    out = [line("-"), "| " + " | ".join(c.ljust(w[c]) for c in cols) + " |", line("=")]
    for r in records:
        out.append("| " + " | ".join(str(r.get(c, "")).ljust(w[c]) for c in cols) + " |")
    out.append(line("-"))
    return "\n".join(out)


def _phase_explanation() -> List[Tuple[str, str]]:
    return [
        ("t0", "Zyklus, in dem OPCUA.TriggerD2/Fehler aktiv gesetzt wurde."),
        ("t-1", "Unmittelbar vorheriger Zyklus (z. B. LastGEMMAStateBeforeFailure=F1)."),
        ("t-2, t-3, ...", "Frühere Vorgeschichte entlang der rekursiv gefundenen Call-/Signalpfade."),
        ("t+1, ...", "Nachlauf nach dem Fehlerereignis (falls im Trace vorhanden)."),
    ]


def _is_unknown_like(value: Any) -> bool:
    s = str(value).strip().lower()
    return s in {"", "-", "none", "null", "unknown", "unbekannt", "n/a", "na", "nan"}


def _compact_chain_records(records: List[Dict[str, str]]) -> List[Dict[str, str]]:
    out: List[Dict[str, str]] = []
    for r in records:
        must_val = r.get("Muss-Wert", "")
        snap_val = r.get("Snapshot-Wert", "")
        if _is_unknown_like(must_val) and _is_unknown_like(snap_val):
            continue
        out.append(r)
    return out


def _dedupe(recs: List[Dict[str, str]]) -> List[Dict[str, str]]:
    out: List[Dict[str, str]] = []
    seen = set()
    for r in recs:
        k = (
            r.get("Pfad"),
            r.get("Zeitphase"),
            r.get("Port/Variable"),
            r.get("Muss-Wert"),
            r.get("POU"),
            r.get("Assignment"),
        )
        if k in seen:
            continue
        seen.add(k)
        out.append(r)
    return out


def _apply_consistency_rules(
    recs: List[Dict[str, str]],
    *,
    path_logic: Dict[str, Dict[str, Any]],
    state_name: str,
    last_state: str,
    event_ctx: Dict[str, Any],
) -> Tuple[List[Dict[str, str]], List[str]]:
    drops: List[str] = []
    out: List[Dict[str, str]] = []

    active_by_path: Dict[str, Optional[str]] = {}
    for p, info in path_logic.items():
        active_by_path[p] = _extract_active_exclusive_token(info.get("clauses", []))

    for r in recs:
        path = r.get("Pfad", "")
        sym = r.get("Port/Variable", "")
        val = str(r.get("Muss-Wert", "")).upper()
        phase = str(r.get("Zeitphase", "t0"))
        pi = _phase_to_int(phase)
        drop = False

        if path.startswith("state_path_"):
            if sym in ("D1", "D2", "D3") and pi is not None and pi < 0 and val == "TRUE":
                drops.append(f"{path}:{phase}:{sym}=TRUE entfernt (Vergangenheit -> FALSE)")
                drop = True
            if sym in ("D1", "D2", "D3") and sym != state_name and phase == "t0" and val == "TRUE":
                drops.append(f"{path}:{phase}:{sym}=TRUE entfernt (nicht aktiver Zustand)")
                drop = True
            if sym == last_state and phase == "t0" and val == "TRUE":
                r["Zeitphase"] = "t-1"
            if sym == state_name and pi is not None and pi < 0 and val == "TRUE":
                drops.append(f"{path}:{phase}:{sym}=TRUE entfernt (target-state nur t0)")
                drop = True

            active = active_by_path.get(path)
            if active and sym in ("Schritt1", "Schritt2") and phase == "t0":
                if sym == active and val == "FALSE":
                    drops.append(f"{path}:{phase}:{sym}=FALSE entfernt ({active} muss TRUE sein)")
                    drop = True
                if sym != active and val == "TRUE":
                    drops.append(f"{path}:{phase}:{sym}=TRUE entfernt (mutual exclusion)")
                    drop = True

        if not drop:
            out.append(r)

    for p, info in path_logic.items():
        if not p.startswith("state_path_"):
            continue
        active = active_by_path.get(p)
        pou = "-"
        for r in out:
            if r.get("Pfad") == p and r.get("POU") and r.get("POU") != "-":
                pou = r.get("POU")
                break

        required = [
            (state_name, "TRUE", "t0", f"Zielzustand gesetzt ({state_name}=TRUE)"),
            (state_name, "FALSE", "t-1", "Vor Eintritt in Fehlerzustand war Zielzustand FALSE"),
            ("D1", "FALSE", "t-1", "Vergangenheitsannahme: D1/D2/D3=FALSE"),
            ("D2", "FALSE", "t-1", "Vergangenheitsannahme: D1/D2/D3=FALSE"),
            ("D3", "FALSE", "t-1", "Vergangenheitsannahme: D1/D2/D3=FALSE"),
        ]
        if last_state:
            required.append((last_state, "TRUE", "t-1", "Hint: LastGEMMAStateBeforeFailure"))
        if active in ("Schritt1", "Schritt2"):
            other = "Schritt2" if active == "Schritt1" else "Schritt1"
            required.append((active, "TRUE", "t0", f"Mutual exclusion im Pfad ({active} aktiv)"))
            required.append((other, "FALSE", "t0", f"Mutual exclusion im Pfad ({active} aktiv)"))

        for sym, val, phase, why in required:
            key = (p, phase, sym, val)
            exists = False
            for r in out:
                if (r.get("Pfad"), r.get("Zeitphase"), r.get("Port/Variable"), str(r.get("Muss-Wert")).upper()) == key:
                    exists = True
                    break
            if not exists:
                out.append({
                    "Pfad": p,
                    "Zeitphase": phase,
                    "Port/Variable": sym,
                    "Muss-Wert": val,
                    "Snapshot-Wert": str(_snap(sym, event_ctx) if _snap(sym, event_ctx) is not None else "-"),
                    "POU": pou,
                    "Assignment": "-",
                    "Herkunft": why,
                })

    return _dedupe(out), drops



def _analyze_skill_state_path_fit(
    records: List[Dict[str, str]],
    *,
    path_logic: Dict[str, Dict[str, Any]],
    last_skill: str,
) -> Dict[str, Any]:
    """
    Prüft, welche state_path_* mit dem lastExecutedSkill-Pfad konsistent sein können.
    Vergleichslogik:
      - exact: gleiche Variable, gleicher Wert, gleiche Phase
      - soft : gleiche Variable, gleicher Wert, Phase ±1
      - conflict: gegenteiliger boolescher Wert in gleicher/naher Phase
      - extra Transition-Checks für Schritt1/Schritt2 mit tSchritt1.Q
    """
    state_paths = sorted([p for p in path_logic.keys() if p.startswith("state_path_")], key=_path_order_key)

    skill_rows = [r for r in records if r.get("Pfad") == "skill_path"]
    skill_requirements: List[Dict[str, str]] = []
    seen_req = set()
    for r in skill_rows:
        symbol = str(r.get("Port/Variable", "")).strip()
        value = str(r.get("Muss-Wert", "")).strip().upper()
        phase = str(r.get("Zeitphase", "t-1")).strip() or "t-1"
        if symbol == "OPCUA.lastExecutedSkill":
            continue
        if _is_unknown_like(value):
            continue
        if value not in ("TRUE", "FALSE"):
            continue
        key = (symbol, phase, value)
        if key in seen_req:
            continue
        seen_req.add(key)
        skill_requirements.append(
            {
                "symbol": symbol,
                "value": value,
                "phase": phase,
                "origin": str(r.get("Herkunft", "-")),
            }
        )

    evaluations: List[Dict[str, Any]] = []
    for state_path in state_paths:
        rows = [r for r in records if r.get("Pfad") == state_path]
        by_exact: Dict[Tuple[str, str], set] = {}
        by_symbol: Dict[str, List[Tuple[str, str]]] = {}
        for r in rows:
            sym = str(r.get("Port/Variable", "")).strip()
            val = str(r.get("Muss-Wert", "")).strip().upper()
            ph = str(r.get("Zeitphase", "")).strip()
            if not sym or not ph or val not in ("TRUE", "FALSE"):
                continue
            by_exact.setdefault((sym, ph), set()).add(val)
            by_symbol.setdefault(sym, []).append((ph, val))

        exact = 0
        soft = 0
        conflicts = 0
        unresolved = 0
        details: List[str] = []
        conflict_items: List[str] = []

        for req in skill_requirements:
            sym = req["symbol"]
            req_val = req["value"]
            req_phase = req["phase"]

            exact_vals = by_exact.get((sym, req_phase), set())
            if exact_vals:
                if req_val in exact_vals:
                    exact += 1
                    details.append(f"exact: {sym}={req_val} @ {req_phase}")
                else:
                    conflicts += 1
                    cf = f"{sym}: skill={req_val} @ {req_phase}, state={sorted(exact_vals)}"
                    conflict_items.append(cf)
                    details.append("conflict: " + cf)
                continue

            near_same: Optional[Tuple[int, str]] = None
            near_conflict: Optional[Tuple[int, str]] = None
            req_pi = _phase_to_int(req_phase)
            for ph, v in by_symbol.get(sym, []):
                ph_i = _phase_to_int(ph)
                if req_pi is None or ph_i is None:
                    continue
                dist = abs(ph_i - req_pi)
                if dist > 1:
                    continue
                if v == req_val:
                    if near_same is None or dist < near_same[0]:
                        near_same = (dist, ph)
                else:
                    if near_conflict is None or dist < near_conflict[0]:
                        near_conflict = (dist, ph)

            if near_same is not None and near_conflict is not None:
                # konservativ: wenn gegenteiliger Wert genauso nah oder näher liegt -> Konflikt
                if near_conflict[0] <= near_same[0]:
                    conflicts += 1
                    cf = f"{sym}: skill={req_val} @ {req_phase}, state auch {_bool_invert(req_val)} @ {near_conflict[1]}"
                    conflict_items.append(cf)
                    details.append("conflict: " + cf)
                else:
                    soft += 1
                    details.append(f"soft: {sym}={req_val} @ {near_same[1]} (für skill@{req_phase})")
                continue

            if near_same is not None:
                soft += 1
                details.append(f"soft: {sym}={req_val} @ {near_same[1]} (für skill@{req_phase})")
            elif near_conflict is not None:
                conflicts += 1
                cf = f"{sym}: skill={req_val} @ {req_phase}, state={_bool_invert(req_val)} @ {near_conflict[1]}"
                conflict_items.append(cf)
                details.append("conflict: " + cf)
            else:
                unresolved += 1
                details.append(f"unresolved: {sym}={req_val} @ {req_phase}")

        # Zusatzregel aus ST-Semantik:
        # Wenn im skill_path Schritt1=TRUE und tSchritt1.Q=FALSE @t-1,
        # dann kann Schritt2=TRUE @t0 (Übergang Schritt1->Schritt2) nicht erfolgt sein.
        req_map = {(r["symbol"], r["phase"]): r["value"] for r in skill_requirements}
        req_s1_t1 = req_map.get(("Schritt1", "t-1"))
        req_t1q_t1 = req_map.get(("tSchritt1.Q", "t-1"))
        if req_s1_t1 == "TRUE" and req_t1q_t1 == "FALSE":
            step2_t0 = by_exact.get(("Schritt2", "t0"), set())
            step1_t0 = by_exact.get(("Schritt1", "t0"), set())
            if "TRUE" in step2_t0:
                conflicts += 1
                cf = "Transition-Konflikt: Schritt1=TRUE & tSchritt1.Q=FALSE @t-1 -> Schritt2 kann @t0 nicht TRUE sein"
                conflict_items.append(cf)
                details.append("conflict: " + cf)
            if "FALSE" in step1_t0:
                conflicts += 1
                cf = "Transition-Konflikt: Schritt1=TRUE & tSchritt1.Q=FALSE @t-1 -> Schritt1 sollte @t0 nicht FALSE sein"
                conflict_items.append(cf)
                details.append("conflict: " + cf)

        score = exact * 3 + soft - conflicts * 4
        possible = conflicts == 0
        evaluations.append(
            {
                "state_path": state_path,
                "possible": possible,
                "score": score,
                "exact": exact,
                "soft": soft,
                "conflicts": conflicts,
                "unresolved": unresolved,
                "conflict_items": conflict_items,
                "details": details,
            }
        )

    evaluations.sort(key=lambda e: (not e["possible"], -e["score"], e["state_path"]))
    possible_paths = [e["state_path"] for e in evaluations if e["possible"]]
    best_paths: List[str] = []
    if possible_paths:
        best_score = max(e["score"] for e in evaluations if e["possible"])
        best_paths = [e["state_path"] for e in evaluations if e["possible"] and e["score"] == best_score]

    return {
        "last_skill": last_skill,
        "skill_requirements": skill_requirements,
        "path_evaluations": evaluations,
        "possible_paths": possible_paths,
        "best_paths": best_paths,
    }

def _analyze_trigger_chain(ttl_path: str, trigger_var: str, max_depth: int) -> Dict[str, Any]:
    _require_global("load_graph")
    _require_global("find_pou_by_name")
    _require_global("analyze_var_assignments_st")
    _require_global("trace_expr")
    _require_global("extract_variables_from_condition")
    _require_global("resolve_input_to_upstream_output")
    _require_global("_extract_innermost_if_condition_from_assignment")
    _require_global("parse_bool_expr")
    _require_global("required_truth_paths")

    g = load_graph(ttl_path, trace=None)
    pou_uri = find_pou_by_name(g, "FB_Diagnose_D2", trace=None)
    if not pou_uri:
        return {"error": "FB_Diagnose_D2 nicht gefunden.", "graph": g}
    code = str(next(g.objects(pou_uri, AG["dp_hasPOUCode"]), ""))
    aa = analyze_var_assignments_st(code, trigger_var, trace=None)
    assigns = aa.get("assignments", [])
    chosen = [a for a in assigns if str(a.get("value", "")).upper() == "TRUE"]
    chosen = chosen[-1] if chosen else (assigns[-1] if assigns else None)
    if not chosen:
        return {"error": f"Keine Zuweisung für {trigger_var} in FB_Diagnose_D2.", "graph": g}
    inner = _extract_innermost_if_condition_from_assignment(chosen) or {}
    expr = inner.get("expr", "")
    paths = required_truth_paths(parse_bool_expr(expr), limit=12) if expr else []

    token_traces = []
    seen = set()
    for pidx, p in enumerate(paths, start=1):
        for clause in p:
            neg = clause.strip().upper().startswith("NOT ")
            ce = clause.strip()[4:].strip() if neg else clause.strip()
            ce = ce.strip("() ")
            toks = extract_variables_from_condition(ce) or ([ce] if re.fullmatch(r"[A-Za-z_]\w*(?:\.[A-Za-z_]\w*)*", ce) else [])
            for tok in toks:
                key = (tok, "FALSE" if neg else "TRUE")
                if key in seen:
                    continue
                seen.add(key)
                tl = trace_expr(g, tok, pou_uri, code, max_depth=max_depth)
                token_traces.append({
                    "token": tok,
                    "expected": "FALSE" if neg else "TRUE",
                    "clause": clause,
                    "trace_lines": [x.text for x in tl],
                })

    bridge_expr = _find_call_param_expr(code, "rtReq", "CLK")
    bridge_token = None
    if bridge_expr:
        bt = extract_variables_from_condition(bridge_expr)
        bridge_token = bt[0] if bt else bridge_expr.strip()
    bridge_res = None
    if bridge_token and re.fullmatch(r"[A-Za-z_]\w*", bridge_token):
        bridge_res = resolve_input_to_upstream_output(g, pou_uri, bridge_token, trace=None)

    return {
        "graph": g,
        "setter_pou_name": "FB_Diagnose_D2",
        "setter_pou_uri": str(pou_uri),
        "dominant_assignment": chosen,
        "condition_paths": paths,
        "token_traces": token_traces,
        "bridge_token": bridge_token,
        "bridge_resolution": bridge_res,
    }


def _normalize_skill_literal(rhs: str) -> str:
    s = str(rhs or "").strip()
    if len(s) >= 2 and ((s.startswith("'") and s.endswith("'")) or (s.startswith('"') and s.endswith('"'))):
        s = s[1:-1]
    return s.strip().lower()


def _condition_entry_to_expr(entry: str) -> Optional[str]:
    e = str(entry or "").strip()
    if not e:
        return None
    m_if = re.match(r"^(?:IF|ELSIF)@\d+\s*:\s*(.+)$", e, flags=re.I)
    if m_if:
        return m_if.group(1).strip()
    m_else = re.match(r"^ELSE@\d+\s*\(zu IF:\s*(.+)\)$", e, flags=re.I)
    if m_else:
        cond = m_else.group(1).strip()
        return f"NOT ({cond})"
    return None


def _assignment_execution_expr(assignment: Dict[str, Any]) -> Optional[str]:
    conds = assignment.get("conditions", []) or []
    parts: List[str] = []
    for c in conds:
        expr = _condition_entry_to_expr(c)
        if expr:
            parts.append(f"({expr})")
    if not parts:
        return None
    return " AND ".join(parts)


def _truth_paths_expr(expr_text: str, *, limit: int = 8) -> List[List[str]]:
    _require_global("parse_bool_expr")
    expr_text = str(expr_text or "").strip()
    if not expr_text:
        return []
    try:
        tree = parse_bool_expr(expr_text)
    except Exception:
        return [[expr_text]]

    def _cross(base_paths: List[List[str]], child_paths: List[List[str]]) -> List[List[str]]:
        if not base_paths:
            return child_paths[:limit]
        if not child_paths:
            return base_paths[:limit]
        out: List[List[str]] = []
        for b in base_paths:
            for c in child_paths:
                out.append(b + c)
                if len(out) >= limit:
                    return out
        return out

    def _pos(node) -> List[List[str]]:
        kind = getattr(node, "kind", "")
        if kind == "atom":
            return [[getattr(node, "value", "") or ""]]
        if kind == "and":
            paths: List[List[str]] = [[]]
            for ch in getattr(node, "children", ()) or ():
                paths = _cross(paths, _pos(ch))
                if len(paths) >= limit:
                    paths = paths[:limit]
            return paths[:limit]
        if kind == "or":
            out: List[List[str]] = []
            for ch in getattr(node, "children", ()) or ():
                out.extend(_pos(ch))
                if len(out) >= limit:
                    break
            return out[:limit]
        if kind == "not":
            children = getattr(node, "children", ()) or ()
            if not children:
                return []
            return _neg(children[0])[:limit]
        return [[expr_text]]

    def _neg(node) -> List[List[str]]:
        kind = getattr(node, "kind", "")
        if kind == "atom":
            return [[f"NOT {getattr(node, 'value', '')}"]]
        if kind == "not":
            children = getattr(node, "children", ()) or ()
            if not children:
                return []
            return _pos(children[0])[:limit]
        if kind == "and":
            # NOT (A AND B) = (NOT A) OR (NOT B)
            out: List[List[str]] = []
            for ch in getattr(node, "children", ()) or ():
                out.extend(_neg(ch))
                if len(out) >= limit:
                    break
            return out[:limit]
        if kind == "or":
            # NOT (A OR B) = (NOT A) AND (NOT B)
            paths: List[List[str]] = [[]]
            for ch in getattr(node, "children", ()) or ():
                paths = _cross(paths, _neg(ch))
                if len(paths) >= limit:
                    paths = paths[:limit]
            return paths[:limit]
        return [[f"NOT ({expr_text})"]]

    out = _pos(tree)
    return out[:limit] if out else [[expr_text]]


def _condition_entry_paths(cnd: str, *, limit: int = 8) -> List[List[str]]:
    expr = _condition_entry_to_expr(cnd)
    if not expr:
        return []
    return _truth_paths_expr(expr, limit=limit)


def _clause_to_literal(clause: str) -> Optional[Tuple[str, bool]]:
    _require_global("extract_variables_from_condition")
    c = str(clause or "").strip()
    if not c:
        return None
    neg = False
    if c.upper().startswith("NOT "):
        neg = True
        c = c[4:].strip()
    while c.startswith("(") and c.endswith(")"):
        inner = c[1:-1].strip()
        if not inner:
            break
        c = inner
    toks = extract_variables_from_condition(c) or []
    if len(toks) == 1 and toks[0] == c:
        return (c, not neg)
    if re.fullmatch(r"[A-Za-z_]\w*(?:\.[A-Za-z_]\w*)*", c):
        return (c, not neg)
    return None


def _path_literals(path: List[str]) -> Optional[Dict[str, bool]]:
    lits: Dict[str, bool] = {}
    for clause in path:
        lit = _clause_to_literal(clause)
        if not lit:
            continue
        sym, val = lit
        if sym in lits and lits[sym] != val:
            return None
        lits[sym] = val
    return lits


def _paths_may_overlap(paths_a: List[List[str]], paths_b: List[List[str]]) -> bool:
    for pa in paths_a or []:
        la = _path_literals(pa)
        if la is None:
            continue
        for pb in paths_b or []:
            lb = _path_literals(pb)
            if lb is None:
                continue
            conflict = False
            for k, va in la.items():
                vb = lb.get(k)
                if vb is not None and vb != va:
                    conflict = True
                    break
            if not conflict:
                return True
    return False


def _analyze_last_skill_chain(g, last_skill: str, preferred_pou: str, process_name: str, max_depth: int) -> Dict[str, Any]:
    _require_global("trace_expr")
    _require_global("extract_variables_from_condition")

    if not last_skill:
        return {"error": "last_skill leer"}

    candidates = find_pou_setters_for_variable(g, "OPCUA.lastExecutedSkill")
    target_norm = _normalize_skill_literal(last_skill)
    scored = []
    for c in candidates:
        assigns = c["analysis"].get("assignments", [])
        exact = [a for a in assigns if _normalize_skill_literal(a.get("rhs", "")) == target_norm]
        if not exact:
            fuzzy = [a for a in assigns if target_norm and target_norm in _normalize_skill_literal(a.get("rhs", ""))]
            exact = fuzzy
        if not exact:
            continue
        score = 0
        code = c.get("code", "")
        if preferred_pou and c.get("pou_name") == preferred_pou:
            score += 30
        if process_name and process_name in code:
            score += 20
        if "rStep1" in code:
            score += 10
        score += len(exact)
        scored.append((score, c, exact[-1], assigns))

    if not scored:
        return {"error": f"Kein Setter für lastExecutedSkill='{last_skill}' gefunden."}
    scored.sort(key=lambda x: x[0], reverse=True)
    _, c, chosen, all_assigns = scored[0]
    pou_uri = URIRef(c["pou_uri"])
    code = c["code"]

    toggle_hint = None
    line_no = int(chosen.get("line_no", 0) or 0)
    lines = code.splitlines()
    rx_toggle = re.compile(r"^\s*lastSkillIsOne\s*:=\s*NOT\s+lastSkillIsOne\s*;\s*$", re.I)
    for i in range(min(line_no - 1, len(lines)), 0, -1):
        if rx_toggle.match(lines[i - 1] or ""):
            toggle_hint = {"line_no": i, "assignment": lines[i - 1].strip(), "pre": "FALSE", "post": "TRUE"}
            break

    cond_paths = [[]]
    cond_entries = chosen.get("conditions", []) or []
    for cnd in cond_entries:
        pths = _condition_entry_paths(cnd, limit=8)
        if not pths:
            continue
        newp = []
        for base in cond_paths:
            for pp in pths:
                newp.append(base + pp)
                if len(newp) >= 30:
                    break
            if len(newp) >= 30:
                break
        cond_paths = newp if newp else cond_paths

    chosen_rhs_norm = _normalize_skill_literal(chosen.get("rhs", ""))
    later = [a for a in all_assigns if int(a.get("line_no", 0) or 0) > line_no]
    overwrite_blockers: List[Dict[str, Any]] = []
    hard_conflict = False

    def _score_neg_candidate(base_path: List[str], neg_path: List[str]) -> int:
        base_lits = _path_literals(base_path) or {}
        cand_lits = _path_literals(neg_path) or {}
        score = 0
        for sym, v in cand_lits.items():
            if sym in base_lits and base_lits[sym] != v:
                return -10_000
            score += 2
            if sym.endswith(".Q") and not v:
                score += 4
            if sym.lower().startswith("t") and sym.endswith(".Q") and not v:
                score += 2
            if "Stoerung" in sym and v:
                score -= 2
        score -= max(0, len(cand_lits) - 1)
        return score

    for a in later:
        rhs_norm = _normalize_skill_literal(a.get("rhs", ""))
        if rhs_norm == chosen_rhs_norm:
            continue

        exec_expr = _assignment_execution_expr(a)
        if not exec_expr:
            hard_conflict = True
            overwrite_blockers.append(
                {
                    "line_no": a.get("line_no"),
                    "rhs": a.get("rhs"),
                    "execution_expr": None,
                    "neg_paths": [],
                    "note": "unbedingte spätere Zuweisung überschreibt chosen skill",
                }
            )
            continue

        exec_paths = _truth_paths_expr(exec_expr, limit=12)
        if not _paths_may_overlap(cond_paths, exec_paths):
            overwrite_blockers.append(
                {
                    "line_no": a.get("line_no"),
                    "rhs": a.get("rhs"),
                    "execution_expr": exec_expr,
                    "neg_paths": [],
                    "note": "spätere Zuweisung ist bereits durch chosen-Bedingung ausgeschlossen",
                }
            )
            continue

        neg_expr = f"NOT ({exec_expr})"
        neg_paths = _truth_paths_expr(neg_expr, limit=8)
        if not neg_paths:
            neg_paths = [[neg_expr]]

        newp: List[List[str]] = []
        selected_neg_paths: List[List[str]] = []
        for base in cond_paths:
            best_np = None
            best_score = -10_000
            for np in neg_paths:
                sc = _score_neg_candidate(base, np)
                if sc > best_score:
                    best_score = sc
                    best_np = np
            if best_np is None:
                continue
            merged = base + best_np
            if _path_literals(merged) is None:
                continue
            newp.append(merged)
            selected_neg_paths.append(best_np)
            if len(newp) >= 30:
                break

        if newp:
            cond_paths = newp
        overwrite_blockers.append(
            {
                "line_no": a.get("line_no"),
                "rhs": a.get("rhs"),
                "execution_expr": exec_expr,
                "neg_paths": selected_neg_paths if selected_neg_paths else neg_paths,
            }
        )

    if hard_conflict:
        return {
            "error": f"lastExecutedSkill='{last_skill}' wird später unbedingt überschrieben.",
            "setter_pou_name": c["pou_name"],
            "setter_pou_uri": c["pou_uri"],
            "dominant_assignment": chosen,
            "overwrite_blockers": overwrite_blockers,
        }

    # R_TRIG-Heuristik: Wenn Instanz.Q=TRUE gefordert ist, dann muss CLK im selben Zyklus TRUE sein.
    enriched_paths: List[List[str]] = []
    for p in cond_paths:
        lits = _path_literals(p) or {}
        extra: List[str] = []
        conflict = False
        for sym, val in lits.items():
            if not val:
                continue
            if not sym.endswith(".Q") or "." not in sym:
                continue
            inst = sym.split(".", 1)[0]
            clk_expr = _find_call_param_expr(code, inst, "CLK")
            if not clk_expr:
                continue
            if not re.fullmatch(r"[A-Za-z_]\w*(?:\.[A-Za-z_]\w*)*", clk_expr):
                continue
            lit = _clause_to_literal(clk_expr)
            if not lit:
                continue
            token, req_true = lit
            if token in lits and lits[token] != req_true:
                conflict = True
                break
            if token not in lits:
                extra.append(clk_expr if req_true else f"NOT {clk_expr}")
        if conflict:
            continue
        enriched_paths.append(p + extra)
        if len(enriched_paths) >= 30:
            break
    if enriched_paths:
        cond_paths = enriched_paths

    # final path cleanup: inkonsistente / doppelte paths entfernen
    cleaned_paths: List[List[str]] = []
    seen_paths = set()
    for p in cond_paths:
        if _path_literals(p) is None:
            continue
        pp = tuple(p)
        if pp in seen_paths:
            continue
        seen_paths.add(pp)
        cleaned_paths.append(list(pp))
    cond_paths = cleaned_paths[:30] if cleaned_paths else cond_paths[:30]

    token_traces = []
    seen = set()
    for p in cond_paths:
        for clause in p:
            neg = clause.strip().upper().startswith("NOT ")
            ce = clause.strip()[4:].strip() if neg else clause.strip()
            ce = ce.strip("() ")
            toks = extract_variables_from_condition(ce) or ([ce] if re.fullmatch(r"[A-Za-z_]\w*(?:\.[A-Za-z_]\w*)*", ce) else [])
            for tok in toks:
                key = (tok, "FALSE" if neg else "TRUE")
                if key in seen:
                    continue
                seen.add(key)
                tl = trace_expr(g, tok, pou_uri, code, max_depth=max_depth)
                token_traces.append(
                    {
                        "token": tok,
                        "expected": "FALSE" if neg else "TRUE",
                        "clause": clause,
                        "trace_lines": [x.text for x in tl],
                    }
                )

    return {
        "setter_pou_name": c["pou_name"],
        "setter_pou_uri": c["pou_uri"],
        "dominant_assignment": chosen,
        "condition_paths": cond_paths,
        "token_traces": token_traces,
        "toggle_hint": toggle_hint,
        "overwrite_blockers": overwrite_blockers,
    }

def _path_order_key(path_name: str) -> Tuple[int, int, str]:
    if path_name == "trigger_path":
        return (0, 0, path_name)
    if path_name.startswith("state_path_"):
        m = re.match(r"^state_path_(\d+)$", path_name)
        idx = int(m.group(1)) if m else 999
        return (1, idx, path_name)
    if path_name == "skill_path":
        return (2, 0, path_name)
    if path_name.startswith("skill_case_"):
        return (3, 0, path_name)
    return (4, 0, path_name)


def _append_skill_records(
    recs: List[Dict[str, str]],
    *,
    path_name: str,
    skill_label: str,
    skill_report: Dict[str, Any],
    event_ctx: Dict[str, Any],
) -> None:
    if "error" in skill_report:
        return

    recs.append({
        "Pfad": path_name,
        "Zeitphase": "t-1",
        "Port/Variable": "OPCUA.lastExecutedSkill",
        "Muss-Wert": skill_label,
        "Snapshot-Wert": str(_snap("OPCUA.lastExecutedSkill", event_ctx) if _snap("OPCUA.lastExecutedSkill", event_ctx) is not None else "-"),
        "POU": skill_report.get("setter_pou_name", "-"),
        "Assignment": str(skill_report.get("dominant_assignment", {}).get("assignment", "-")).strip(),
        "Herkunft": f"Skill-Setter-Pfad ({skill_label})",
    })

    th = skill_report.get("toggle_hint")
    if isinstance(th, dict):
        recs.append({
            "Pfad": path_name,
            "Zeitphase": "t-2",
            "Port/Variable": "lastSkillIsOne",
            "Muss-Wert": str(th.get("pre", "FALSE")),
            "Snapshot-Wert": str(_snap("lastSkillIsOne", event_ctx) if _snap("lastSkillIsOne", event_ctx) is not None else "-"),
            "POU": skill_report.get("setter_pou_name", "-"),
            "Assignment": str(th.get("assignment", "-")),
            "Herkunft": "Vor Toggle (lastSkillIsOne := NOT lastSkillIsOne)",
        })
        recs.append({
            "Pfad": path_name,
            "Zeitphase": "t-1",
            "Port/Variable": "lastSkillIsOne",
            "Muss-Wert": str(th.get("post", "TRUE")),
            "Snapshot-Wert": str(_snap("lastSkillIsOne", event_ctx) if _snap("lastSkillIsOne", event_ctx) is not None else "-"),
            "POU": skill_report.get("setter_pou_name", "-"),
            "Assignment": str(th.get("assignment", "-")),
            "Herkunft": "Nach Toggle (Pfad zur Skill-Zuweisung)",
        })

    for t in skill_report.get("token_traces", []):
        assigns = _extract_assignments_from_trace_lines(t.get("trace_lines", []))
        tok = t.get("token", "")
        recs.append({
            "Pfad": path_name,
            "Zeitphase": "t-1",
            "Port/Variable": tok,
            "Muss-Wert": t.get("expected", "TRUE"),
            "Snapshot-Wert": str(_snap(tok, event_ctx) if _snap(tok, event_ctx) is not None else "-"),
            "POU": skill_report.get("setter_pou_name", "-"),
            "Assignment": " || ".join(assigns[:3]) if assigns else "-",
            "Herkunft": f"Skill-Klausel: {t.get('clause', '-')}",
        })


def build_complete_signal_chain_report(
    ttl_path: str,
    *,
    event_json_path: str = "",
    trigger_var: str = "OPCUA.TriggerD2",
    state_name: str = "D2",
    last_gemma_state_before_failure: str = "",
    last_skill: str = "",
    skill_case: str = "TestSkill3",
    gemma_pou_name: str = "",
    max_depth: int = 40,
    max_truth_paths: int = 8,
    verbose_core_trace: bool = False,
) -> Dict[str, Any]:
    _require_global("build_concrete_d2_error_paths")

    event_ctx = _load_event_context(event_json_path)
    if not last_skill:
        last_skill = str(event_ctx.get("last_skill") or "")
    if not last_gemma_state_before_failure:
        last_gemma_state_before_failure = str(event_ctx.get("last_gemma_state") or "F1")

    trigger = _analyze_trigger_chain(ttl_path, trigger_var, max_depth=max_depth)
    d2 = build_concrete_d2_error_paths(
        ttl_path=ttl_path,
        state_name=state_name,
        last_gemma_state_before_failure=last_gemma_state_before_failure,
        gemma_pou_name=gemma_pou_name,
        max_depth=max_depth,
        max_truth_paths=max_truth_paths,
        verbose_core_trace=verbose_core_trace,
    )

    g = trigger.get("graph")
    preferred_pou = d2.get("context", {}).get("target_pou", "") if isinstance(d2, dict) and "error" not in d2 else ""
    skill = _analyze_last_skill_chain(g, last_skill, preferred_pou, str(event_ctx.get("process_name") or ""), max_depth=max_depth) if g else {"error": "kein Graph"}
    skill_case_key = f"skill_case_{re.sub(r'[^A-Za-z0-9_]+', '_', str(skill_case or 'TestSkill3'))}"
    skill_case_report = _analyze_last_skill_chain(g, str(skill_case or "TestSkill3"), preferred_pou, str(event_ctx.get("process_name") or ""), max_depth=max_depth) if g else {"error": "kein Graph"}

    path_logic: Dict[str, Dict[str, Any]] = {}
    if "error" not in trigger:
        path_logic["trigger_path"] = {
            "expr": trigger.get("inner_if_expr", ""),
            "clauses": trigger.get("condition_paths", [[]])[0] if trigger.get("condition_paths") else [],
        }
    if "error" not in skill:
        first_skill = skill.get("condition_paths", [[]])[0] if skill.get("condition_paths") else []
        path_logic["skill_path"] = {
            "expr": " AND ".join(first_skill),
            "clauses": first_skill,
        }
    if "error" not in skill_case_report:
        first_skill_case = skill_case_report.get("condition_paths", [[]])[0] if skill_case_report.get("condition_paths") else []
        path_logic[skill_case_key] = {
            "expr": " AND ".join(first_skill_case),
            "clauses": first_skill_case,
        }
    if isinstance(d2, dict) and "error" not in d2:
        for p in d2.get("paths", []):
            path_name = f"state_path_{p.get('path_index')}"
            clauses = p.get("clause_path", [])
            path_logic[path_name] = {"expr": " AND ".join(clauses), "clauses": clauses}

    recs: List[Dict[str, str]] = []

    if "error" not in trigger:
        recs.append({
            "Pfad": "trigger_path",
            "Zeitphase": "t0",
            "Port/Variable": trigger_var,
            "Muss-Wert": "TRUE",
            "Snapshot-Wert": str(_snap(trigger_var, event_ctx) if _snap(trigger_var, event_ctx) is not None else "-"),
            "POU": trigger.get("setter_pou_name", "-"),
            "Assignment": str(trigger.get("dominant_assignment", {}).get("assignment", "-")).strip(),
            "Herkunft": "Dominante Trigger-Zuweisung",
        })
        for t in trigger.get("token_traces", []):
            assigns = _extract_assignments_from_trace_lines(t.get("trace_lines", []))
            tok = t.get("token", "")
            recs.append({
                "Pfad": "trigger_path",
                "Zeitphase": "t0",
                "Port/Variable": tok,
                "Muss-Wert": t.get("expected", "TRUE"),
                "Snapshot-Wert": str(_snap(tok, event_ctx) if _snap(tok, event_ctx) is not None else "-"),
                "POU": trigger.get("setter_pou_name", "-"),
                "Assignment": " || ".join(assigns[:3]) if assigns else "-",
                "Herkunft": f"Trigger-Klausel: {t.get('clause', '-')}",
            })
        br = trigger.get("bridge_resolution")
        if isinstance(br, dict) and not br.get("error"):
            assign = f"{br.get('input_port_name','?')} <= {br.get('caller_pou_name','?')}.{br.get('caller_port_name','?')}"
            if br.get("assignment"):
                assign += f" [{br.get('assignment')}]"
            bt = trigger.get("bridge_token") or br.get("input_port_name", "")
            recs.append({
                "Pfad": "trigger_path",
                "Zeitphase": "t0",
                "Port/Variable": bt,
                "Muss-Wert": "TRUE",
                "Snapshot-Wert": str(_snap(bt, event_ctx) if _snap(bt, event_ctx) is not None else "-"),
                "POU": trigger.get("setter_pou_name", "-"),
                "Assignment": assign,
                "Herkunft": "Bridge rtReq.CLK -> Diagnose_gefordert",
            })
            recs.append({
                "Pfad": "trigger_path",
                "Zeitphase": "t0",
                "Port/Variable": br.get("caller_port_name", ""),
                "Muss-Wert": "TRUE",
                "Snapshot-Wert": str(_snap(br.get("caller_port_name", ""), event_ctx) if _snap(br.get("caller_port_name", ""), event_ctx) is not None else "-"),
                "POU": br.get("caller_pou_name", "-"),
                "Assignment": assign,
                "Herkunft": "Upstream-Port laut KG-Verdrahtung",
            })

    _append_skill_records(
        recs,
        path_name="skill_path",
        skill_label=last_skill,
        skill_report=skill,
        event_ctx=event_ctx,
    )
    _append_skill_records(
        recs,
        path_name=skill_case_key,
        skill_label=str(skill_case or "TestSkill3"),
        skill_report=skill_case_report,
        event_ctx=event_ctx,
    )

    if isinstance(d2, dict) and "error" not in d2:
        default_pou = d2.get("context", {}).get("target_pou", "-")
        for p in d2.get("paths", []):
            path_label = f"state_path_{p.get('path_index')}"
            recs.extend(
                _rows_to_chain_records(
                    p.get("rows", {}),
                    path_label,
                    default_pou,
                    event_ctx,
                    state_name=state_name,
                    last_state=last_gemma_state_before_failure,
                    path_clauses=p.get("clause_path", []),
                )
            )

    recs = _dedupe(recs)
    recs, drops = _apply_consistency_rules(
        recs,
        path_logic=path_logic,
        state_name=state_name,
        last_state=last_gemma_state_before_failure,
        event_ctx=event_ctx,
    )

    recs.sort(
        key=lambda r: (
            _path_order_key(r.get("Pfad", "")),
            _phase_sort_key_reverse(r.get("Zeitphase", "t0")),
            r.get("Port/Variable", ""),
        )
    )

    compact_recs = _compact_chain_records(recs)
    skill_state_fit = _analyze_skill_state_path_fit(
        recs,
        path_logic=path_logic,
        last_skill=last_skill,
    )

    return {
        "context": {
            "ttl_path": ttl_path,
            "event_json_path": event_json_path,
            "event_type": event_ctx.get("event_type"),
            "trigger_event": event_ctx.get("trigger_event"),
            "process_name": event_ctx.get("process_name"),
            "trigger_var": trigger_var,
            "state_name": state_name,
            "last_skill": last_skill,
            "skill_case": str(skill_case or "TestSkill3"),
            "last_gemma_state_before_failure": last_gemma_state_before_failure,
            "snapshot_trigger_d2": event_ctx.get("trigger_d2"),
        },
        "event_context": event_ctx,
        "trigger_report": trigger,
        "skill_report": skill,
        "skill_case_report": skill_case_report,
        "d2_report": d2,
        "path_logic": path_logic,
        "phase_explanation": _phase_explanation(),
        "consistency_drops": drops,
        "skill_state_path_fit": skill_state_fit,
        "chain_records": recs,
        "compact_chain_records": compact_recs,
    }

def print_complete_signal_chain_report(report: Dict[str, Any], *, max_rows_per_path: int = 120) -> None:
    if "error" in report:
        print("[ERROR]", report["error"])
        return
    ctx = report.get("context", {})
    print("\n=== KONTEXT ===")
    print("Event:", ctx.get("event_type"), "| TriggerEvent:", ctx.get("trigger_event"), "| Process:", ctx.get("process_name"))
    print("Trigger:", ctx.get("trigger_var"), "| State:", ctx.get("state_name"))
    print("lastSkill:", ctx.get("last_skill"), "| LastGEMMAState:", ctx.get("last_gemma_state_before_failure"))
    print("Skill-Case:", ctx.get("skill_case"))
    print("Snapshot TriggerD2:", ctx.get("snapshot_trigger_d2"))

    skill_rep = report.get("skill_report", {})
    if isinstance(skill_rep, dict) and skill_rep.get("error"):
        print("[WARN] skill_path:", skill_rep.get("error"))
    skill_case_rep = report.get("skill_case_report", {})
    if isinstance(skill_case_rep, dict) and skill_case_rep.get("error"):
        print("[WARN] skill_case_path:", skill_case_rep.get("error"))

    phase_info = report.get("phase_explanation", [])
    if phase_info:
        print("\n=== ZEITPHASEN-ERKLÄRUNG ===")
        for ph, txt in phase_info:
            print(f"- {ph}: {txt}")

    path_logic = report.get("path_logic", {})
    if path_logic:
        print("\n=== LOGIKKETTEN (rückwärts) ===")
        for p in sorted(path_logic.keys(), key=_path_order_key):
            info = path_logic[p]
            expr = info.get("expr") or "n/a"
            print(f"- {p}: {expr}")

    fit = report.get("skill_state_path_fit", {})
    evals = fit.get("path_evaluations", []) if isinstance(fit, dict) else []
    if evals:
        print("\n=== SKILL -> STATE-PFAD ABGLEICH ===")
        print("Skill-Hinweis:", fit.get("last_skill"), "| Skill-Anforderungen:", len(fit.get("skill_requirements", [])))
        reqs = fit.get("skill_requirements", [])
        if reqs:
            for rq in reqs[:12]:
                print(f"- {rq.get('symbol')}={rq.get('value')} @ {rq.get('phase')}")
            if len(reqs) > 12:
                print(f"- ... weitere {len(reqs)-12} Anforderungen")

        eval_rows: List[Dict[str, str]] = []
        for e in evals:
            eval_rows.append({
                "state_path": str(e.get("state_path")),
                "möglich": "ja" if e.get("possible") else "nein",
                "score": str(e.get("score")),
                "exact": str(e.get("exact")),
                "soft": str(e.get("soft")),
                "conflicts": str(e.get("conflicts")),
                "unresolved": str(e.get("unresolved")),
                "Konflikte": "; ".join((e.get("conflict_items") or [])[:2]) if e.get("conflict_items") else "-",
            })
        print(_format_table(eval_rows, ["state_path", "möglich", "score", "exact", "soft", "conflicts", "unresolved", "Konflikte"]))

        best_paths = fit.get("best_paths", [])
        if best_paths:
            print("=> Beste konsistente State-Paths:", ", ".join(best_paths))
        else:
            print("=> Keine konsistente State-Path gefunden.")

    drops = report.get("consistency_drops", [])
    if drops:
        print("\n=== KONSISTENZ-FILTER ===")
        for d in drops[:40]:
            print("-", d)
        if len(drops) > 40:
            print(f"- ... weitere {len(drops)-40} Einträge")

    recs = report.get("chain_records", [])
    if not recs:
        print("(keine Kettendaten)")
        return

    grouped: Dict[str, List[Dict[str, str]]] = {}
    for r in recs:
        grouped.setdefault(r.get("Pfad", "unknown"), []).append(r)

    cols = ["Pfad", "Zeitphase", "Port/Variable", "Muss-Wert", "Snapshot-Wert", "POU", "Assignment", "Herkunft"]
    for k in sorted(grouped.keys(), key=_path_order_key):
        rows = grouped[k]
        rows = sorted(rows, key=lambda r: (_phase_sort_key_reverse(r.get("Zeitphase", "t0")), r.get("Port/Variable", "")))
        rows = rows[:max_rows_per_path]
        print(f"\n=== {k} ===")
        print(_format_table(rows, cols))

    compact = report.get("compact_chain_records", [])
    if compact:
        print("\n=== KOMPAKTAUSGABE (nur informative Werte) ===")
        grouped_c: Dict[str, List[Dict[str, str]]] = {}
        for r in compact:
            grouped_c.setdefault(r.get("Pfad", "unknown"), []).append(r)
        compact_cols = ["Pfad", "Zeitphase", "Port/Variable", "Muss-Wert", "Snapshot-Wert", "POU", "Assignment"]
        for k in sorted(grouped_c.keys(), key=_path_order_key):
            rows = grouped_c[k]
            rows = sorted(rows, key=lambda r: (_phase_sort_key_reverse(r.get("Zeitphase", "t0")), r.get("Port/Variable", "")))
            rows = rows[:max_rows_per_path]
            print(f"\n--- {k} (kompakt) ---")
            print(_format_table(rows, compact_cols))

# -----------------------------
# Ausführung
# -----------------------------
TTL_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl"
EVENT_PATH = r"D:\MA_Python_Agent\MSRGuard_Anpassung\python\agent_results\evD2-1744012170567400_event.json"

complete_chain_report = build_complete_signal_chain_report(
    ttl_path=TTL_PATH,
    event_json_path=EVENT_PATH,
    trigger_var="OPCUA.TriggerD2",
    state_name="D2",
    last_gemma_state_before_failure="",
    last_skill="",
    gemma_pou_name="",
    max_depth=40,
    max_truth_paths=8,
    verbose_core_trace=False,
)

print_complete_signal_chain_report(complete_chain_report, max_rows_per_path=120)



=== KONTEXT ===
Event: None | TriggerEvent: None | Process: None
Trigger: OPCUA.TriggerD2 | State: D2
lastSkill:  | LastGEMMAState: F1
Skill-Case: TestSkill3
Snapshot TriggerD2: None
[WARN] skill_path: last_skill leer

=== ZEITPHASEN-ERKLÄRUNG ===
- t0: Zyklus, in dem OPCUA.TriggerD2/Fehler aktiv gesetzt wurde.
- t-1: Unmittelbar vorheriger Zyklus (z. B. LastGEMMAStateBeforeFailure=F1).
- t-2, t-3, ...: Frühere Vorgeschichte entlang der rekursiv gefundenen Call-/Signalpfade.
- t+1, ...: Nachlauf nach dem Fehlerereignis (falls im Trace vorhanden).

=== LOGIKKETTEN (rückwärts) ===
- trigger_path: n/a
- state_path_1: Schritt1 AND pPer.Q
- state_path_2: Schritt2 AND pPer.Q
- skill_case_TestSkill3: rStep1.Q AND lastSkillIsOne AND NOT tSchritt1.Q AND Schritt1

=== SKILL -> STATE-PFAD ABGLEICH ===
Skill-Hinweis:  | Skill-Anforderungen: 0
+--------------+---------+-------+-------+------+-----------+------------+-----------+
| state_path   | möglich | score | exact | soft | conflicts | unresol